In [1]:
// ==== Swing ====
import javax.swing.*;
import javax.swing.border.*;
import javax.swing.event.TableModelEvent;
import javax.swing.filechooser.FileNameExtensionFilter;
import javax.swing.table.DefaultTableModel;
import javax.swing.border.EmptyBorder;



// ==== AWT (sélectif pour éviter le conflit avec java.util.List) ====
import java.awt.*; 
import java.awt.AlphaComposite;
import java.awt.BasicStroke;
import java.awt.BorderLayout;
import java.awt.Color;
import java.awt.Dimension;
import java.awt.FlowLayout;
import java.awt.Graphics2D;
import java.awt.Rectangle;
import java.awt.RenderingHints;
import java.awt.Toolkit;
import java.awt.event.*;
import java.awt.geom.*;

// ==== Collections & util ====
import java.util.ArrayList;
import java.util.LinkedHashMap;
import java.util.LinkedHashSet;
import java.util.List;
import java.util.Map;
import java.util.Set;
import java.util.concurrent.atomic.AtomicReference;

// ==== IO ====
import java.io.File;
import java.io.FileInputStream;
import java.io.FileOutputStream;
import java.io.ObjectInputStream;
import java.io.ObjectOutputStream;
import java.nio.charset.StandardCharsets;

import java.awt.Component;
import java.awt.Font;


In [2]:
Thread.setDefaultUncaughtExceptionHandler((t, e) -> e.printStackTrace());

// --- Types d'outils
enum Tool { SELECT, RECT, OVAL, LINE }

// --- Élément géométrique stocké et dessiné
enum ShapeKind { RECT, OVAL, LINE, POLY }

class ShapeItem {
    final String id;           // ex: r1, ov2, l3
    final ShapeKind kind;      // RECT/OVAL/LINE
    java.awt.Shape shape;
    java.awt.Color color = java.awt.Color.DARK_GRAY;
    boolean selected = false;

    ShapeItem(String id, ShapeKind kind, java.awt.Shape s){
        this.id = id; this.kind = kind; this.shape = s;
    }
}

// ===== Globals ==============================================================
class Globals {

    // ---------------- Paramètres (onglet Paramètres) ----------------
    static javax.swing.table.DefaultTableModel paramsModel;

    // ---------------- Géométrie / Domaines ----------------
    enum CoordSys { CARTESIAN_2D, AXISYM_2D, POLAR_2D, Q2D_STELLARATOR }
    static CoordSys currentCoordSys = CoordSys.CARTESIAN_2D;

    // --- Gestion robuste des listeners de changement de repère ---
    private static final java.util.concurrent.CopyOnWriteArrayList<Runnable> coordListeners =
            new java.util.concurrent.CopyOnWriteArrayList<>();

    static void addCoordListener(Runnable r){ if (r != null) coordListeners.add(r); }
    static void removeCoordListener(Runnable r){ if (r != null) coordListeners.remove(r); }
    // Compat (ancien nom)
    static void onCoordSysChange(Runnable r){ addCoordListener(r); }

    static void fireCoordChange(){
        for (Runnable r : coordListeners){
            try { r.run(); } catch (Throwable t) {
                // Listener obsolète (ex: instance détruite via JShell) -> nettoyage
                coordListeners.remove(r);
            }
        }
        repaintAll();
    }

    static void clearCoordListeners(){ coordListeners.clear(); }

    static void setCoordSys(CoordSys cs){
        if (cs != null && currentCoordSys != cs){
            currentCoordSys = cs;
            fireCoordChange();
        }
    }

    // --- Données géométrie / domaines ---
    static java.util.Map<String, java.awt.geom.Area> domainAreas = new java.util.LinkedHashMap<>();
    static java.util.List<ShapeItem> shapes = new java.util.ArrayList<>();
    static int rectCount = 0, ovalCount = 0, lineCount = 0, polyCount = 0;

    static javax.swing.table.DefaultTableModel domainsModel;
    static java.util.Map<String, java.util.LinkedHashSet<String>> domainToShapes = new java.util.LinkedHashMap<>();

    // Pour chaque forme dessinée, stocker tous ses segments (ids de lignes)
    static java.util.Map<String, java.util.List<String>> shapeEdgeLines = new java.util.LinkedHashMap<>();

    // --- Canvases / rafraîchissement global --------------------------------
    static javax.swing.JComponent canvasGeom = null;   // canvas Géométrie
    static javax.swing.JComponent canvasDom  = null;   // canvas Domaines
    static javax.swing.JComponent canvasCI   = null;   // canvas Conditions Initiales

    // Callback optionnel déclenché quand la géométrie des domaines change
    static Runnable domainsChanged = null;

    static void fireDomainsChanged() {
        try { if (domainsChanged != null) domainsChanged.run(); } catch (Throwable ignore) {}
        if (canvasGeom != null) canvasGeom.repaint();
        if (canvasDom  != null) canvasDom.repaint();
        if (canvasCI   != null) canvasCI.repaint();
    }

    static void repaintAll() {
        if (canvasGeom != null) canvasGeom.repaint();
        if (canvasDom  != null) canvasDom.repaint();
        if (canvasCI   != null) canvasCI.repaint();
    }

    // ---------------- Matériaux ----------------
    static javax.swing.DefaultComboBoxModel<String> materialsModel = new javax.swing.DefaultComboBoxModel<>();
    static java.util.Map<String, java.util.LinkedHashSet<String>> materialDomains = new java.util.LinkedHashMap<>();
    static java.util.Map<String, String> domainOwner = new java.util.LinkedHashMap<>();

    static void ensureMaterial(String name){
        if (materialsModel.getIndexOf(name) < 0) materialsModel.addElement(name);
        materialDomains.putIfAbsent(name, new java.util.LinkedHashSet<String>());
    }

    // ==== Définition des variables par matériau ====
    static class VarSpec {
        boolean useFunc = true;
        String  funcExpr = "";
        String  csvPath  = "";
        boolean useT = false, useP = false, useConc = false;
    }
    static java.util.Map<String, java.util.Map<String, VarSpec>> matVars = new java.util.LinkedHashMap<>();

    static VarSpec getVarSpec(String material, String varName){
        if (material == null || varName == null) return new VarSpec();
        java.util.Map<String, VarSpec> byVar = matVars.computeIfAbsent(material, k -> new java.util.LinkedHashMap<>());
        return byVar.computeIfAbsent(varName, k -> new VarSpec());
    }

    // ==== Réactions par matériau ==== 
    static class ReactionSpec {
        String kExpr = "";
        String energyMeV = "";
        String products = "";
        String reactants = "";
    }
    static java.util.Map<String, java.util.List<ReactionSpec>> matReactions = new java.util.LinkedHashMap<>();

    static java.util.List<ReactionSpec> getReactions(String material){
        return matReactions.computeIfAbsent(material, k -> new java.util.ArrayList<>());
    }

    // ---------------- Historique d’édition des domaines ----------------
    enum DomOp { UNION, SUB, INTER, XOR }

    static class DomainEdit {
        final String shapeId;
        final DomOp op;
        DomainEdit(String id, DomOp op){ this.shapeId = id; this.op = op; }
    }
    static java.util.Map<String, java.util.List<DomainEdit>> domainEdits = new java.util.LinkedHashMap<>();

    static java.util.Map<String, DomOp> lastOpsForDomain(String dom){
        java.util.Map<String, DomOp> map = new java.util.LinkedHashMap<>();
        java.util.List<DomainEdit> L = domainEdits.get(dom);
        if (L != null) for (DomainEdit de : L) map.put(de.shapeId, de.op);
        return map;
    }

    // ----- CONDITIONS LIMITES -----
    static javax.swing.table.DefaultTableModel clModel;
    static class CLSpec {
        String ux = "", uy = "", p = "", T = "", B = "", mass = "";
        boolean external = true;
    }
    static java.util.Map<String, CLSpec> boundarySpecs = new java.util.LinkedHashMap<>();
    static java.util.Map<String, java.util.LinkedHashSet<String>> boundaryLines = new java.util.LinkedHashMap<>();

    // ----- CONDITIONS INITIALES -----
    static javax.swing.table.DefaultTableModel ciModel;
    static java.util.Map<String, CLSpec> initSpecs = new java.util.LinkedHashMap<>();
    static java.util.Map<String, java.util.LinkedHashSet<String>> initDomains = new java.util.LinkedHashMap<>();

    // ==== MAILLAGE ====
    enum MeshKind { TRI_REG, RECT_REG, TRI_GRAD, RECT_GRAD }

    static class MeshSpec {
        MeshKind kind = MeshKind.TRI_REG;
        java.lang.Double dx, dy;
        java.lang.Double dxmin, dxmax, dymin, dymax;
        boolean growFromBoundary = true;
        java.util.LinkedHashSet<String> growFromBoundaries = new java.util.LinkedHashSet<>();
    }

    static boolean isMeshSpecComplete(String dom) {
        if (dom == null) return false;
        MeshSpec s = meshSpecs.get(dom);
        if (s == null) return false;
        switch (s.kind) {
            case TRI_REG:
            case RECT_REG:
                return s.dx != null && s.dy != null;
            case TRI_GRAD:
            case RECT_GRAD:
                return s.dxmin != null && s.dxmax != null && s.dymin != null && s.dymax != null;
            default:
                return false;
        }
    }
    static java.util.Map<String, MeshSpec> meshSpecs = new java.util.LinkedHashMap<>();

    // ---------------- Helpers Paramètres ----------------
    static java.util.List<String> listParamNames() {
        java.util.List<String> names = new java.util.ArrayList<>();
        if (paramsModel == null) return names;
        for (int r = 0; r < paramsModel.getRowCount(); r++) {
            Object n = paramsModel.getValueAt(r, 0);
            if (n != null) {
                String s = n.toString().trim();
                if (!s.isEmpty()) names.add(s);
            }
        }
        return names;
    }

    static java.lang.Double getParamValue(String name) {
        if (paramsModel == null || name == null) return null;
        String target = name.trim();
        for (int r = 0; r < paramsModel.getRowCount(); r++) {
            Object n = paramsModel.getValueAt(r, 0);
            if (n != null && target.equalsIgnoreCase(n.toString().trim())) {
                Object v = paramsModel.getValueAt(r, 2);
                if (v == null) return null;
                String s = v.toString().trim().replace(',', '.');
                if (s.isEmpty()) return null;
                try { return Double.parseDouble(s); }
                catch (NumberFormatException ex) { return null; }
            }
        }
        return null;
    }

    // ---------------- Helpers géométrie ----------------
    static boolean isRightAnglePolygon(java.awt.geom.Area a, double tolDeg) {
        if (a == null || a.isEmpty()) return false;
        java.awt.geom.PathIterator it = a.getPathIterator(null, 0.5);
        double[] c = new double[6];
        java.util.List<java.util.List<java.awt.geom.Point2D.Double>> polygons = new java.util.ArrayList<>();
        java.util.List<java.awt.geom.Point2D.Double> cur = null;

        while (!it.isDone()) {
            int t = it.currentSegment(c);
            switch (t) {
                case java.awt.geom.PathIterator.SEG_MOVETO:
                    if (cur != null && cur.size() >= 3) polygons.add(cur);
                    cur = new java.util.ArrayList<>();
                    cur.add(new java.awt.geom.Point2D.Double(c[0], c[1]));
                    break;
                case java.awt.geom.PathIterator.SEG_LINETO:
                    if (cur == null) return false;
                    cur.add(new java.awt.geom.Point2D.Double(c[0], c[1]));
                    break;
                case java.awt.geom.PathIterator.SEG_CLOSE:
                    if (cur != null && cur.size() >= 3) {
                        java.awt.geom.Point2D.Double f = cur.get(0);
                        java.awt.geom.Point2D.Double l = cur.get(cur.size()-1);
                        if (Math.hypot(f.x - l.x, f.y - l.y) > 1e-9)
                            cur.add(new java.awt.geom.Point2D.Double(f.x, f.y));
                        polygons.add(cur);
                        cur = null;
                    }
                    break;
                default:
                    return false;
            }
            it.next();
        }
        if (cur != null && cur.size() >= 3) polygons.add(cur);
        if (polygons.isEmpty()) return false;

        final double toDeg = 180.0 / Math.PI;
        for (java.util.List<java.awt.geom.Point2D.Double> pts : polygons) {
            int n = pts.size() - 1;
            for (int i = 0; i < n; i++) {
                java.awt.geom.Point2D.Double pPrev = pts.get((i - 1 + n) % n);
                java.awt.geom.Point2D.Double p     = pts.get(i);
                java.awt.geom.Point2D.Double pNext = pts.get((i + 1) % n);
                double v1x = p.x - pPrev.x, v1y = p.y - pPrev.y;
                double v2x = pNext.x - p.x, v2y = pNext.y - p.y;
                double n1 = Math.hypot(v1x, v1y), n2 = Math.hypot(v2x, v2y);
                if (n1 < 1e-9 || n2 < 1e-9) continue;
                if (Math.abs(v1x) > 1e-9 && Math.abs(v1y) > 1e-9) return false;
                if (Math.abs(v2x) > 1e-9 && Math.abs(v2y) > 1e-9) return false;
                double dot = (v1x*v2x + v1y*v2y) / (n1*n2);
                dot = Math.max(-1, Math.min(1, dot));
                double angle = Math.acos(dot) * toDeg;
                double dev1 = Math.abs(90.0 - angle);
                double dev2 = Math.abs(270.0 - angle);
                if (dev1 > tolDeg && dev2 > tolDeg) return false;
            }
        }
        return true;
    }

    static boolean isAxisAlignedOrRectangle(java.awt.geom.Area a) {
        if (a==null || a.isEmpty()) return false;
        java.awt.geom.Rectangle2D b = a.getBounds2D();
        java.awt.geom.Area box = new java.awt.geom.Area(b);
        java.awt.geom.Area xor = new java.awt.geom.Area(a);
        xor.exclusiveOr(box);
        if (xor.isEmpty()) return true;

        java.awt.geom.PathIterator it = a.getPathIterator(null, 0.5);
        double[] coords = new double[6];
        double sx=0, sy=0, px=0, py=0;
        boolean hasSeg = false;
        while (!it.isDone()) {
            int type = it.currentSegment(coords);
            switch (type) {
                case java.awt.geom.PathIterator.SEG_MOVETO:
                    sx = px = coords[0]; sy = py = coords[1];
                    break;
                case java.awt.geom.PathIterator.SEG_LINETO: {
                    double x = coords[0], y = coords[1];
                    if (Math.abs(x - px) > 1e-6 && Math.abs(y - py) > 1e-6) return false;
                    px = x; py = y; hasSeg = true;
                    break;
                }
                case java.awt.geom.PathIterator.SEG_CLOSE:
                    if (hasSeg && (Math.abs(px - sx) > 1e-6 && Math.abs(py - sy) > 1e-6)) return false;
                    break;
                default:
                    return false;
            }
            it.next();
        }
        return true;
    }
}


// ==== Canvas2D (NOUVELLE VERSION, listeners sûrs) ==========================
class Canvas2D extends javax.swing.JPanel {

    enum EdgeView { NONE, ALL_EDGES }

    // marges écran (pixels)
    final int pad = 40;
    final boolean domainMode;
    final java.util.function.Supplier<String> currentDomainSupplier;
    final EdgeView edgeView;

    // vue (monde -> écran)
    private double zoom = 1.0;
    private double tx = 0.0, ty = 0.0; // translation en monde (avant zoom)
    private java.awt.geom.AffineTransform worldToScreen = new java.awt.geom.AffineTransform();
    private java.awt.geom.AffineTransform screenToWorld = new java.awt.geom.AffineTransform();

    // Listener de repaint (gardé pour pouvoir le retirer)
    private final Runnable repaintListener = () -> javax.swing.SwingUtilities.invokeLater(this::repaint);

    Canvas2D() { this(false, () -> null, EdgeView.ALL_EDGES); }
    Canvas2D(boolean domainMode, java.util.function.Supplier<String> currentDomainSupplier) {
        this(domainMode, currentDomainSupplier, EdgeView.NONE);
    }
    Canvas2D(boolean domainMode, java.util.function.Supplier<String> currentDomainSupplier, EdgeView edgeView) {
        this.domainMode = domainMode;
        this.currentDomainSupplier = currentDomainSupplier;
        this.edgeView = edgeView;

        setBackground(new java.awt.Color(246,246,246));
        setBorder(javax.swing.BorderFactory.createDashedBorder(java.awt.Color.GRAY));
        setPreferredSize(new java.awt.Dimension(1000, 700));

        // Sélection (clic)
        addMouseListener(new java.awt.event.MouseAdapter() {
            @Override public void mouseClicked(java.awt.event.MouseEvent e) {
                java.awt.geom.Point2D ptW = toWorld(e.getPoint());
                boolean toggle = ( (e.getModifiersEx() & java.awt.event.InputEvent.CTRL_DOWN_MASK)  != 0
                                || (e.getModifiersEx() & java.awt.event.InputEvent.SHIFT_DOWN_MASK) != 0 );
                selectAt(ptW, toggle);
                repaint();
            }
        });

        // curseur main si survole une ligne
        addMouseMotionListener(new java.awt.event.MouseMotionAdapter() {
            @Override public void mouseMoved(java.awt.event.MouseEvent e) {
                java.awt.geom.Point2D ptW = toWorld(e.getPoint());
                boolean over = false;
                for (int i = Globals.shapes.size()-1; i>=0; --i){
                    ShapeItem it = Globals.shapes.get(i);
                    if (it.kind != ShapeKind.LINE) continue;
                    java.awt.Shape stro = new java.awt.BasicStroke(6f/ (float) zoom).createStrokedShape(it.shape);
                    if (stro.contains(ptW)) { over = true; break; }
                }
                setCursor(over ? java.awt.Cursor.getPredefinedCursor(java.awt.Cursor.HAND_CURSOR)
                               : java.awt.Cursor.getDefaultCursor());
            }
        });

        // Zoom molette (centré sur le pointeur)
        addMouseWheelListener(e -> {
            double step = Math.pow(1.1, -e.getWheelRotation());
            zoomAt(e.getPoint(), step);
            repaint();
        });

        updateTransforms();
    }

    // S'abonner/désabonner proprement aux changements de repère
    @Override public void addNotify() {
        super.addNotify();
        Globals.addCoordListener(repaintListener);
    }
    @Override public void removeNotify() {
        Globals.removeCoordListener(repaintListener);
        super.removeNotify();
    }

    // ---------- Transfos / util ----------
    private void updateTransforms(){
        worldToScreen = new java.awt.geom.AffineTransform();
        worldToScreen.translate(pad, pad);
        worldToScreen.translate(tx, ty);
        worldToScreen.scale(zoom, zoom);
        try { screenToWorld = worldToScreen.createInverse(); }
        catch (java.awt.geom.NoninvertibleTransformException ex) { screenToWorld = new java.awt.geom.AffineTransform(); }
    }

    private java.awt.geom.Point2D toWorld(java.awt.Point p){
        return screenToWorld.transform(p, null);
    }

    // zoom autour d'un point écran (le garde immobile)
    private void zoomAt(java.awt.Point pivotScreen, double factor){
        java.awt.geom.Point2D before = toWorld(pivotScreen);
        zoom = Math.max(0.02, Math.min(50.0, zoom * factor));
        updateTransforms();
        java.awt.geom.Point2D after = toWorld(pivotScreen);
        // corriger la translation monde pour conserver le point
        tx += (after.getX() - before.getX()) * zoom;
        ty += (after.getY() - before.getY()) * zoom;
        updateTransforms();
    }

    // recadrer sur toutes les formes
    void fitAll(){
        java.awt.geom.Rectangle2D bounds = null;
        for (ShapeItem it : Globals.shapes){
            if (it.shape == null) continue;
            java.awt.geom.Rectangle2D b = it.shape.getBounds2D();
            bounds = (bounds == null) ? (java.awt.geom.Rectangle2D) b.clone()
                                      : bounds.createUnion(b);
        }
        if (bounds == null) bounds = new java.awt.geom.Rectangle2D.Double(0,0,400,300);

        int W = Math.max(1, getWidth() - 2*pad);
        int H = Math.max(1, getHeight()- 2*pad);
        double s = 0.9 * Math.min(W / bounds.getWidth(), H / bounds.getHeight());
        zoom = Math.max(0.02, Math.min(50.0, s));
        // placer l'origine monde pour centrer
        double wx = -bounds.getX() - bounds.getWidth()/2.0;
        double wy = -bounds.getY() - bounds.getHeight()/2.0;
        double sx = W/2.0, sy = H/2.0;
        tx = sx/zoom + wx;
        ty = sy/zoom + wy;

        updateTransforms();
        repaint();
    }

    // sélection en coordonnées monde
    void selectAt(java.awt.geom.Point2D p, boolean toggle){
        int hitIndex = -1;
        for (int i = Globals.shapes.size()-1; i>=0; --i){
            ShapeItem si = Globals.shapes.get(i);
            java.awt.Shape s = si.shape;
            boolean hit;
            if (si.kind == ShapeKind.LINE) {
                hit = new java.awt.BasicStroke(6f/(float)zoom).createStrokedShape(s).contains(p);
            } else {
                hit = s.contains(p);
            }
            if (hit){ hitIndex = i; break; }
        }
        if (!toggle){
            for (ShapeItem it : Globals.shapes) it.selected = false;
            if (hitIndex >= 0) Globals.shapes.get(hitIndex).selected = true;
        } else if (hitIndex >= 0) {
            Globals.shapes.get(hitIndex).selected = !Globals.shapes.get(hitIndex).selected;
        }
    }
    static boolean isClosed(ShapeItem it){ return it.kind != ShapeKind.LINE; }

    static java.awt.geom.Area areaOfSelectedClosed(){
        java.awt.geom.Area A = null;
        for (ShapeItem it : Globals.shapes){
            if (it.selected && isClosed(it)){
                java.awt.geom.Area a = new java.awt.geom.Area(it.shape);
                if (A == null) A = new java.awt.geom.Area(a); else A.add(a);
            }
        }
        return A;
    }

    // edges internes
    private String addEdgeShape(String parentId, java.awt.Shape edgeShape) {
        String lid = "l" + (++Globals.lineCount);
        ShapeItem li = new ShapeItem(lid, ShapeKind.LINE, edgeShape);
        li.color = new java.awt.Color(30,30,30);
        Globals.shapes.add(li);
        java.util.List<String> list = Globals.shapeEdgeLines.computeIfAbsent(parentId, k -> new java.util.ArrayList<>());
        list.add(lid);
        return lid;
    }

    // créations (coordonnées monde)
    void addRect(double X, double Y, double W, double H){
        String id = "r"+(++Globals.rectCount);
        java.awt.geom.Rectangle2D.Double rect = new java.awt.geom.Rectangle2D.Double(X, Y, W, H);
        Globals.shapes.add(new ShapeItem(id, ShapeKind.RECT, rect));
        addEdgeShape(id, new java.awt.geom.Line2D.Double(X, Y, X+W, Y));
        addEdgeShape(id, new java.awt.geom.Line2D.Double(X+W, Y, X+W, Y+H));
        addEdgeShape(id, new java.awt.geom.Line2D.Double(X+W, Y+H, X, Y+H));
        addEdgeShape(id, new java.awt.geom.Line2D.Double(X, Y+H, X, Y));
        Globals.repaintAll();
    }

    void addOvalCenter(double XC, double YC, double RX, double RY){
        String id = "ov"+(++Globals.ovalCount);
        java.awt.geom.Ellipse2D.Double ellipse =
            new java.awt.geom.Ellipse2D.Double(XC-RX, YC-RY, 2*RX, 2*RY);
        Globals.shapes.add(new ShapeItem(id, ShapeKind.OVAL, ellipse));
        addEdgeShape(id, ellipse); // une seule arête pour l’ovale
        Globals.repaintAll();
    }

    void addLine(double X1, double Y1, double X2, double Y2){
        double dx=X2-X1, dy=Y2-Y1; if (dx*dx+dy*dy <= 1e-12) return;
        String lid = "l"+(++Globals.lineCount);
        java.awt.geom.Line2D.Double L = new java.awt.geom.Line2D.Double(X1,Y1,X2,Y2);
        Globals.shapes.add(new ShapeItem(lid, ShapeKind.LINE, L));
        Globals.shapeEdgeLines.put(lid, java.util.Collections.singletonList(lid)); // ligne libre = sa propre arête
        Globals.repaintAll();
    }

    void deleteSelected(){
        java.util.Set<String> toRemove = new java.util.LinkedHashSet<>();
        for (ShapeItem it : Globals.shapes) if (it.selected) toRemove.add(it.id);
        if (toRemove.isEmpty()) return;

        for (String id : new java.util.ArrayList<>(toRemove)) {
            java.util.List<String> edges = Globals.shapeEdgeLines.remove(id);
            if (edges != null) toRemove.addAll(edges);
        }
        for (java.util.Map.Entry<String, java.util.List<String>> en : new java.util.ArrayList<>(Globals.shapeEdgeLines.entrySet())) {
            java.util.List<String> list = en.getValue();
            if (list != null) list.removeIf(toRemove::contains);
            if (list != null && list.isEmpty()) Globals.shapeEdgeLines.remove(en.getKey());
        }
        Globals.shapes.removeIf(si -> toRemove.contains(si.id));
        for (java.util.List<Globals.DomainEdit> list : Globals.domainEdits.values())
            list.removeIf(de -> toRemove.contains(de.shapeId));
        for (java.util.Set<String> set : Globals.boundaryLines.values())
            if (set != null) set.removeAll(toRemove);

        Globals.repaintAll();
    }

    java.awt.geom.Path2D buildClosedPathFromSelectedLines(){
        java.util.List<java.awt.Shape> segs = new java.util.ArrayList<>();
        for (ShapeItem it : Globals.shapes)
            if (it.selected && it.kind == ShapeKind.LINE)
                segs.add(it.shape);
        if (segs.size() < 3) return null;

        java.util.List<java.awt.geom.Line2D> L = new java.util.ArrayList<>();
        for (java.awt.Shape s : segs) {
            if (s instanceof java.awt.geom.Line2D) L.add((java.awt.geom.Line2D) s);
            else return null;
        }

        final double TOL = 1e-6;
        class Key {
            final long xk, yk;
            Key(double x, double y){
                double sx = Math.rint(x / TOL) * TOL;
                double sy = Math.rint(y / TOL) * TOL;
                this.xk = Double.doubleToLongBits(sx);
                this.yk = Double.doubleToLongBits(sy);
            }
            @Override public int hashCode(){ return (int)((xk ^ (xk>>>32)) * 31 + (yk ^ (yk>>>32))); }
            @Override public boolean equals(Object o){
                if (!(o instanceof Key)) return false;
                Key k = (Key) o; return xk==k.xk && yk==k.yk;
            }
        }
        java.util.Map<Key, java.util.List<Key>> adj = new java.util.LinkedHashMap<>();
        java.util.Map<Key, java.awt.geom.Point2D> key2pt = new java.util.LinkedHashMap<>();
        java.util.function.Function<java.awt.geom.Point2D, Key> K = (pt) -> {
            Key k = new Key(pt.getX(), pt.getY());
            if (!key2pt.containsKey(k)) key2pt.put(k, new java.awt.geom.Point2D.Double(pt.getX(), pt.getY()));
            return k;
        };
        for (java.awt.geom.Line2D l : L) {
            Key a = K.apply(new java.awt.geom.Point2D.Double(l.getX1(), l.getY1()));
            Key b = K.apply(new java.awt.geom.Point2D.Double(l.getX2(), l.getY2()));
            adj.computeIfAbsent(a, __ -> new java.util.ArrayList<>()).add(b);
            adj.computeIfAbsent(b, __ -> new java.util.ArrayList<>()).add(a);
        }
        for (java.util.Map.Entry<Key, java.util.List<Key>> e : adj.entrySet())
            if (e.getValue().size() != 2) return null;

        Key start = adj.keySet().iterator().next();
        java.util.List<Key> order = new java.util.ArrayList<>();
        Key prev = null, cur = start;
        int guard = 0, max = adj.size()+5;
        while (guard++ < max){
            order.add(cur);
            java.util.List<Key> neigh = adj.get(cur);
            Key next = neigh.get(0).equals(prev) ? neigh.get(1) : neigh.get(0);
            prev = cur; cur = next;
            if (cur.equals(start)) break;
        }
        if (!cur.equals(start) || order.size() < 3) return null;

        java.awt.geom.Path2D path = new java.awt.geom.Path2D.Double();
        java.awt.geom.Point2D p0 = key2pt.get(order.get(0));
        path.moveTo(p0.getX(), p0.getY());
        for (int i=1;i<order.size();i++){
            java.awt.geom.Point2D p = key2pt.get(order.get(i));
            path.lineTo(p.getX(), p.getY());
        }
        path.closePath();
        return path;
    }

    // ----------- Rendu ------------------------------------------------------
    @Override
    protected void paintComponent(java.awt.Graphics g) {
        super.paintComponent(g);
        updateTransforms();
        java.awt.Graphics2D g2 = (java.awt.Graphics2D) g.create();
        g2.setRenderingHint(java.awt.RenderingHints.KEY_ANTIALIASING, java.awt.RenderingHints.VALUE_ANTIALIAS_ON);

        // Passe en repère monde
        g2.translate(pad, pad);
        g2.translate(tx, ty);
        g2.scale(zoom, zoom);

        // 0) grille + axes selon le système
        drawGridAndAxes(g2);

        // domaine actif
        String activeDom = (domainMode && currentDomainSupplier != null) ? currentDomainSupplier.get() : null;
        java.util.Map<String, Globals.DomOp> domOps =
                (domainMode && activeDom != null) ? Globals.lastOpsForDomain(activeDom)
                                                  : java.util.Collections.emptyMap();

        // 1) formes fermées
        for (ShapeItem it : Globals.shapes) {
            if (it.kind == ShapeKind.LINE) continue;

            boolean isMember = domOps.containsKey(it.id);
            java.awt.Color edge = new java.awt.Color(80,80,80);
            java.awt.Color fill = new java.awt.Color(50,50,50);
            float fillAlpha = isMember ? 0.16f : 0.05f;

            if (domainMode && isMember) {
                Globals.DomOp op = domOps.get(it.id);
                if (op != null) {
                    switch (op) {
                        case UNION: edge = new java.awt.Color(0,110,230); fill = edge; break;
                        case SUB:   edge = new java.awt.Color(200,60,60); fill = edge; break;
                        case INTER: edge = new java.awt.Color(0,150,100); fill = edge; break;
                        case XOR:   edge = new java.awt.Color(140,60,180); fill = edge; break;
                    }
                }
            }

            g2.setStroke(new java.awt.BasicStroke(1.6f/(float)zoom));
            g2.setColor(edge);
            g2.draw(it.shape);
            g2.setComposite(java.awt.AlphaComposite.SrcOver.derive(fillAlpha));
            g2.setColor(fill);
            g2.fill(it.shape);
            g2.setComposite(java.awt.AlphaComposite.SrcOver);

            if (it.selected) {
                g2.setColor(new java.awt.Color(0,120,215));
                g2.setStroke(new java.awt.BasicStroke(2f/(float)zoom));
                g2.draw(it.shape.getBounds2D());
            }

            java.awt.Rectangle b = it.shape.getBounds();
            g2.setColor(isMember ? edge.darker() : new java.awt.Color(60,60,60));
            g2.scale(1/zoom, 1/zoom);
            g2.drawString(it.id, (float)(b.x + b.width + 4)* (float)zoom, (float)(b.y + 12)* (float)zoom);
            g2.scale(zoom, zoom);
        }

        // 2) lignes (si demandé)
        if (edgeView == EdgeView.ALL_EDGES) {
            for (ShapeItem it : Globals.shapes) {
                if (it.kind != ShapeKind.LINE) continue;
                g2.setColor(it.selected ? new java.awt.Color(0,120,215) : new java.awt.Color(40,40,40));
                g2.setStroke(new java.awt.BasicStroke(it.selected ? 3.0f/(float)zoom : 2.0f/(float)zoom,
                                                      java.awt.BasicStroke.CAP_ROUND,
                                                      java.awt.BasicStroke.JOIN_ROUND));
                g2.draw(it.shape);

                java.awt.Rectangle b = it.shape.getBounds();
                g2.setColor(new java.awt.Color(60,60,60));
                g2.scale(1/zoom, 1/zoom);
                g2.drawString(it.id, (float)(b.x + b.width + 4)* (float)zoom, (float)(b.y + 12)* (float)zoom);
                g2.scale(zoom, zoom);
            }
        }

        // 3) aire du domaine actif
        if (domainMode && activeDom != null) {
            java.awt.geom.Area DA = Globals.domainAreas.get(activeDom);
            if (DA != null && !DA.isEmpty()) {
                g2.setComposite(java.awt.AlphaComposite.SrcOver.derive(0.10f));
                g2.setColor(new java.awt.Color(0,110,230));
                g2.fill(DA);
                g2.setComposite(java.awt.AlphaComposite.SrcOver);
                g2.setColor(new java.awt.Color(0,110,230));
                g2.setStroke(new java.awt.BasicStroke(1.8f/(float)zoom));
                g2.draw(DA);
            }
        }
        g2.dispose();
    }

    // --------- Grilles selon système ----------------------------------------
    private void drawGridAndAxes(java.awt.Graphics2D g2){
        java.awt.geom.Rectangle2D vis = screenToWorld.createTransformedShape(
                new java.awt.Rectangle(0,0, getWidth(), getHeight())
        ).getBounds2D();
        double xmin = vis.getMinX()-50, xmax = vis.getMaxX()+50;
        double ymin = vis.getMinY()-50, ymax = vis.getMaxY()+50;

        switch (Globals.currentCoordSys){
            case CARTESIAN_2D -> {
                double step = 20.0, major = 100.0;
                for (double x = Math.floor(xmin/step)*step; x <= xmax; x += step){
                    g2.setColor(Math.round(Math.abs(x)/major) == Math.abs(x)/major ? new java.awt.Color(210,210,210) : new java.awt.Color(235,235,235));
                    g2.draw(new java.awt.geom.Line2D.Double(x, ymin, x, ymax));
                }
                for (double y = Math.floor(ymin/step)*step; y <= ymax; y += step){
                    g2.setColor(Math.round(Math.abs(y)/major) == Math.abs(y)/major ? new java.awt.Color(210,210,210) : new java.awt.Color(235,235,235));
                    g2.draw(new java.awt.geom.Line2D.Double(xmin, y, xmax, y));
                }
                // axes
                g2.setColor(new java.awt.Color(120,120,120));
                g2.setStroke(new java.awt.BasicStroke(1.5f/(float)zoom));
                g2.draw(new java.awt.geom.Line2D.Double(xmin, 0, xmax, 0));
                g2.draw(new java.awt.geom.Line2D.Double(0, ymin, 0, ymax));
            }
            case AXISYM_2D -> {
                double stepR = 20.0, stepZ = 20.0, major = 100.0;
                for (double r = Math.max(0, Math.floor(0/stepR)*stepR); r <= xmax; r += stepR){
                    g2.setColor(Math.round(r/major) == r/major ? new java.awt.Color(210,210,210) : new java.awt.Color(235,235,235));
                    g2.draw(new java.awt.geom.Line2D.Double(r, ymin, r, ymax));
                }
                for (double z = Math.floor(ymin/stepZ)*stepZ; z <= ymax; z += stepZ){
                    g2.setColor(Math.round(Math.abs(z)/major) == Math.abs(z)/major ? new java.awt.Color(210,210,210) : new java.awt.Color(235,235,235));
                    g2.draw(new java.awt.geom.Line2D.Double(0, z, xmax, z));
                }
                g2.setColor(new java.awt.Color(120,120,120));
                g2.setStroke(new java.awt.BasicStroke(1.5f/(float)zoom));
                g2.draw(new java.awt.geom.Line2D.Double(0, ymin, 0, ymax)); // r=0
                g2.draw(new java.awt.geom.Line2D.Double(0, 0,  xmax, 0));   // z=0
            }
            case POLAR_2D -> {
                double rMax = Math.max(Math.abs(xmin), Math.max(Math.abs(xmax), Math.max(Math.abs(ymin), Math.abs(ymax))));
                double rStep = 50.0;
                for (double r = rStep; r <= rMax; r += rStep){
                    g2.setColor(((int)Math.round(r/rStep))%5==0 ? new java.awt.Color(210,210,210) : new java.awt.Color(235,235,235));
                    g2.draw(new java.awt.geom.Ellipse2D.Double(-r, -r, 2*r, 2*r));
                }
                for (int deg = 0; deg < 360; deg += 15){
                    double a = Math.toRadians(deg);
                    g2.setColor(deg % 90 == 0 ? new java.awt.Color(210,210,210) : new java.awt.Color(235,235,235));
                    g2.draw(new java.awt.geom.Line2D.Double(0,0, rMax*Math.cos(a), rMax*Math.sin(a)));
                }
                g2.setColor(new java.awt.Color(120,120,120));
                g2.setStroke(new java.awt.BasicStroke(1.5f/(float)zoom));
                g2.draw(new java.awt.geom.Line2D.Double(xmin,0,xmax,0));
                g2.draw(new java.awt.geom.Line2D.Double(0,ymin,0,ymax));
            }
            case Q2D_STELLARATOR -> {
                double rMax = Math.max(Math.abs(xmin), Math.max(Math.abs(xmax), Math.max(Math.abs(ymin), Math.abs(ymax))));
                double rStep = 60.0; int n = 6; double eps = 0.08;
                for (double r = rStep; r <= rMax; r += rStep){
                    java.awt.geom.Path2D iso = new java.awt.geom.Path2D.Double();
                    for (int k=0;k<=360;k++){
                        double t = Math.toRadians(k);
                        double rr = r * (1 + eps*Math.sin(n*t));
                        double x = rr*Math.cos(t), y = rr*Math.sin(t);
                        if (k==0) iso.moveTo(x,y); else iso.lineTo(x,y);
                    }
                    iso.closePath();
                    g2.setColor(((int)Math.round(r/rStep))%3==0 ? new java.awt.Color(210,210,210) : new java.awt.Color(235,235,235));
                    g2.draw(iso);
                }
                int m = 12;
                for (int i=0;i<m;i++){
                    double t = 2*Math.PI*i/m;
                    java.awt.geom.Path2D ray = new java.awt.geom.Path2D.Double();
                    double R = rMax;
                    for (int k=0;k<=200;k++){
                        double s = R*k/200.0;
                        double phi = t + 0.5*eps*Math.sin(3*s/rStep);
                        double x = s*Math.cos(phi), y = s*Math.sin(phi);
                        if (k==0) ray.moveTo(0,0); else ray.lineTo(x,y);
                    }
                    g2.setColor(i%3==0 ? new java.awt.Color(210,210,210) : new java.awt.Color(235,235,235));
                    g2.draw(ray);
                }
                g2.setColor(new java.awt.Color(120,120,120));
                g2.setStroke(new java.awt.BasicStroke(1.5f/(float)zoom));
                g2.draw(new java.awt.geom.Line2D.Double(xmin,0,xmax,0));
                g2.draw(new java.awt.geom.Line2D.Double(0,ymin,0,ymax));
            }
        }
    }
}


// Récupère le texte d'un champ
static String textOfField(javax.swing.JTextField tf) {
    return tf == null ? "" : tf.getText().trim();
}

// Essaie: 1) nombre direct, 2) nom de paramètre du tableau Paramètres -> valeur numérique
static Double resolveScalarFromText(String s) {
    if (s == null) return null;
    s = s.trim().replace(',', '.');
    if (s.isEmpty()) return null;

    // (1) nombre direct ?
    try { return Double.parseDouble(s); } catch (NumberFormatException ignore) {}

    // (2) chercher dans la table "Paramètres" (Globals.paramsModel: Nom, Unité, Valeur, Desc)
    try {
        javax.swing.table.DefaultTableModel pm = Globals.paramsModel;
        if (pm != null) {
            for (int r = 0; r < pm.getRowCount(); r++) {
                Object name = pm.getValueAt(r, 0);
                if (name != null && s.equals(name.toString().trim())) {
                    Object val = pm.getValueAt(r, 2); // colonne "Valeur"
                    if (val != null) {
                        String vs = val.toString().trim().replace(',', '.');
                        try { return Double.parseDouble(vs); } catch (NumberFormatException ignore) {}
                    }
                }
            }
        }
    } catch (Throwable ignore) {}

    // rien trouvé
    return null;
}


In [3]:
// =====================================
//  MHDPersistence : bloc unique Jupyter
//  - Boutons: Enregistrer, Enregistrer-sous, Ouvrir
//  - Aucune dépendance externe; classes internes static
// =====================================

public final class MHDPersistence {

    private MHDPersistence() {}

    // =================== UTILS IHM / RÉSOLUTION PARAMS =========================
    public static String textOf(javax.swing.JSpinner sp) {
        javax.swing.JComponent ed = sp.getEditor();
        if (ed instanceof javax.swing.JSpinner.DefaultEditor de) {
            return de.getTextField().getText().trim();
        }
        Object v = sp.getValue();
        return (v == null) ? "" : v.toString().trim();
    }

    public static java.lang.Double resolveScalar(String s) {
        if (s == null) return null;
        s = s.trim().replace(',', '.');
        if (s.isEmpty()) return null;
        try { return java.lang.Double.parseDouble(s); }
        catch (NumberFormatException ignore) {}
        return Globals.getParamValue(s); // peut être null
    }

    public static String textOfField(javax.swing.JTextField tf) { return tf.getText().trim(); }

    public static java.lang.Double resolveScalarFromText(String s) {
        if (s == null) return null;
        s = s.trim().replace(',', '.');
        if (s.isEmpty()) return null;
        try { return java.lang.Double.parseDouble(s); }
        catch (NumberFormatException ignore) {}
        return Globals.getParamValue(s);
    }

    // =================== DTOs (SERIALISABLES) ==================================
    public static class ParamRow implements java.io.Serializable {
        private static final long serialVersionUID = 1L;
        public String name, unit, value, desc;
    }

    public static class DomainEditDTO implements java.io.Serializable {
        private static final long serialVersionUID = 1L;
        public String shapeId;
        public String op; // "UNION" | "SUB" | "INTER" | "XOR"
        public DomainEditDTO(){}
        public DomainEditDTO(String id, String op){ this.shapeId=id; this.op=op; }
    }

    public static class VarSpecDTO implements java.io.Serializable {
        private static final long serialVersionUID = 1L;
        public boolean useFunc = true;
        public String funcExpr = "";
        public String csvPath  = "";
        public boolean useT=false, useP=false, useConc=false;

        static VarSpecDTO from(Globals.VarSpec s){
            VarSpecDTO d = new VarSpecDTO();
            if (s==null) return d;
            d.useFunc=s.useFunc; d.funcExpr=s.funcExpr; d.csvPath=s.csvPath;
            d.useT=s.useT; d.useP=s.useP; d.useConc=s.useConc; return d;
        }
        Globals.VarSpec toRuntime(){
            Globals.VarSpec s = new Globals.VarSpec();
            s.useFunc=useFunc; s.funcExpr=funcExpr; s.csvPath=csvPath;
            s.useT=useT; s.useP=useP; s.useConc=useConc; return s;
        }
    }

    public static class ReactionSpecDTO implements java.io.Serializable {
        private static final long serialVersionUID = 1L;
        public String kExpr="", energyMeV="", products="", reactants="";
        static ReactionSpecDTO from(Globals.ReactionSpec r){
            ReactionSpecDTO d = new ReactionSpecDTO();
            if (r==null) return d;
            d.kExpr=r.kExpr; d.energyMeV=r.energyMeV; d.products=r.products; d.reactants=r.reactants;
            return d;
        }
        Globals.ReactionSpec toRuntime(){
            Globals.ReactionSpec r = new Globals.ReactionSpec();
            r.kExpr=kExpr; r.energyMeV=energyMeV; r.products=products; r.reactants=reactants; return r;
        }
    }

    public static class CLSpecDTO implements java.io.Serializable {
        private static final long serialVersionUID = 1L;
        public String ux="", uy="", p="", T="", B="", mass="";
        public boolean external = true;
        static CLSpecDTO from(Globals.CLSpec s){
            CLSpecDTO d = new CLSpecDTO();
            if (s==null) return d;
            d.ux=s.ux; d.uy=s.uy; d.p=s.p; d.T=s.T; d.B=s.B; d.mass=s.mass; d.external=s.external; return d;
        }
        Globals.CLSpec toRuntime(){
            Globals.CLSpec s = new Globals.CLSpec();
            s.ux=ux; s.uy=uy; s.p=p; s.T=T; s.B=B; s.mass=mass; s.external=external; return s;
        }
    }

    public static class MeshSpecDTO implements java.io.Serializable {
        private static final long serialVersionUID = 1L;
        public String kind = "TRI_REG";
        public java.lang.Double dx, dy, dxmin, dxmax, dymin, dymax;
        public boolean growFromBoundary = true;

        static MeshSpecDTO from(Globals.MeshSpec s){
            MeshSpecDTO d = new MeshSpecDTO();
            if (s==null) return d;
            d.kind = s.kind.name();
            d.dx=s.dx; d.dy=s.dy;
            d.dxmin=s.dxmin; d.dxmax=s.dxmax; d.dymin=s.dymin; d.dymax=s.dymax;
            d.growFromBoundary = s.growFromBoundary;
            return d;
        }
        Globals.MeshSpec toRuntime(){
            Globals.MeshSpec s = new Globals.MeshSpec();
            s.kind = Globals.MeshKind.valueOf(kind);
            s.dx=dx; s.dy=dy; s.dxmin=dxmin; s.dxmax=dxmax; s.dymin=dymin; s.dymax=dymax;
            s.growFromBoundary = growFromBoundary;
            return s;
        }
    }

    public static class SolverDTO implements java.io.Serializable {
        private static final long serialVersionUID = 1L;
        public String timeScheme = "Euler implicite";
        public String linSolver  = "CG";
        public String precond    = "ILU";
        public String tol        = "1e-8";
        public String iters      = "500";
    }

    public static class ShapeDTO implements java.io.Serializable {
        private static final long serialVersionUID = 1L;
        public String id;
        public String kind; // "RECT","OVAL","LINE","POLY"
        public double x, y, w, h;           // RECT
        public double cx, cy, rx, ry;       // OVAL
        public double x1, y1, x2, y2;       // LINE
        public java.util.List<double[]> polyPoints; // POLY

        static ShapeDTO fromShapeItem(ShapeItem it){
            ShapeDTO d = new ShapeDTO();
            d.id = it.id;
            d.kind = it.kind.name();
            java.awt.Shape s = it.shape;

            switch (it.kind) {
                case RECT -> {
                    java.awt.geom.Rectangle2D r = s.getBounds2D();
                    d.x = r.getX(); d.y = r.getY(); d.w = r.getWidth(); d.h = r.getHeight();
                }
                case OVAL -> {
                    if (s instanceof Ellipse2D e) {
                        d.cx = e.getCenterX(); d.cy = e.getCenterY();
                        d.rx = e.getWidth()/2.0; d.ry = e.getHeight()/2.0;
                    } else {
                        Rectangle2D r = s.getBounds2D();
                        d.cx = r.getCenterX(); d.cy = r.getCenterY();
                        d.rx = r.getWidth()/2.0; d.ry = r.getHeight()/2.0;
                    }
                }
                case LINE -> {
                    if (s instanceof Line2D l) {
                        d.x1 = l.getX1(); d.y1 = l.getY1(); d.x2 = l.getX2(); d.y2 = l.getY2();
                    } else {
                        System.err.println("[WARN] Shape " + it.id + " déclaré LINE mais n’est pas une Line2D: " + s.getClass());
                        Rectangle2D r = s.getBounds2D();
                        d.x1 = r.getMinX(); d.y1 = r.getMinY();
                        d.x2 = r.getMaxX(); d.y2 = r.getMaxY();
                    }
                }
                case POLY -> {
                    d.polyPoints = new java.util.ArrayList<>();
                    java.awt.geom.PathIterator itp = s.getPathIterator(null, 0.5);
                    double[] c = new double[6];
                    while (!itp.isDone()){
                        int t = itp.currentSegment(c);
                        if (t==java.awt.geom.PathIterator.SEG_MOVETO || t==java.awt.geom.PathIterator.SEG_LINETO)
                            d.polyPoints.add(new double[]{c[0], c[1]});
                        itp.next();
                    }
                }
            }
            return d;
        }

        ShapeItem toShapeItem(){
            ShapeKind k = ShapeKind.valueOf(kind);
            java.awt.Shape s;
            switch (k) {
                case RECT -> s = new java.awt.geom.Rectangle2D.Double(x, y, w, h);
                case OVAL -> s = new java.awt.geom.Ellipse2D.Double(cx - rx, cy - ry, 2*rx, 2*ry);
                case LINE -> s = new java.awt.geom.Line2D.Double(x1, y1, x2, y2);
                case POLY -> {
                    java.awt.geom.Path2D p = new java.awt.geom.Path2D.Double();
                    boolean first = true;
                    if (polyPoints != null) for (double[] pt : polyPoints) {
                        if (first) { p.moveTo(pt[0], pt[1]); first=false; }
                        else p.lineTo(pt[0], pt[1]);
                    }
                    p.closePath();
                    s = p;
                }
                default -> throw new IllegalArgumentException("kind inconnu: "+kind);
            }
            return new ShapeItem(id, k, s);
        }
    }

    // =================== ÉTAT DE PROJET (SERIALISABLE) =========================
    public static class ProjectState implements java.io.Serializable {
        private static final long serialVersionUID = 1L;
        public int version = 1;

        // Paramètres
        public java.util.List<ParamRow> params = new java.util.ArrayList<>();

        // Géométrie
        public java.util.List<ShapeDTO> shapes = new java.util.ArrayList<>();
        public java.util.Map<String, java.util.List<DomainEditDTO>> domainEdits = new java.util.LinkedHashMap<>();
        public java.util.Map<String, java.util.List<String>> shapeEdgeLines = new java.util.LinkedHashMap<>();

        // Domaines / maillage
        public java.util.Map<String, MeshSpecDTO> meshSpecs = new java.util.LinkedHashMap<>();

        // Matériaux
        public java.util.List<String> materials = new java.util.ArrayList<>();
        public java.util.Map<String, java.util.LinkedHashSet<String>> materialDomains = new java.util.LinkedHashMap<>();
        public java.util.Map<String, String> domainOwner = new java.util.LinkedHashMap<>();
        public java.util.Map<String, java.util.Map<String, VarSpecDTO>> matVars = new java.util.LinkedHashMap<>();
        public java.util.Map<String, java.util.List<ReactionSpecDTO>> reactions = new java.util.LinkedHashMap<>();

        // CL / CI
        public java.util.Map<String, CLSpecDTO> boundarySpecs = new java.util.LinkedHashMap<>();
        public java.util.Map<String, java.util.LinkedHashSet<String>> boundaryLines = new java.util.LinkedHashMap<>();
        public java.util.Map<String, CLSpecDTO> initSpecs = new java.util.LinkedHashMap<>();
        public java.util.Map<String, java.util.LinkedHashSet<String>> initDomains = new java.util.LinkedHashMap<>();

        // Solveur (optionnel)
        public SolverDTO solver = new SolverDTO();

        // ----- Construction depuis Globals -----
        public static ProjectState fromGlobals() {
            ProjectState s = new ProjectState();
            s.version = 1;

            if (Globals.paramsModel != null) {
                javax.swing.table.DefaultTableModel m = Globals.paramsModel;
                for (int r=0;r<m.getRowCount();r++){
                    ParamRow pr = new ParamRow();
                    pr.name = str(m.getValueAt(r,0));
                    pr.unit = str(m.getValueAt(r,1));
                    pr.value = str(m.getValueAt(r,2));
                    pr.desc = str(m.getValueAt(r,3));
                    s.params.add(pr);
                }
            }

            for (ShapeItem it : Globals.shapes) s.shapes.add(ShapeDTO.fromShapeItem(it));
            for (java.util.Map.Entry<String, java.util.List<String>> en : Globals.shapeEdgeLines.entrySet())
                s.shapeEdgeLines.put(en.getKey(), new java.util.ArrayList<>(en.getValue()));

            for (java.util.Map.Entry<String, java.util.List<Globals.DomainEdit>> en : Globals.domainEdits.entrySet()) {
                java.util.List<DomainEditDTO> L = new java.util.ArrayList<>();
                if (en.getValue()!=null) {
                    for (Globals.DomainEdit de : en.getValue())
                        L.add(new DomainEditDTO(de.shapeId, de.op.name()));
                }
                s.domainEdits.put(en.getKey(), L);
            }

            for (java.util.Map.Entry<String, Globals.MeshSpec> en : Globals.meshSpecs.entrySet())
                s.meshSpecs.put(en.getKey(), MeshSpecDTO.from(en.getValue()));

            if (Globals.materialsModel != null) {
                for (int i=0;i<Globals.materialsModel.getSize();i++)
                    s.materials.add(Globals.materialsModel.getElementAt(i));
            }
            for (java.util.Map.Entry<String, java.util.LinkedHashSet<String>> en : Globals.materialDomains.entrySet())
                s.materialDomains.put(en.getKey(), new java.util.LinkedHashSet<>(en.getValue()));
            s.domainOwner.putAll(Globals.domainOwner);

            for (java.util.Map.Entry<String, java.util.Map<String, Globals.VarSpec>> eMat : Globals.matVars.entrySet()) {
                java.util.Map<String, VarSpecDTO> inner = new java.util.LinkedHashMap<>();
                for (java.util.Map.Entry<String, Globals.VarSpec> eVar : eMat.getValue().entrySet())
                    inner.put(eVar.getKey(), VarSpecDTO.from(eVar.getValue()));
                s.matVars.put(eMat.getKey(), inner);
            }

            for (java.util.Map.Entry<String, java.util.List<Globals.ReactionSpec>> e : Globals.matReactions.entrySet()) {
                java.util.List<ReactionSpecDTO> L = new java.util.ArrayList<>();
                for (Globals.ReactionSpec r : e.getValue()) L.add(ReactionSpecDTO.from(r));
                s.reactions.put(e.getKey(), L);
            }

            for (java.util.Map.Entry<String, Globals.CLSpec> en : Globals.boundarySpecs.entrySet())
                s.boundarySpecs.put(en.getKey(), CLSpecDTO.from(en.getValue()));
            for (java.util.Map.Entry<String, java.util.LinkedHashSet<String>> en : Globals.boundaryLines.entrySet())
                s.boundaryLines.put(en.getKey(), new java.util.LinkedHashSet<>(en.getValue()));

            for (java.util.Map.Entry<String, Globals.CLSpec> en : Globals.initSpecs.entrySet())
                s.initSpecs.put(en.getKey(), CLSpecDTO.from(en.getValue()));
            for (java.util.Map.Entry<String, java.util.LinkedHashSet<String>> en : Globals.initDomains.entrySet())
                s.initDomains.put(en.getKey(), new java.util.LinkedHashSet<>(en.getValue()));

            return s;
        }

        // ----- Application vers Globals -----
        public void applyToGlobals() {
            Globals.paramsModel = new javax.swing.table.DefaultTableModel(new String[]{"Nom","Unité","Valeur","Description"}, 0);
            for (ParamRow pr : params) {
                Globals.paramsModel.addRow(new Object[]{nz(pr.name), nz(pr.unit), nz(pr.value), nz(pr.desc)});
            }

            Globals.shapes.clear();
            Globals.shapeEdgeLines.clear();
            Globals.rectCount = Globals.ovalCount = Globals.lineCount = Globals.polyCount = 0;

            for (ShapeDTO sd : shapes) {
                ShapeItem it = sd.toShapeItem();
                Globals.shapes.add(it);
                if (it.id.startsWith("r"))   Globals.rectCount = java.lang.Math.max(Globals.rectCount, parseIntSafe(it.id.substring(1)));
                if (it.id.startsWith("ov"))  Globals.ovalCount = java.lang.Math.max(Globals.ovalCount, parseIntSafe(it.id.substring(2)));
                if (it.id.startsWith("l"))   Globals.lineCount = java.lang.Math.max(Globals.lineCount, parseIntSafe(it.id.substring(1)));
                if (it.id.startsWith("poly"))Globals.polyCount = java.lang.Math.max(Globals.polyCount, parseIntSafe(it.id.substring(4)));
            }
            Globals.shapeEdgeLines.putAll(shapeEdgeLines);

            Globals.domainEdits.clear();
            Globals.domainAreas.clear();
            for (java.util.Map.Entry<String, java.util.List<DomainEditDTO>> en : domainEdits.entrySet()) {
                java.util.List<Globals.DomainEdit> L = new java.util.ArrayList<>();
                for (DomainEditDTO de : en.getValue())
                    L.add(new Globals.DomainEdit(de.shapeId, Globals.DomOp.valueOf(de.op)));
                Globals.domainEdits.put(en.getKey(), L);
            }
            for (String dom : Globals.domainEdits.keySet()) {
                java.awt.geom.Area A = new java.awt.geom.Area();
                boolean init = false;
                for (Globals.DomainEdit de : Globals.domainEdits.get(dom)) {
                    ShapeItem si = findShapeById(de.shapeId);
                    if (si == null || si.kind == ShapeKind.LINE) continue;
                    java.awt.geom.Area B = new java.awt.geom.Area(si.shape);
                    if (!init) { A = new java.awt.geom.Area(B); init = true; continue; }
                    switch (de.op) {
                        case UNION -> A.add(B);
                        case SUB   -> A.subtract(B);
                        case INTER -> A.intersect(B);
                        case XOR   -> A.exclusiveOr(B);
                    }
                }
                Globals.domainAreas.put(dom, A);
            }

            Globals.meshSpecs.clear();
            for (java.util.Map.Entry<String, MeshSpecDTO> en : meshSpecs.entrySet())
                Globals.meshSpecs.put(en.getKey(), en.getValue().toRuntime());

            Globals.materialsModel = new javax.swing.DefaultComboBoxModel<>();
            for (String m : materials) Globals.materialsModel.addElement(m);
            Globals.materialDomains.clear();
            for (java.util.Map.Entry<String, java.util.LinkedHashSet<String>> en : materialDomains.entrySet())
                Globals.materialDomains.put(en.getKey(), new java.util.LinkedHashSet<>(en.getValue()));
            Globals.domainOwner.clear();
            Globals.domainOwner.putAll(domainOwner);

            Globals.matVars.clear();
            for (java.util.Map.Entry<String, java.util.Map<String, VarSpecDTO>> eMat : matVars.entrySet()) {
                java.util.Map<String, Globals.VarSpec> inner = new java.util.LinkedHashMap<>();
                for (java.util.Map.Entry<String, VarSpecDTO> eVar : eMat.getValue().entrySet())
                    inner.put(eVar.getKey(), eVar.getValue().toRuntime());
                Globals.matVars.put(eMat.getKey(), inner);
            }

            Globals.matReactions.clear();
            for (java.util.Map.Entry<String, java.util.List<ReactionSpecDTO>> e : reactions.entrySet()) {
                java.util.List<Globals.ReactionSpec> L = new java.util.ArrayList<>();
                for (ReactionSpecDTO r : e.getValue()) L.add(r.toRuntime());
                Globals.matReactions.put(e.getKey(), L);
            }

            Globals.boundarySpecs.clear();
            for (java.util.Map.Entry<String, CLSpecDTO> en : boundarySpecs.entrySet())
                Globals.boundarySpecs.put(en.getKey(), en.getValue().toRuntime());
            Globals.boundaryLines.clear();
            for (java.util.Map.Entry<String, java.util.LinkedHashSet<String>> en : boundaryLines.entrySet())
                Globals.boundaryLines.put(en.getKey(), new java.util.LinkedHashSet<>(en.getValue()));

            Globals.initSpecs.clear();
            for (java.util.Map.Entry<String, CLSpecDTO> en : initSpecs.entrySet())
                Globals.initSpecs.put(en.getKey(), en.getValue().toRuntime());
            Globals.initDomains.clear();
            for (java.util.Map.Entry<String, java.util.LinkedHashSet<String>> en : initDomains.entrySet())
                Globals.initDomains.put(en.getKey(), new java.util.LinkedHashSet<>(en.getValue()));

            Globals.fireDomainsChanged();
            Globals.repaintAll();
        }

        // --- utils ---
        static String str(Object o){ return (o==null)?"":o.toString(); }
        static String nz(String s){ return (s==null)?"":s; }
        static int parseIntSafe(String s){ try { return java.lang.Integer.parseInt(s); } catch(Exception ex){ return 0; } }
        static ShapeItem findShapeById(String id){
            for (ShapeItem it : Globals.shapes) if (it.id.equals(id)) return it;
            return null;
        }
    }

    // ===== JSON minimal (sans lib externe) =====
    public static class Json {
        static String esc(String s){
            if (s == null) return "null";
            StringBuilder b = new StringBuilder(s.length()+16);
            b.append('"');
            for (int i=0;i<s.length();i++){
                char c=s.charAt(i);
                switch(c){
                    case '"':  b.append("\\\""); break;
                    case '\\': b.append("\\\\"); break;
                    case '\n': b.append("\\n");  break;
                    case '\r': b.append("\\r");  break;
                    case '\t': b.append("\\t");  break;
                    default:
                        if (c<32) b.append(String.format("\\u%04x",(int)c));
                        else b.append(c);
                }
            }
            b.append('"'); return b.toString();
        }
        static String num(java.lang.Double d){ return (d==null)?"null":(java.lang.Double.toString(d)); }
        static String bool(boolean v){ return v?"true":"false"; }
    }

    // =================== PERSISTENCE (SANS LIB EXTERNE) ==========================
    public static class ProjectIO {

        private static final javax.swing.filechooser.FileNameExtensionFilter filterProj =
            new javax.swing.filechooser.FileNameExtensionFilter("Projet MHD (*.mhdproj)", "mhdproj");

        /** Enregistrer (si currentFile == null, on bascule en "Enregistrer sous…"). */
        public static boolean saveWithChooser(java.awt.Component parent, java.io.File currentFile) {
            try {
                java.io.File target = currentFile;
                if (target == null) {
                    javax.swing.JFileChooser ch = new javax.swing.JFileChooser();
                    ch.setDialogTitle("Enregistrer le projet");
                    ch.setFileFilter(filterProj);
                    if (ch.showSaveDialog(parent) != javax.swing.JFileChooser.APPROVE_OPTION) return false;
                    target = ch.getSelectedFile();
                    if (!target.getName().toLowerCase().endsWith(".mhdproj"))
                        target = new java.io.File(target.getAbsolutePath() + ".mhdproj");
                }
                // S'assure que le dossier existe
                if (target.getParentFile() != null && !target.getParentFile().exists()) {
                    target.getParentFile().mkdirs();
                }

                ProjectState state = ProjectState.fromGlobals();
                try (java.io.ObjectOutputStream oos = new java.io.ObjectOutputStream(new java.io.FileOutputStream(target))) {
                    oos.writeObject(state);
                }
                return true;
            } catch (Exception ex) {
                ex.printStackTrace();
                javax.swing.JOptionPane.showMessageDialog(parent, ex.toString(), "Erreur sauvegarde",
                        javax.swing.JOptionPane.ERROR_MESSAGE);
                return false;
            }
        }

        /** Enregistrer sous… (force le sélecteur) */
        public static java.io.File saveAsWithChooser(java.awt.Component parent) {
            javax.swing.JFileChooser ch = new javax.swing.JFileChooser();
            ch.setDialogTitle("Enregistrer le projet");
            ch.setFileFilter(filterProj);
            if (ch.showSaveDialog(parent) != javax.swing.JFileChooser.APPROVE_OPTION) return null;
            java.io.File target = ch.getSelectedFile();
            if (!target.getName().toLowerCase().endsWith(".mhdproj"))
                target = new java.io.File(target.getAbsolutePath() + ".mhdproj");
            if (target.getParentFile()!=null && !target.getParentFile().exists()) target.getParentFile().mkdirs();
            if (saveTo(target)) return target;
            return null;
        }

        public static java.io.File openWithChooser(java.awt.Component parent) {
            try {
                javax.swing.JFileChooser ch = new javax.swing.JFileChooser();
                ch.setDialogTitle("Ouvrir un projet");
                ch.setFileFilter(filterProj);
                if (ch.showOpenDialog(parent) != javax.swing.JFileChooser.APPROVE_OPTION) return null;
                java.io.File f = ch.getSelectedFile();
                try (java.io.ObjectInputStream ois = new java.io.ObjectInputStream(new java.io.FileInputStream(f))) {
                    ProjectState state = (ProjectState) ois.readObject();
                    state.applyToGlobals();
                }
                return f;
            } catch (java.io.InvalidClassException ice) {
                ice.printStackTrace();
                javax.swing.JOptionPane.showMessageDialog(parent,
                    "Le fichier a été créé avec une version différente du programme.\n" +
                    "Détail: " + ice.toString() + "\n\n" +
                    "Solution: ré-enregistrer le projet avec cette version, ou utilisez l’export JSON.",
                    "Incompatibilité de version", javax.swing.JOptionPane.ERROR_MESSAGE);
                return null;
            } catch (Exception ex) {
                ex.printStackTrace();
                javax.swing.JOptionPane.showMessageDialog(parent, ex.toString(), "Erreur ouverture",
                        javax.swing.JOptionPane.ERROR_MESSAGE);
                return null;
            }
        }

        /** Renvoie le fichier choisi (ou null). N’enregistre rien. */
        public static java.io.File chooseSaveFile(java.awt.Component parent) {
            javax.swing.JFileChooser ch = new javax.swing.JFileChooser();
            ch.setDialogTitle("Enregistrer le projet");
            ch.setFileFilter(filterProj);
            if (ch.showSaveDialog(parent) != javax.swing.JFileChooser.APPROVE_OPTION) return null;
            java.io.File target = ch.getSelectedFile();
            if (!target.getName().toLowerCase().endsWith(".mhdproj"))
                target = new java.io.File(target.getAbsolutePath()+".mhdproj");
            if (target.getParentFile()!=null && !target.getParentFile().exists()) target.getParentFile().mkdirs();
            return target;
        }

        /** Écrit l’état courant dans le fichier cible. */
        public static boolean saveTo(java.io.File target){
            try (java.io.ObjectOutputStream oos = new java.io.ObjectOutputStream(new java.io.FileOutputStream(target))) {
                oos.writeObject(ProjectState.fromGlobals());
                return true;
            } catch (Exception ex) { ex.printStackTrace(); return false; }
        }

        /** Helper pratique pour un bouton "Enregistrer" qui fait Enregistrer/Enregistrer-sous. */
        public static java.io.File saveOrSaveAs(java.awt.Component parent, java.io.File currentFile) {
            if (currentFile == null) {
                java.io.File chosen = chooseSaveFile(parent);
                if (chosen == null) return null;
                if (saveTo(chosen)) return chosen;
                return null;
            } else {
                return saveWithChooser(parent, currentFile) ? currentFile : null;
            }
        }
    }

    // ========== EXPORT JSON (optionnel) ==========
    public static class ProjectIOJson {

        public static boolean saveJsonWithChooser(java.awt.Component parent){
            try{
                javax.swing.JFileChooser ch = new javax.swing.JFileChooser();
                ch.setDialogTitle("Exporter en JSON");
                ch.setSelectedFile(new java.io.File("projet.json"));
                if (ch.showSaveDialog(parent) != javax.swing.JFileChooser.APPROVE_OPTION) return false;
                java.io.File f = ch.getSelectedFile();
                if (!f.getName().toLowerCase().endsWith(".json"))
                    f = new java.io.File(f.getAbsolutePath()+".json");

                ProjectState s = ProjectState.fromGlobals();
                String json = toJson(s);
                try (java.io.OutputStreamWriter w = new java.io.OutputStreamWriter(
                        new java.io.FileOutputStream(f), java.nio.charset.StandardCharsets.UTF_8)) {
                    w.write(json);
                }
                return true;
            } catch(Exception ex){
                ex.printStackTrace();
                javax.swing.JOptionPane.showMessageDialog(parent, ex.toString(), "Erreur export JSON",
                    javax.swing.JOptionPane.ERROR_MESSAGE);
                return false;
            }
        }

        public static boolean openJsonWithChooser(java.awt.Component parent){
            javax.swing.JOptionPane.showMessageDialog(parent,
                "Import JSON à compléter (structure prête) — utilisez pour l’instant Ouvrir .mhdproj.",
                "Import JSON", javax.swing.JOptionPane.INFORMATION_MESSAGE);
            return false;
        }

        public static String toJson(ProjectState s){
            StringBuilder b = new StringBuilder(1<<16);
            b.append("{\n");
            // version
            b.append("  \"version\": ").append(s.version).append(",\n");

            // params
            b.append("  \"params\": [\n");
            for (int i=0;i<s.params.size();i++){
                ParamRow p = s.params.get(i);
                b.append("    {")
                 .append("\"name\":").append(Json.esc(p.name)).append(',')
                 .append("\"unit\":").append(Json.esc(p.unit)).append(',')
                 .append("\"value\":").append(Json.esc(p.value)).append(',')
                 .append("\"desc\":").append(Json.esc(p.desc))
                 .append("}");
                if (i+1<s.params.size()) b.append(',');
                b.append('\n');
            }
            b.append("  ],\n");

            // shapes
            b.append("  \"shapes\": [\n");
            for (int i=0;i<s.shapes.size();i++){
                ShapeDTO sh = s.shapes.get(i);
                b.append("    {")
                 .append("\"id\":").append(Json.esc(sh.id)).append(',')
                 .append("\"kind\":").append(Json.esc(sh.kind)).append(',');
                switch (sh.kind){
                    case "RECT" -> b.append("\"x\":").append(sh.x).append(",\"y\":").append(sh.y)
                                    .append(",\"w\":").append(sh.w).append(",\"h\":").append(sh.h);
                    case "OVAL" -> b.append("\"cx\":").append(sh.cx).append(",\"cy\":").append(sh.cy)
                                    .append(",\"rx\":").append(sh.rx).append(",\"ry\":").append(sh.ry);
                    case "LINE" -> b.append("\"x1\":").append(sh.x1).append(",\"y1\":").append(sh.y1)
                                    .append(",\"x2\":").append(sh.x2).append(",\"y2\":").append(sh.y2);
                    case "POLY" -> {
                        b.append("\"polyPoints\":[");
                        if (sh.polyPoints!=null){
                            for (int k=0;k<sh.polyPoints.size();k++){
                                double[] pt = sh.polyPoints.get(k);
                                b.append("[").append(pt[0]).append(",").append(pt[1]).append("]");
                                if (k+1<sh.polyPoints.size()) b.append(",");
                            }
                        }
                        b.append("]");
                    }
                }
                b.append("}");
                if (i+1<s.shapes.size()) b.append(',');
                b.append('\n');
            }
            b.append("  ],\n");

            // shapeEdgeLines
            b.append("  \"shapeEdgeLines\": {\n");
            writeMapListString(b, s.shapeEdgeLines);
            b.append("  },\n");

            // domainEdits
            b.append("  \"domainEdits\": {\n");
            int idx=0, N=s.domainEdits.size();
            for (java.util.Map.Entry<String, java.util.List<DomainEditDTO>> e : s.domainEdits.entrySet()){
                b.append("    ").append(Json.esc(e.getKey())).append(": [");
                java.util.List<DomainEditDTO> L = e.getValue();
                for (int k=0;k<L.size();k++){
                    DomainEditDTO de = L.get(k);
                    b.append("{\"shapeId\":").append(Json.esc(de.shapeId))
                     .append(",\"op\":").append(Json.esc(de.op)).append("}");
                    if (k+1<L.size()) b.append(',');
                }
                b.append("]");
                if (++idx<N) b.append(',');
                b.append('\n');
            }
            b.append("  },\n");

            // meshSpecs
            b.append("  \"meshSpecs\": {\n");
            idx=0; N=s.meshSpecs.size();
            for (java.util.Map.Entry<String, MeshSpecDTO> e : s.meshSpecs.entrySet()){
                MeshSpecDTO ms = e.getValue();
                b.append("    ").append(Json.esc(e.getKey())).append(": {")
                 .append("\"kind\":").append(Json.esc(ms.kind)).append(',')
                 .append("\"dx\":").append(Json.num(ms.dx)).append(',')
                 .append("\"dy\":").append(Json.num(ms.dy)).append(',')
                 .append("\"dxmin\":").append(Json.num(ms.dxmin)).append(',')
                 .append("\"dxmax\":").append(Json.num(ms.dxmax)).append(',')
                 .append("\"dymin\":").append(Json.num(ms.dymin)).append(',')
                 .append("\"dymax\":").append(Json.num(ms.dymax)).append(',')
                 .append("\"growFromBoundary\":").append(Json.bool(ms.growFromBoundary))
                 .append("}");
                if (++idx<N) b.append(',');
                b.append('\n');
            }
            b.append("  },\n");

            // materials
            b.append("  \"materials\": [");
            for (int i=0;i<s.materials.size();i++){
                if (i>0) b.append(',');
                b.append(Json.esc(s.materials.get(i)));
            }
            b.append("],\n");

            // materialDomains
            b.append("  \"materialDomains\": {\n");
            writeMapSetString(b, s.materialDomains);
            b.append("  },\n");

            // domainOwner
            b.append("  \"domainOwner\": {\n");
            writeMapString(b, s.domainOwner);
            b.append("  },\n");

            // matVars
            b.append("  \"matVars\": {\n");
            idx=0; N=s.matVars.size();
            for (java.util.Map.Entry<String, java.util.Map<String, VarSpecDTO>> e : s.matVars.entrySet()){
                b.append("    ").append(Json.esc(e.getKey())).append(": {");
                int j=0, M=e.getValue().size();
                for (java.util.Map.Entry<String, VarSpecDTO> v : e.getValue().entrySet()){
                    VarSpecDTO vs = v.getValue();
                    b.append(Json.esc(v.getKey())).append(": {")
                     .append("\"useFunc\":").append(Json.bool(vs.useFunc)).append(',')
                     .append("\"funcExpr\":").append(Json.esc(vs.funcExpr)).append(',')
                     .append("\"csvPath\":").append(Json.esc(vs.csvPath)).append(',')
                     .append("\"useT\":").append(Json.bool(vs.useT)).append(',')
                     .append("\"useP\":").append(Json.bool(vs.useP)).append(',')
                     .append("\"useConc\":").append(Json.bool(vs.useConc))
                     .append("}");
                    if (++j<M) b.append(',');
                }
                b.append("}");
                if (++idx<N) b.append(',');
                b.append('\n');
            }
            b.append("  },\n");

            // reactions
            b.append("  \"reactions\": {\n");
            idx=0; N=s.reactions.size();
            for (java.util.Map.Entry<String, java.util.List<ReactionSpecDTO>> e : s.reactions.entrySet()){
                b.append("    ").append(Json.esc(e.getKey())).append(": [");
                java.util.List<ReactionSpecDTO> L = e.getValue();
                for (int k=0;k<L.size();k++){
                    ReactionSpecDTO r = L.get(k);
                    b.append("{\"kExpr\":").append(Json.esc(r.kExpr))
                     .append(",\"energyMeV\":").append(Json.esc(r.energyMeV))
                     .append(",\"products\":").append(Json.esc(r.products))
                     .append(",\"reactants\":").append(Json.esc(r.reactants))
                     .append("}");
                    if (k+1<L.size()) b.append(',');
                }
                b.append("]");
                if (++idx<N) b.append(',');
                b.append('\n');
            }
            b.append("  },\n");

            // boundarySpecs
            b.append("  \"boundarySpecs\": {\n");
            writeMapCLSpec(b, s.boundarySpecs);
            b.append("  },\n");

            // boundaryLines
            b.append("  \"boundaryLines\": {\n");
            writeMapSetString(b, s.boundaryLines);
            b.append("  },\n");

            // initSpecs
            b.append("  \"initSpecs\": {\n");
            writeMapCLSpec(b, s.initSpecs);
            b.append("  },\n");

            // initDomains
            b.append("  \"initDomains\": {\n");
            writeMapSetString(b, s.initDomains);
            b.append("  },\n");

            // solver
            b.append("  \"solver\": {")
             .append("\"timeScheme\":").append(Json.esc(s.solver.timeScheme)).append(',')
             .append("\"linSolver\":").append(Json.esc(s.solver.linSolver)).append(',')
             .append("\"precond\":").append(Json.esc(s.solver.precond)).append(',')
             .append("\"tol\":").append(Json.esc(s.solver.tol)).append(',')
             .append("\"iters\":").append(Json.esc(s.solver.iters))
             .append("}\n");

            b.append("}\n");
            return b.toString();
        }

        // --- helpers pour blocs JSON ---
        private static void writeMapString(StringBuilder b, java.util.Map<String,String> m){
            int i=0, n=m.size();
            for (java.util.Map.Entry<String,String> e : m.entrySet()){
                b.append("    ").append(Json.esc(e.getKey())).append(": ").append(Json.esc(e.getValue()));
                if (++i<n) b.append(',');
                b.append('\n');
            }
        }
        private static void writeMapListString(StringBuilder b, java.util.Map<String,java.util.List<String>> m){
            int i=0, n=m.size();
            for (java.util.Map.Entry<String,java.util.List<String>> e : m.entrySet()){
                b.append("    ").append(Json.esc(e.getKey())).append(": [");
                java.util.List<String> L=e.getValue();
                for (int k=0;k<L.size();k++){ if (k>0) b.append(','); b.append(Json.esc(L.get(k))); }
                b.append("]");
                if (++i<n) b.append(',');
                b.append('\n');
            }
        }
        private static void writeMapSetString(StringBuilder b, java.util.Map<String,java.util.LinkedHashSet<String>> m){
            int i=0, n=m.size();
            for (java.util.Map.Entry<String,java.util.LinkedHashSet<String>> e : m.entrySet()){
                b.append("    ").append(Json.esc(e.getKey())).append(": [");
                java.util.LinkedHashSet<String> S=e.getValue();
                if (S!=null){
                    int k=0, K=S.size();
                    for (String v:S){ if (k++>0) b.append(','); b.append(Json.esc(v)); }
                }
                b.append("]");
                if (++i<n) b.append(',');
                b.append('\n');
            }
        }
        private static void writeMapCLSpec(StringBuilder b, java.util.Map<String,CLSpecDTO> m){
            int i=0, n=m.size();
            for (java.util.Map.Entry<String,CLSpecDTO> e : m.entrySet()){
                CLSpecDTO s=e.getValue();
                b.append("    ").append(Json.esc(e.getKey())).append(": {")
                 .append("\"ux\":").append(Json.esc(s.ux)).append(',')
                 .append("\"uy\":").append(Json.esc(s.uy)).append(',')
                 .append("\"p\":").append(Json.esc(s.p)).append(',')
                 .append("\"T\":").append(Json.esc(s.T)).append(',')
                 .append("\"B\":").append(Json.esc(s.B)).append(',')
                 .append("\"mass\":").append(Json.esc(s.mass)).append(',')
                 .append("\"external\":").append(Json.bool(s.external))
                 .append("}");
                if (++i<n) b.append(',');
                b.append('\n');
            }
        }
    }
}

// =======================================================
//  DÉLÉGUÉ top-level ProjectIO pour compatibilité IOBridge
//  (IOBridge cherchait une classe "ProjectIO" au top-level)
// =======================================================
public final class ProjectIO {
    private ProjectIO() {}
    public static java.io.File openWithChooser(java.awt.Component parent) {
        return MHDPersistence.ProjectIO.openWithChooser(parent);
    }
    public static boolean saveWithChooser(java.awt.Component parent, java.io.File currentFile) {
        return MHDPersistence.ProjectIO.saveWithChooser(parent, currentFile);
    }
    public static java.io.File saveAsWithChooser(java.awt.Component parent) {
        return MHDPersistence.ProjectIO.saveAsWithChooser(parent);
    }
    public static java.io.File chooseSaveFile(java.awt.Component parent) {
        return MHDPersistence.ProjectIO.chooseSaveFile(parent);
    }
    public static boolean saveTo(java.io.File f) {
        return MHDPersistence.ProjectIO.saveTo(f);
    }
    public static java.io.File saveOrSaveAs(java.awt.Component parent, java.io.File currentFile) {
        return MHDPersistence.ProjectIO.saveOrSaveAs(parent, currentFile);
    }
}







In [4]:
// ---- UI
enum Step {
    PARAMETRES("Paramètres"),
    GEOMETRIE("Géométrie"),
    DOMAINES("Domaines"),
    MATERIAUX("Matériaux"),
    CL("Conditions limites"),
    CI("Conditions initiales"),
    SOLVEUR("Solveur"),
    MAILLAGE("Maillage");
    final String label;
    Step(String l){ this.label = l; }
    public String toString(){ return label; }
}

class BoldSectionRenderer extends DefaultListCellRenderer {
    public Component getListCellRendererComponent(JList<?> list, Object value, int index,
                                                  boolean isSelected, boolean cellHasFocus) {
        JLabel c = (JLabel) super.getListCellRendererComponent(list, value, index, isSelected, cellHasFocus);
        c.setFont(c.getFont().deriveFont(Font.BOLD));
        return c;
    }
}

// Helper builders
JComponent[] row(String label, JComponent field){
    JLabel l = new JLabel(label);
    l.setFont(l.getFont().deriveFont(Font.BOLD));
    return new JComponent[]{ l, field };
}

JPanel formGrid(JComponent[]... rows) {
    JPanel panel = new JPanel(new GridBagLayout());
    GridBagConstraints g = new GridBagConstraints();
    g.insets = new Insets(6,6,6,6);
    g.fill = GridBagConstraints.HORIZONTAL;
    for (int i=0;i<rows.length;i++){
        g.gridx = 0; g.gridy = i; g.weightx = 0;
        panel.add(rows[i][0], g);
        g.gridx = 1; g.weightx = 1;
        panel.add(rows[i][1], g);
    }
    return panel;
}

JPanel wrapTitled(JComponent c, String title) {
    JPanel p = new JPanel(new BorderLayout());
    p.setBorder(BorderFactory.createCompoundBorder(
            BorderFactory.createTitledBorder(title),
            new EmptyBorder(8,8,8,8)
    ));
    p.add(c, BorderLayout.CENTER);
    return p;
}

// Screens
JPanel uiParametres() {
    // --- Modèle
    String[] columns = {"Nom", "Unité", "Valeur", "Description"};
    DefaultTableModel model = new DefaultTableModel(columns, 0) {
        @Override public boolean isCellEditable(int r, int c) { return true; }
    };
    Globals.paramsModel = model;
    
    // quelques lignes de test (tu peux enlever ensuite)
    model.addRow(new Object[]{"rho", "kg/m³", "1000", "Masse volumique"});
    model.addRow(new Object[]{"mu", "Pa·s", "1e-3", "Viscosité dynamique"});

    // --- Tableau
    JTable table = new JTable(model);
    table.setFillsViewportHeight(true);
    table.setSelectionMode(ListSelectionModel.SINGLE_SELECTION);
    table.setRowHeight(24);
    table.setShowGrid(true);
    table.setGridColor(Color.LIGHT_GRAY);

    table.getColumnModel().getColumn(0).setPreferredWidth(150);
    table.getColumnModel().getColumn(1).setPreferredWidth(80);
    table.getColumnModel().getColumn(2).setPreferredWidth(80);
    table.getColumnModel().getColumn(3).setPreferredWidth(260);

    JScrollPane scroll = new JScrollPane(table);
    scroll.setBorder(BorderFactory.createTitledBorder("Paramètres du modèle"));

    // --- Barre d’outils (boutons en haut, toujours visibles)
    JToolBar toolbar = new JToolBar();
    toolbar.setFloatable(false);
    JButton addBtn = new JButton("Ajouter");
    JButton delBtn = new JButton("Supprimer");
    toolbar.add(addBtn);
    toolbar.add(delBtn);

    addBtn.addActionListener(e -> {
        model.addRow(new Object[]{"", "", "", ""});
        int r = model.getRowCount() - 1;
        table.changeSelection(r, 0, false, false);
        table.editCellAt(r, 0);
    });
    delBtn.addActionListener(e -> {
        int r = table.getSelectedRow();
        if (r >= 0) model.removeRow(r);
        else JOptionPane.showMessageDialog(table, "Sélectionnez une ligne à supprimer.",
                "Aucune sélection", JOptionPane.WARNING_MESSAGE);
    });

    // Raccourci clavier Suppr
    table.getInputMap(JComponent.WHEN_FOCUSED)
         .put(KeyStroke.getKeyStroke("DELETE"), "delRow");
    table.getActionMap().put("delRow",
        new AbstractAction() { @Override public void actionPerformed(java.awt.event.ActionEvent e) {
            int r = table.getSelectedRow(); if (r >= 0) model.removeRow(r);
        }});

    // --- Panneau final
    JPanel p = new JPanel(new BorderLayout(8, 8));
    p.add(toolbar, BorderLayout.NORTH);   // <-- boutons bien visibles
    p.add(scroll,  BorderLayout.CENTER);
    return p;
}


// --- UI Géométrie : outils + paramètres (cards) + canvas
JPanel uiGeometrie() {
    Canvas2D canvas = new Canvas2D(false, () -> null, Canvas2D.EdgeView.NONE);
    Globals.canvasGeom = canvas;

    // Toolbar outils
    JToolBar tools = new JToolBar(); tools.setFloatable(false);
    JToggleButton selBtn  = new JToggleButton("Sélection");
    JToggleButton rectBtn = new JToggleButton("Rectangle");
    JToggleButton ovalBtn = new JToggleButton("Ovale (centre,Rx,Ry)");
    JToggleButton lineBtn = new JToggleButton("Ligne");
    JButton deleteBtn     = new JButton("Supprimer");
    JButton validateLoopBtn = new JButton("Valider contour");

    ButtonGroup grp = new ButtonGroup();
    for (AbstractButton b : new AbstractButton[]{selBtn, rectBtn, ovalBtn, lineBtn}) grp.add(b);
    selBtn.setSelected(true);

    tools.add(selBtn); tools.add(rectBtn); tools.add(ovalBtn); tools.add(lineBtn);
    tools.addSeparator(); tools.add(deleteBtn);
    tools.addSeparator(); tools.add(validateLoopBtn);

    // --- Champs RECTANGLE
    JTextField sx_rect = new JTextField("60");   // x
    JTextField sy_rect = new JTextField("60");   // y
    JTextField sw_rect = new JTextField("120");  // largeur
    JTextField sh_rect = new JTextField("80");   // hauteur

    // --- Champs OVALE (centre + Rx/Ry)
    JTextField sx_oval = new JTextField("200"); // x centre
    JTextField sy_oval = new JTextField("120"); // y centre
    JTextField sw_oval = new JTextField("80");  // Rx (peut être 'Rayon')
    JTextField sh_oval = new JTextField("40");  // Ry

    // --- Champs LIGNE
    JTextField sx_line  = new JTextField("60");
    JTextField sy_line  = new JTextField("60");
    JTextField sx2_line = new JTextField("220");
    JTextField sy2_line = new JTextField("140");

    // largeur visuelle
    for (JTextField tf : new JTextField[]{
            sx_rect, sy_rect, sw_rect, sh_rect,
            sx_oval, sy_oval, sw_oval, sh_oval,
            sx_line, sy_line, sx2_line, sy2_line
    }) tf.setColumns(6);

    // Cartes de paramètres
    JPanel cardRect = new JPanel(new GridLayout(2,4,6,4));
    cardRect.add(new JLabel("x"));       cardRect.add(sx_rect);
    cardRect.add(new JLabel("y"));       cardRect.add(sy_rect);
    cardRect.add(new JLabel("largeur")); cardRect.add(sw_rect);
    cardRect.add(new JLabel("hauteur")); cardRect.add(sh_rect);

    JPanel cardOval = new JPanel(new GridLayout(2,4,6,4));
    cardOval.add(new JLabel("x centre")); cardOval.add(sx_oval);
    cardOval.add(new JLabel("y centre")); cardOval.add(sy_oval);
    cardOval.add(new JLabel("Rx"));       cardOval.add(sw_oval);
    cardOval.add(new JLabel("Ry"));       cardOval.add(sh_oval);

    JPanel cardLine = new JPanel(new GridLayout(2,4,6,4));
    cardLine.add(new JLabel("x1")); cardLine.add(sx_line);
    cardLine.add(new JLabel("y1")); cardLine.add(sy_line);
    cardLine.add(new JLabel("x2")); cardLine.add(sx2_line);
    cardLine.add(new JLabel("y2")); cardLine.add(sy2_line);

    // Pile de cartes
    JPanel cards = new JPanel(new CardLayout());
    cards.add(cardRect, "RECT");
    cards.add(cardOval, "OVAL");
    cards.add(cardLine, "LINE");

    Runnable refreshCards = () -> {
        CardLayout cl = (CardLayout) cards.getLayout();
        if (rectBtn.isSelected()) cl.show(cards, "RECT");
        else if (ovalBtn.isSelected()) cl.show(cards, "OVAL");
        else if (lineBtn.isSelected()) cl.show(cards, "LINE");
        else cl.show(cards, "RECT");
    };
    selBtn.addActionListener(e -> refreshCards.run());
    rectBtn.addActionListener(e -> refreshCards.run());
    ovalBtn.addActionListener(e -> refreshCards.run());
    lineBtn.addActionListener(e -> refreshCards.run());
    refreshCards.run();

    // Actions
    JButton draw = new JButton("Dessiner");

    // --- Handler "Dessiner"
    draw.addActionListener(e -> {
        if (rectBtn.isSelected()){
            Double X = resolveScalarFromText(textOfField(sx_rect));
            Double Y = resolveScalarFromText(textOfField(sy_rect));
            Double W = resolveScalarFromText(textOfField(sw_rect));
            Double H = resolveScalarFromText(textOfField(sh_rect));
            if (X==null||Y==null||W==null||H==null) {
                JOptionPane.showMessageDialog(cards,
                    "Rectangle : entrez un nombre ou le nom d’un paramètre existant.",
                    "Entrée invalide", JOptionPane.ERROR_MESSAGE);
                return;
            }
            canvas.addRect(X, Y, W, H);

        } else if (ovalBtn.isSelected()){
            Double XC = resolveScalarFromText(textOfField(sx_oval));
            Double YC = resolveScalarFromText(textOfField(sy_oval));
            Double RX = resolveScalarFromText(textOfField(sw_oval));
            Double RY = resolveScalarFromText(textOfField(sh_oval));
            if (XC==null||YC==null||RX==null||RY==null) {
                JOptionPane.showMessageDialog(cards,
                    "Ovale : entrez un nombre ou le nom d’un paramètre existant.",
                    "Entrée invalide", JOptionPane.ERROR_MESSAGE);
                return;
            }
            canvas.addOvalCenter(XC, YC, RX, RY);

        } else if (lineBtn.isSelected()){
            Double X1 = resolveScalarFromText(textOfField(sx_line));
            Double Y1 = resolveScalarFromText(textOfField(sy_line));
            Double X2 = resolveScalarFromText(textOfField(sx2_line));
            Double Y2 = resolveScalarFromText(textOfField(sy2_line));
            if (X1==null||Y1==null||X2==null||Y2==null) {
                JOptionPane.showMessageDialog(cards,
                    "Ligne : entrez un nombre ou le nom d’un paramètre existant.",
                    "Entrée invalide", JOptionPane.ERROR_MESSAGE);
                return;
            }
            canvas.addLine(X1, Y1, X2, Y2);
        }
    });

    // Valider contour -> construit un polygone fermé à partir des lignes sélectionnées
    validateLoopBtn.addActionListener(e -> {
        java.awt.geom.Path2D poly = canvas.buildClosedPathFromSelectedLines();
        if (poly == null) {
            JOptionPane.showMessageDialog(canvas,
                "La sélection de lignes ne forme pas un contour fermé simple.",
                "Contour invalide", JOptionPane.WARNING_MESSAGE);
            return;
        }
        String id = "poly" + (++Globals.polyCount);
        ShapeItem polyItem = new ShapeItem(id, ShapeKind.POLY, poly);
        polyItem.color = new Color(80, 80, 80);
        Globals.shapes.add(polyItem);
        Globals.repaintAll();
    });

    deleteBtn.addActionListener(e -> canvas.deleteSelected());

    JPanel top = new JPanel(new BorderLayout(8,8));
    top.add(tools, BorderLayout.WEST);
    top.add(cards, BorderLayout.CENTER);
    JPanel right = new JPanel(new FlowLayout(FlowLayout.RIGHT, 8, 0));
    right.add(draw);
    top.add(right, BorderLayout.EAST);

    JPanel p = new JPanel(new BorderLayout(8,8));
    p.add(top, BorderLayout.NORTH);
    p.add(new JScrollPane(canvas), BorderLayout.CENTER);
    return p;
}


JPanel uiDomaines() {
    // --- Modèle du tableau Domaines
    String[] cols = {"Nom", "Description", "Figures"};
    final javax.swing.table.DefaultTableModel dm = new javax.swing.table.DefaultTableModel(cols, 0) {
        @Override public boolean isCellEditable(int r, int c) { return c != 2; } // "Figures" non éditable
    };
    Globals.domainsModel = dm;

    final javax.swing.JTable table = new javax.swing.JTable(dm);
    table.setRowHeight(22);
    table.getColumnModel().getColumn(0).setPreferredWidth(160);
    table.getColumnModel().getColumn(1).setPreferredWidth(260);
    table.getColumnModel().getColumn(2).setPreferredWidth(340);

    javax.swing.JButton add = new javax.swing.JButton("Ajouter");
    javax.swing.JButton rm  = new javax.swing.JButton("Supprimer");

    // Ajout d’un domaine
    add.addActionListener(e -> {
        String name = "Domaine_" + (dm.getRowCount() + 1);
        dm.addRow(new Object[]{name, "", ""});
        int r = dm.getRowCount() - 1;
        table.changeSelection(r, 0, false, false);
        Globals.domainAreas.putIfAbsent(name, new java.awt.geom.Area());
        Globals.domainEdits.putIfAbsent(name, new java.util.ArrayList<Globals.DomainEdit>());
        Globals.meshSpecs.putIfAbsent(name, new Globals.MeshSpec());
        Globals.repaintAll();
    });

    // Suppression d’un domaine (nettoyage complet)
    rm.addActionListener(e -> {
        int r = table.getSelectedRow();
        if (r >= 0) {
            String name = java.util.Objects.toString(dm.getValueAt(r, 0), "");
            dm.removeRow(r);
            // Nettoyage structures
            Globals.domainAreas.remove(name);
            Globals.domainEdits.remove(name);
            Globals.meshSpecs.remove(name);
            // Nettoyage côté Matériaux (si affecté)
            for (java.util.Map.Entry<String, java.util.LinkedHashSet<String>> en : Globals.materialDomains.entrySet()) {
                if (en.getValue() != null) en.getValue().remove(name);
            }
            if (name.equals(Globals.domainOwner.get(name))) {
                Globals.domainOwner.remove(name);
            } else {
                // sinon, purge toutes les entrées dont la valeur == name
                for (java.util.Map.Entry<String,String> en : new java.util.ArrayList<>(Globals.domainOwner.entrySet())) {
                    if (name.equals(en.getKey()) || name.equals(en.getValue())) {
                        Globals.domainOwner.remove(en.getKey());
                    }
                }
            }
            Globals.repaintAll();
        }
    });

    // Domaine actif
    java.util.function.Supplier<String> currentDomain = () -> {
        int r = table.getSelectedRow();
        return (r >= 0) ? java.util.Objects.toString(dm.getValueAt(r, 0), "").trim() : null;
    };

    // Canvas filtré (mode domaine)
    Canvas2D canvas = new Canvas2D(true, currentDomain, Canvas2D.EdgeView.NONE);
    Globals.canvasDom = canvas;

    // Raccourcis signe
    java.util.function.Function<Globals.DomOp, String> opSign = (op) -> {
        switch (op) {
            case UNION: return "+";
            case SUB:   return "−";
            case INTER: return "∩";
            case XOR:   return "⊕";
            default:    return "?";
        }
    };

    // Maj colonne "Figures" pour un domaine
    java.util.function.Consumer<String> refreshFiguresCol = (dom) -> {
        java.util.Map<String, Globals.DomOp> last = Globals.lastOpsForDomain(dom);
        java.util.List<String> parts = new java.util.ArrayList<>();
        for (java.util.Map.Entry<String, Globals.DomOp> e : last.entrySet())
            parts.add(opSign.apply(e.getValue()) + e.getKey());
        java.util.Collections.sort(parts);
        for (int r = 0; r < dm.getRowCount(); r++) {
            String name = java.util.Objects.toString(dm.getValueAt(r,0), "");
            if (dom.equals(name)) {
                dm.setValueAt(String.join(", ", parts), r, 2);
                break;
            }
        }
    };

    // Exécution d’une opération booléenne sur le domaine actif
    java.util.function.Consumer<Globals.DomOp> runOp = (whichOp) -> {
        int r = table.getSelectedRow();
        if (r < 0) {
            javax.swing.JOptionPane.showMessageDialog(table, "Sélectionnez un domaine dans le tableau.", "Aucun domaine", javax.swing.JOptionPane.WARNING_MESSAGE);
            return;
        }
        String dom = java.util.Objects.toString(dm.getValueAt(r,0),"").trim();
        if (dom.isEmpty()){
            javax.swing.JOptionPane.showMessageDialog(table, "Nom de domaine vide.", "Erreur", javax.swing.JOptionPane.ERROR_MESSAGE);
            return;
        }
        java.awt.geom.Area sel = Canvas2D.areaOfSelectedClosed();
        if (sel == null){
            javax.swing.JOptionPane.showMessageDialog(table, "Sélectionnez au moins une forme fermée (rectangle, ovale, polygone).", "Sélection invalide", javax.swing.JOptionPane.WARNING_MESSAGE);
            return;
        }
        java.awt.geom.Area cur = Globals.domainAreas.get(dom);
        if (cur == null) cur = new java.awt.geom.Area();
        java.awt.geom.Area B = new java.awt.geom.Area(sel);
        switch (whichOp){
            case UNION: cur.add(B); break;
            case SUB:   cur.subtract(B); break;
            case INTER: cur.intersect(B); break;
            case XOR:   cur.exclusiveOr(B); break;
        }
        Globals.domainAreas.put(dom, cur);

        java.util.List<Globals.DomainEdit> hist = Globals.domainEdits.computeIfAbsent(dom, k -> new java.util.ArrayList<>());
        for (ShapeItem it : Globals.shapes){
            if (it.selected && it.kind != ShapeKind.LINE){
                hist.add(new Globals.DomainEdit(it.id, whichOp));
            }
        }
        refreshFiguresCol.accept(dom);
        // Informer tous les écrans dépendants (dont le canvas CI)
        Globals.fireDomainsChanged();
    };

    javax.swing.JButton opUnion = new javax.swing.JButton("Union (+)");
    javax.swing.JButton opDiff  = new javax.swing.JButton("Soustraction (−)");
    javax.swing.JButton opInter = new javax.swing.JButton("Intersection (∩)");
    javax.swing.JButton opXor   = new javax.swing.JButton("XOR (⊕)");

    opUnion.addActionListener(e -> runOp.accept(Globals.DomOp.UNION));
    opDiff .addActionListener(e -> runOp.accept(Globals.DomOp.SUB));
    opInter.addActionListener(e -> runOp.accept(Globals.DomOp.INTER));
    opXor  .addActionListener(e -> runOp.accept(Globals.DomOp.XOR));

    // Listener de RENOMMAGE : déplace aussi l’entrée dans meshSpecs
    dm.addTableModelListener(ev -> {
        if (ev.getType() == javax.swing.event.TableModelEvent.UPDATE && ev.getColumn() == 0) {
            int r = ev.getFirstRow();
            if (r < 0 || r >= dm.getRowCount()) return;

            // Nouveau nom
            String newName = java.util.Objects.toString(dm.getValueAt(r, 0), "").trim();
            if (newName.isEmpty()) return;

            // Liste des noms présents dans la table
            java.util.Set<String> names = new java.util.LinkedHashSet<>();
            for (int i = 0; i < dm.getRowCount(); i++) {
                Object v = dm.getValueAt(i, 0);
                if (v != null && !v.toString().trim().isEmpty()) names.add(v.toString().trim());
            }

            // S’assurer que domainAreas/domainEdits existent pour chaque nom
            for (String n : names) {
                Globals.domainAreas.putIfAbsent(n, new java.awt.geom.Area());
                Globals.domainEdits.putIfAbsent(n, new java.util.ArrayList<Globals.DomainEdit>());
                Globals.meshSpecs.putIfAbsent(n, new Globals.MeshSpec());
            }

            // Trouver l'ancien nom : on lit l'édition depuis l'événement
            // Pour fiabiliser : si exactement une clé de domainAreas n'est plus dans 'names', on la considère comme l'ancien nom.
            java.util.List<String> disappeared = new java.util.ArrayList<>();
            for (String k : new java.util.ArrayList<>(Globals.domainAreas.keySet())) {
                if (!names.contains(k)) disappeared.add(k);
            }
            if (disappeared.size() == 1) {
                String oldName = disappeared.get(0);

                // Déplacer Areas
                java.awt.geom.Area A = Globals.domainAreas.remove(oldName);
                if (A != null) Globals.domainAreas.put(newName, A);

                // Déplacer Edits
                java.util.List<Globals.DomainEdit> L = Globals.domainEdits.remove(oldName);
                if (L != null) Globals.domainEdits.put(newName, L);

                // 🔧 Déplacer MeshSpec
                Globals.MeshSpec ms = Globals.meshSpecs.remove(oldName);
                if (ms != null) Globals.meshSpecs.put(newName, ms);

                // MAJ matériel : propriétaire & affectations
                String owner = Globals.domainOwner.remove(oldName);
                if (owner != null) Globals.domainOwner.put(newName, owner);
                for (java.util.Map.Entry<String, java.util.LinkedHashSet<String>> en : Globals.materialDomains.entrySet()) {
                    if (en.getValue() != null) {
                        if (en.getValue().remove(oldName)) en.getValue().add(newName);
                    }
                }
            }

            // Rafraîchir Figures pour le nouveau nom
            refreshFiguresCol.accept(newName);
            Globals.repaintAll();
        }
    });

    // Layout
    javax.swing.JPanel ops = new javax.swing.JPanel(new java.awt.FlowLayout(java.awt.FlowLayout.LEFT, 6, 0));
    ops.add(opUnion); ops.add(opDiff); ops.add(opInter); ops.add(opXor);

    javax.swing.JPanel btns = new javax.swing.JPanel(new java.awt.FlowLayout(java.awt.FlowLayout.LEFT, 8, 0));
    btns.add(add); btns.add(rm);

    javax.swing.JPanel leftTop = new javax.swing.JPanel(new java.awt.BorderLayout(6, 6));
    leftTop.add(btns, java.awt.BorderLayout.NORTH);
    leftTop.add(ops,  java.awt.BorderLayout.SOUTH);

    javax.swing.JPanel leftStack = new javax.swing.JPanel(new java.awt.BorderLayout(6, 6));
    leftStack.add(leftTop, java.awt.BorderLayout.NORTH);
    leftStack.add(new javax.swing.JScrollPane(table), java.awt.BorderLayout.CENTER);

    javax.swing.JPanel left = new javax.swing.JPanel(new java.awt.BorderLayout(6, 6));
    left.add(new javax.swing.JLabel("Domaines"), java.awt.BorderLayout.NORTH);
    left.add(leftStack, java.awt.BorderLayout.CENTER);
    left.setPreferredSize(new java.awt.Dimension(420, 0));

    javax.swing.JPanel root = new javax.swing.JPanel(new java.awt.BorderLayout(8, 8));
    root.add(left, java.awt.BorderLayout.WEST);
    root.add(new javax.swing.JScrollPane(canvas), java.awt.BorderLayout.CENTER);

    // Repaint quand on change de sélection
    table.getSelectionModel().addListSelectionListener(ev -> {
        if (!ev.getValueIsAdjusting()) Globals.repaintAll();
    });

    return root;
}


// --- UI Matériaux : gestion matériaux, propriétés, réactions, et sélection de domaines
JPanel uiMateriaux() {

    // ==== ÉTAT GLOBAL MATÉRIAUX (garanties) ====
    if (Globals.materialsModel == null) {
        Globals.materialsModel = new DefaultComboBoxModel<>();
        Globals.ensureMaterial("Matériau_1");
    }
    if (Globals.materialDomains == null) Globals.materialDomains = new LinkedHashMap<>();
    if (Globals.domainOwner == null) Globals.domainOwner = new LinkedHashMap<>();

    // ====== Sélecteur de matériau (tête) ======
    JComboBox<String> cbMaterials = new JComboBox<>(Globals.materialsModel);
    cbMaterials.setPreferredSize(new Dimension(260, 26));
    JButton btnAddMat = new JButton("+");
    JButton btnDelMat = new JButton("-");
    btnAddMat.setMargin(new Insets(2,8,2,8));
    btnDelMat.setMargin(new Insets(2,8,2,8));

    JTextField tfName = new JTextField(18);
    JButton btnRename = new JButton("Renommer");

    // ====== Options de base ======
    JCheckBox chkAlloy = new JCheckBox("Alliage ?");
    JTextField tfConcVars = new JTextField(24); // "cA;cB;..."
    JLabel labMolMasses = new JLabel("M (g/mol)");
    JTextField tfMolMasses = new JTextField(18);
    tfConcVars.setEnabled(false);
    labMolMasses.setEnabled(false);
    tfMolMasses.setEnabled(false);

    JCheckBox chkTherm = new JCheckBox("Thermique");
    JCheckBox chkMag   = new JCheckBox("Magnétique");
    JCheckBox chkReact = new JCheckBox("Réactions chimiques/nucléaires");

    // Mise en évidence des champs vides (lorsque activés)
    java.util.function.Consumer<JTextField> markValid = (t) -> {
        boolean en = t.isEnabled();
        Color bg = (!en) ? UIManager.getColor("TextField.background")
                         : (t.getText().trim().isEmpty() ? new Color(255,220,220) : Color.white);
        t.setBackground(bg);
    };
    for (JTextField t : new JTextField[]{tfMolMasses, tfConcVars}) {
        t.getDocument().addDocumentListener(new javax.swing.event.DocumentListener() {
            void go(){ markValid.accept(t); }
            public void insertUpdate(javax.swing.event.DocumentEvent e){ go(); }
            public void removeUpdate(javax.swing.event.DocumentEvent e){ go(); }
            public void changedUpdate(javax.swing.event.DocumentEvent e){ go(); }
        });
        markValid.accept(t);
    }

    // ====== LISTE DES VARIABLES ======
    DefaultListModel<String> varModel = new DefaultListModel<>();
    JList<String> varList = new JList<>(varModel);
    varList.setVisibleRowCount(12);
    varList.setFixedCellHeight(22);

    java.util.List<String> magVars = java.util.Arrays.asList("sigma_e","mu_r","chi_m");

    Runnable refreshVarList = () -> {
        varModel.clear();
        varModel.addElement("rho");
        varModel.addElement("mu");
        if (chkTherm.isSelected()) {
            varModel.addElement("k");
            varModel.addElement("cp");
        }
        if (chkMag.isSelected()) for (String v:magVars) varModel.addElement(v);
        if (chkReact.isSelected()) varModel.addElement("Réactions");
        if (!varModel.isEmpty()) varList.setSelectedIndex(0);
    };

    // ====== ÉDITEUR (Fonction/Table) ======
    JRadioButton rbFunc = new JRadioButton("Fonction:", true);
    JRadioButton rbTable = new JRadioButton("Table d'interpolation:");
    ButtonGroup grpMode = new ButtonGroup();
    grpMode.add(rbFunc);
    grpMode.add(rbTable);

    JTextField tfFunc = new JTextField(32);

    JTextField tfCSVPath = new JTextField(28);
    tfCSVPath.setEditable(false);
    JButton btnPickCSV = new JButton("…");

    JCheckBox cbUseT = new JCheckBox("Variable de T");
    JCheckBox cbUseP = new JCheckBox("Variable de P");
    JCheckBox cbUseConc = new JCheckBox("Variable(s) de % massiques");

    JPanel editorVars = new JPanel(new GridBagLayout());
    GridBagConstraints ge = new GridBagConstraints();
    ge.insets = new Insets(4,8,4,8);
    ge.anchor = GridBagConstraints.WEST;

    ge.gridx=0; ge.gridy=0; editorVars.add(rbFunc, ge);
    ge.gridx=2; ge.gridy=0; editorVars.add(rbTable, ge);

    ge.gridx=0; ge.gridy=1; ge.gridwidth=5; ge.fill = GridBagConstraints.HORIZONTAL;
    editorVars.add(tfFunc, ge);

    ge.gridx=3; ge.gridy=0; ge.gridwidth=1; ge.fill = GridBagConstraints.HORIZONTAL;
    editorVars.add(tfCSVPath, ge);
    ge.gridx=4; ge.gridy=0; ge.fill = GridBagConstraints.NONE;
    editorVars.add(btnPickCSV, ge);

    ge.gridx=3; ge.gridy=2; editorVars.add(cbUseT, ge);
    ge.gridy=3; editorVars.add(cbUseP, ge);
    ge.gridy=4; editorVars.add(cbUseConc, ge);

    Runnable refreshEditorEnable = () -> {
        boolean useFunc = rbFunc.isSelected();
        tfFunc.setEnabled(useFunc);

        boolean useTable = rbTable.isSelected();
        tfCSVPath.setEnabled(useTable);
        btnPickCSV.setEnabled(useTable);
        cbUseT.setEnabled(useTable);
        cbUseP.setEnabled(useTable);
        cbUseConc.setEnabled(useTable);
    };
    rbFunc.addActionListener(e -> refreshEditorEnable.run());
    rbTable.addActionListener(e -> refreshEditorEnable.run());
    refreshEditorEnable.run();

    btnPickCSV.addActionListener(e -> {
        JFileChooser ch = new JFileChooser();
        ch.setDialogTitle("Choisir un fichier CSV (interpolation)");
        if (ch.showOpenDialog(editorVars) == JFileChooser.APPROVE_OPTION) {
            tfCSVPath.setText(ch.getSelectedFile().getAbsolutePath());
            rbTable.setSelected(true);
            refreshEditorEnable.run();
        }
    });

    // ====== CARTE « RÉACTIONS » ======
    DefaultListModel<String> reactModel = new DefaultListModel<>();
    JList<String> reactList = new JList<>(reactModel);
    reactList.setVisibleRowCount(6);
    JButton btnAddR = new JButton("+");
    JButton btnDelR = new JButton("-");

    JTextField tfKexpr = new JTextField(18);
    JTextField tfEmev  = new JTextField(8);
    JTextField tfProd  = new JTextField(24);
    JTextField tfReac  = new JTextField(24);

    JPanel reactionsCard = new JPanel(new GridBagLayout());
    reactionsCard.setBorder(BorderFactory.createTitledBorder("Liste Réactions"));
    GridBagConstraints gr = new GridBagConstraints();
    gr.insets = new Insets(4,8,4,8);
    gr.anchor = GridBagConstraints.WEST;

    gr.gridx=0; gr.gridy=0; reactionsCard.add(new JScrollPane(reactList), gr);
    JPanel rBtns = new JPanel(new FlowLayout(FlowLayout.LEFT,6,0));
    rBtns.add(btnAddR); rBtns.add(btnDelR);
    gr.gridx=1; reactionsCard.add(rBtns, gr);

    gr.gridx=0; gr.gridy=1; reactionsCard.add(new JLabel("k (expression)"), gr);
    gr.gridx=1; reactionsCard.add(tfKexpr, gr);

    gr.gridx=0; gr.gridy=2; reactionsCard.add(new JLabel("Énergie de réaction (MeV)"), gr);
    gr.gridx=1; reactionsCard.add(tfEmev, gr);

    gr.gridx=0; gr.gridy=3; reactionsCard.add(new JLabel("Liste espèces produit :"), gr);
    gr.gridx=1; reactionsCard.add(tfProd, gr);

    gr.gridx=0; gr.gridy=4; reactionsCard.add(new JLabel("Liste espèces réactive :"), gr);
    gr.gridx=1; reactionsCard.add(tfReac, gr);

    // ====== PILE (éditeur / réactions) ======
    JPanel rightTopStack = new JPanel(new CardLayout());
    rightTopStack.add(editorVars, "EDITOR");
    rightTopStack.add(reactionsCard,"REACTIONS");

    // ====== PERSISTANCE : Réactions par matériau ======
    java.util.function.Supplier<String> curMatForReact = () -> (String) cbMaterials.getSelectedItem();
    final int[] lastReactIndex = { 0 };

    Runnable refreshReactListForMaterial = () -> {
        String m = curMatForReact.get();
        reactModel.clear();
        if (m == null) return;
        java.util.List<Globals.ReactionSpec> L = Globals.getReactions(m);
        if (L.isEmpty()) L.add(new Globals.ReactionSpec());
        for (int i=0;i<L.size();i++) reactModel.addElement("Réaction_" + (i+1));
        int sel = Math.min(lastReactIndex[0], reactModel.size()-1);
        if (sel < 0) sel = 0;
        reactList.setSelectedIndex(sel);
    };

    Runnable loadReactionFromState = () -> {
        String m = curMatForReact.get();
        if (m == null) return;
        int idx = reactList.getSelectedIndex();
        if (idx < 0) idx = 0;
        java.util.List<Globals.ReactionSpec> L = Globals.getReactions(m);
        if (idx >= L.size()) return;
        Globals.ReactionSpec r = L.get(idx);
        tfKexpr.setText(r.kExpr);
        tfEmev.setText(r.energyMeV);
        tfProd.setText(r.products);
        tfReac.setText(r.reactants);
        lastReactIndex[0] = idx;
    };

    Runnable saveReactionToState = () -> {
        String m = curMatForReact.get();
        if (m == null) return;
        int idx = reactList.getSelectedIndex();
        if (idx < 0) return;
        java.util.List<Globals.ReactionSpec> L = Globals.getReactions(m);
        while (idx >= L.size()) L.add(new Globals.ReactionSpec());
        Globals.ReactionSpec r = L.get(idx);
        r.kExpr     = tfKexpr.getText().trim();
        r.energyMeV = tfEmev.getText().trim();
        r.products  = tfProd.getText().trim();
        r.reactants = tfReac.getText().trim();
    };

    // sauvegarde "live"
    java.awt.event.KeyAdapter saveReactKA = new java.awt.event.KeyAdapter() {
        @Override public void keyReleased(java.awt.event.KeyEvent e){ saveReactionToState.run(); }
    };
    for (JTextField t : new JTextField[]{tfKexpr, tfEmev, tfProd, tfReac}) t.addKeyListener(saveReactKA);

    // sélection réaction
    reactList.addListSelectionListener(e -> {
        if (e.getValueIsAdjusting()) return;
        saveReactionToState.run();
        loadReactionFromState.run();
    });

    // + / -
    btnAddR.addActionListener(e -> {
        String m = curMatForReact.get(); if (m == null) return;
        saveReactionToState.run();
        java.util.List<Globals.ReactionSpec> L = Globals.getReactions(m);
        L.add(new Globals.ReactionSpec());
        refreshReactListForMaterial.run();
        reactList.setSelectedIndex(L.size()-1);
        loadReactionFromState.run();
    });
    btnDelR.addActionListener(e -> {
        String m = curMatForReact.get(); if (m == null) return;
        int idx = reactList.getSelectedIndex();
        java.util.List<Globals.ReactionSpec> L = Globals.getReactions(m);
        if (idx >= 0 && idx < L.size()) {
            L.remove(idx);
            if (L.isEmpty()) L.add(new Globals.ReactionSpec());
            lastReactIndex[0] = Math.max(0, idx-1);
            refreshReactListForMaterial.run();
            loadReactionFromState.run();
        }
    });

    // ====== PERSISTANCE : Éditeur Fonction/Table par (matériau, variable) ======
    java.util.function.Supplier<String> currentMaterial = () -> (String) cbMaterials.getSelectedItem();
    java.util.function.Supplier<String> currentVariable = () -> varList.getSelectedValue();
    final String[] lastMat = { null };
    final String[] lastVar = { null };

    Runnable saveEditorToState = () -> {
        String m = lastMat[0], v = lastVar[0];
        if (m == null || v == null || "Réactions".equals(v)) return;
        Globals.VarSpec s = Globals.getVarSpec(m, v);
        s.useFunc = rbFunc.isSelected();
        s.funcExpr = tfFunc.getText().trim();
        s.csvPath = tfCSVPath.getText().trim();
        s.useT = cbUseT.isSelected();
        s.useP = cbUseP.isSelected();
        s.useConc = cbUseConc.isSelected();
    };

    Runnable loadEditorFromState = () -> {
        String m = currentMaterial.get();
        String v = currentVariable.get();
        lastMat[0] = m;
        lastVar[0] = v;

        CardLayout cl = (CardLayout) rightTopStack.getLayout();
        if ("Réactions".equals(v)) {
            cl.show(rightTopStack, "REACTIONS");
            refreshReactListForMaterial.run();
            loadReactionFromState.run();
            return;
        }
        cl.show(rightTopStack, "EDITOR");
        if (m == null || v == null) return;
        Globals.VarSpec s = Globals.getVarSpec(m, v);
        rbFunc.setSelected(s.useFunc);
        rbTable.setSelected(!s.useFunc);
        tfFunc.setText(s.funcExpr == null ? "" : s.funcExpr);
        tfCSVPath.setText(s.csvPath == null ? "" : s.csvPath);
        cbUseT.setSelected(s.useT);
        cbUseP.setSelected(s.useP);
        cbUseConc.setSelected(s.useConc);
        refreshEditorEnable.run();
    };

    // save-on-change
    java.awt.event.ActionListener saveAL = e -> saveEditorToState.run();
    java.awt.event.ItemListener    saveIL = e -> saveEditorToState.run();
    java.awt.event.KeyAdapter      saveKA = new java.awt.event.KeyAdapter() {
        @Override public void keyReleased(java.awt.event.KeyEvent e){ saveEditorToState.run(); }
    };
    rbFunc.addActionListener(saveAL);
    rbTable.addActionListener(saveAL);
    cbUseT.addItemListener(saveIL);
    cbUseP.addItemListener(saveIL);
    cbUseConc.addItemListener(saveIL);
    tfFunc.addKeyListener(saveKA);
    tfCSVPath.addKeyListener(saveKA);

    // changement variable / matériau
    varList.addListSelectionListener(e -> {
        if (e.getValueIsAdjusting()) return;
        saveEditorToState.run();
        loadEditorFromState.run();
    });
    cbMaterials.addActionListener(e -> {
        saveEditorToState.run();
        saveReactionToState.run();
        loadEditorFromState.run(); // charge EDITOR ou REACTIONS selon var
    });

    // ====== CANVAS Domaines & liste ======
    JTextArea taSelectedDomains = new JTextArea(2, 28);
    taSelectedDomains.setEditable(false);
    taSelectedDomains.setLineWrap(true);
    taSelectedDomains.setWrapStyleWord(true);
    taSelectedDomains.setBorder(BorderFactory.createTitledBorder("Domaines sélectionnés"));

    final java.util.concurrent.atomic.AtomicReference<java.util.function.Consumer<String>> refreshSelectedDomainsRef = new java.util.concurrent.atomic.AtomicReference<>(null);

    class DomainCanvas extends JPanel {
        final java.util.function.Supplier<String> curMat;
        AffineTransform worldToScreen = new AffineTransform();
        AffineTransform screenToWorld = new AffineTransform();

        DomainCanvas(java.util.function.Supplier<String> curMat){
            this.curMat = curMat;
            setPreferredSize(new Dimension(560, 260));
            setBorder(BorderFactory.createTitledBorder("Affectation des domaines au matériau"));
            setBackground(new Color(246,246,246));

            addComponentListener(new ComponentAdapter() {
                @Override public void componentResized(ComponentEvent e) { fitAll(); repaint(); }
            });

            addMouseListener(new java.awt.event.MouseAdapter() {
                @Override public void mouseClicked(java.awt.event.MouseEvent e) {
                    String m = curMat.get(); if (m == null) return;

                    java.awt.geom.Point2D ptS = e.getPoint();
                    java.awt.geom.Point2D ptW = new java.awt.geom.Point2D.Double();
                    try { screenToWorld.transform(ptS, ptW); } catch (Exception ex) { return; }

                    for (java.util.Map.Entry<String, java.awt.geom.Area> en : Globals.domainAreas.entrySet()) {
                        String dom = en.getKey(); java.awt.geom.Area A = en.getValue();
                        if (A == null) continue;
                        if (A.contains(ptW)) {
                            String owner = Globals.domainOwner.get(dom);
                            if (owner != null && !owner.equals(m)) {
                                JOptionPane.showMessageDialog(DomainCanvas.this,
                                    "Le domaine \"" + dom + "\" est déjà affecté à « " + owner + " ».\n"
                                  + "Désélectionnez-le d’abord dans ce matériau pour le réutiliser.",
                                    "Domaine déjà utilisé", JOptionPane.WARNING_MESSAGE);
                                return;
                            }
                            java.util.LinkedHashSet<String> set =
                                Globals.materialDomains.computeIfAbsent(m, k -> new java.util.LinkedHashSet<>());
                            if (set.contains(dom)) {
                                set.remove(dom);
                                Globals.domainOwner.remove(dom);
                            } else {
                                set.add(dom);
                                Globals.domainOwner.put(dom, m);
                            }
                            java.util.function.Consumer<String> r = refreshSelectedDomainsRef.get();
                            if (r != null) r.accept(m);
                            repaint();
                            break;
                        }
                    }
                }
            });
        }

        void fitAll(){
            Rectangle2D bounds = null;
            for (java.awt.geom.Area A : Globals.domainAreas.values()){
                if (A == null || A.isEmpty()) continue;
                Rectangle2D b = A.getBounds2D();
                bounds = (bounds == null) ? (Rectangle2D) b.clone() : bounds.createUnion(b);
            }
            if (bounds == null) bounds = new Rectangle2D.Double(0,0,400,300);

            Insets in = getInsets();
            int W = Math.max(1, getWidth()-in.left-in.right-16);
            int H = Math.max(1, getHeight()-in.top -in.bottom-40);
            double sx = W / bounds.getWidth();
            double sy = H / bounds.getHeight();
            double s  = 0.9 * Math.min(sx, sy);

            double tx = (getWidth() - s*bounds.getWidth()) * 0.5 - s*bounds.getX();
            double ty = (getHeight() - s*bounds.getHeight()) * 0.5 - s*bounds.getY();

            worldToScreen = new AffineTransform();
            worldToScreen.translate(tx, ty);
            worldToScreen.scale(s, s);

            try { screenToWorld = worldToScreen.createInverse(); }
            catch (NoninvertibleTransformException ex) { screenToWorld = new AffineTransform(); }
        }

        @Override protected void paintComponent(Graphics g){
            super.paintComponent(g);
            if (worldToScreen == null) fitAll();

            Graphics2D g2 = (Graphics2D) g.create();
            g2.setRenderingHint(RenderingHints.KEY_ANTIALIASING, RenderingHints.VALUE_ANTIALIAS_ON);

            // grille
            g2.setTransform(worldToScreen);
            g2.setStroke(new BasicStroke(1f / (float) worldToScreen.getScaleX()));
            for (int x=-1000; x<=2000; x+=100){
                g2.setColor(new Color(210,210,210));
                g2.draw(new Line2D.Double(x, -1000, x, 2000));
                for (int xx=x+20; xx<x+100; xx+=20){
                    g2.setColor(new Color(235,235,235));
                    g2.draw(new Line2D.Double(xx, -1000, xx, 2000));
                }
            }
            for (int y=-1000; y<=2000; y+=100){
                g2.setColor(new Color(210,210,210));
                g2.draw(new Line2D.Double(-1000, y, 2000, y));
                for (int yy=y+20; yy<y+100; yy+=20){
                    g2.setColor(new Color(235,235,235));
                    g2.draw(new Line2D.Double(-1000, yy, 2000, yy));
                }
            }
            g2.setColor(new Color(120,120,120));
            g2.setStroke(new BasicStroke(1.4f / (float) worldToScreen.getScaleX()));
            g2.draw(new Line2D.Double(-1000, 0, 2000, 0));
            g2.draw(new Line2D.Double(0, -1000, 0, 2000));

            String m = curMat.get();
            Set<String> mine = (m == null) ? Collections.emptySet()
                                           : Globals.materialDomains.getOrDefault(m, new LinkedHashSet<>());

            for (Map.Entry<String, java.awt.geom.Area> en : Globals.domainAreas.entrySet()){
                String dom = en.getKey(); java.awt.geom.Area A = en.getValue();
                if (A == null || A.isEmpty()) continue;

                String owner = Globals.domainOwner.get(dom);
                boolean isMine = m != null && mine.contains(dom);

                if (isMine) {
                    g2.setComposite(AlphaComposite.SrcOver.derive(0.22f));
                    g2.setColor(new Color(40,170,80));
                } else if (owner != null && (m==null || !owner.equals(m))) {
                    g2.setComposite(AlphaComposite.SrcOver.derive(0.10f));
                    g2.setColor(new Color(180,60,60));
                } else {
                    g2.setComposite(AlphaComposite.SrcOver.derive(0.06f));
                    g2.setColor(new Color(50,50,50));
                }
                g2.fill(A);
                g2.setComposite(AlphaComposite.SrcOver);
                if (isMine) g2.setColor(new Color(40,170,80));
                else if (owner != null && (m==null || !owner.equals(m))) g2.setColor(new Color(180,60,60));
                else g2.setColor(new Color(100,100,100));
                g2.setStroke(new BasicStroke(1.8f / (float) worldToScreen.getScaleX()));
                g2.draw(A);

                Rectangle b = A.getBounds();
                g2.setColor(new Color(60,60,60));
                g2.scale(1.0/worldToScreen.getScaleX(), 1.0/worldToScreen.getScaleY());
                Point2D p = worldToScreen.transform(new Point2D.Double(b.getMaxX(), b.getMinY()), null);
                g2.drawString(dom, (float)p.getX()+4, (float)p.getY()+12);
                g2.setTransform(worldToScreen);
            }
            g2.dispose();
        }
    }

    DomainCanvas domainCanvas = new DomainCanvas(currentMaterial);

    java.util.function.Consumer<String> refreshSelectedDomains = (mat) -> {
        LinkedHashSet<String> set = Globals.materialDomains.getOrDefault(mat, new LinkedHashSet<>());
        taSelectedDomains.setText(String.join(", ", set));
    };
    refreshSelectedDomainsRef.set(refreshSelectedDomains);

    // ====== Interactions matériaux ======
    Runnable loadMaterialIntoForm = () -> {
        String m = currentMaterial.get(); if (m == null) return;
        tfName.setText(m);
        refreshSelectedDomains.accept(m);
        domainCanvas.fitAll();
        domainCanvas.repaint();
        // Synchroniser la carte affichée/chargée selon la variable courante
        loadEditorFromState.run();
    };
    cbMaterials.addActionListener(e -> loadMaterialIntoForm.run());

    btnAddMat.addActionListener(e -> {
        String base = "Matériau_";
        int i = Globals.materialsModel.getSize() + 1;
        String name;
        while (true) {
            name = base + i++;
            boolean exist = false;
            for (int k=0;k<Globals.materialsModel.getSize();k++)
                if (name.equals(Globals.materialsModel.getElementAt(k))) { exist=true; break; }
            if (!exist) break;
        }
        Globals.ensureMaterial(name);
        cbMaterials.setSelectedItem(name);
        loadMaterialIntoForm.run();
    });

    btnDelMat.addActionListener(e -> {
        if (Globals.materialsModel.getSize() <= 1) return;
        String first = Globals.materialsModel.getElementAt(0);
        String cur = (String) cbMaterials.getSelectedItem();
        if (first.equals(cur)) {
            JOptionPane.showMessageDialog(cbMaterials, "Le premier matériau n'est pas supprimable.",
                    "Action refusée", JOptionPane.WARNING_MESSAGE);
            return;
        }
        Set<String> owned = Globals.materialDomains.getOrDefault(cur, new LinkedHashSet<>());
        for (String d : owned)
            if (cur.equals(Globals.domainOwner.get(d))) Globals.domainOwner.remove(d);
        Globals.materialDomains.remove(cur);
        Globals.matReactions.remove(cur); // supprime aussi les réactions de ce matériau
        Globals.materialsModel.removeElement(cur);
        cbMaterials.setSelectedItem(first);
        loadMaterialIntoForm.run();
        Globals.repaintAll();
    });

    btnRename.addActionListener(e -> {
        String old = (String) cbMaterials.getSelectedItem();
        String now = tfName.getText().trim();
        if (now.isEmpty()) return;
        for (int i=0;i<Globals.materialsModel.getSize();i++){
            if (now.equals(Globals.materialsModel.getElementAt(i))){
                JOptionPane.showMessageDialog(cbMaterials, "Un matériau du même nom existe déjà.",
                        "Doublon", JOptionPane.ERROR_MESSAGE);
                return;
            }
        }
        int idx = cbMaterials.getSelectedIndex();
        Globals.materialsModel.removeElementAt(idx);
        Globals.materialsModel.insertElementAt(now, idx);
        cbMaterials.setSelectedIndex(idx);

        // déplacer domaines et propriétaire
        LinkedHashSet<String> set = Globals.materialDomains.remove(old);
        Globals.materialDomains.put(now, set!=null?set:new LinkedHashSet<>());
        for (Map.Entry<String,String> en : new ArrayList<>(Globals.domainOwner.entrySet()))
            if (old.equals(en.getValue())) Globals.domainOwner.put(en.getKey(), now);

        // déplacer aussi les RÉACTIONS
        java.util.List<Globals.ReactionSpec> moved = Globals.matReactions.remove(old);
        if (moved != null) Globals.matReactions.put(now, moved);

        loadMaterialIntoForm.run();
        Globals.repaintAll();
    });

    // Alliage → active champs associés
    chkAlloy.addActionListener(e -> {
        boolean on = chkAlloy.isSelected();
        tfConcVars.setEnabled(on);
        labMolMasses.setEnabled(on);
        tfMolMasses.setEnabled(on);
        markValid.accept(tfConcVars);
        markValid.accept(tfMolMasses);
    });

    // Physiques → met à jour la liste "Variables"
    chkTherm.addActionListener(e -> refreshVarList.run());
    chkReact.addActionListener(e -> refreshVarList.run());
    chkMag  .addActionListener(e -> refreshVarList.run());

    // ==== Layout général ====
    refreshVarList.run(); // initialise la liste

    // Charge l’éditeur/réactions correspondant à la variable par défaut
    varList.addListSelectionListener(e -> {
        if (e.getValueIsAdjusting()) return;
        saveEditorToState.run();
        loadEditorFromState.run();
    });
    loadMaterialIntoForm.run();

    JPanel left = new JPanel(new GridBagLayout());
    GridBagConstraints gl = new GridBagConstraints();
    gl.insets = new Insets(6,8,6,8);
    gl.anchor = GridBagConstraints.WEST;
    gl.fill = GridBagConstraints.HORIZONTAL;

    gl.gridx=0; gl.gridy=0; left.add(new JLabel("Matériaux:"), gl);
    JPanel selRow = new JPanel(new FlowLayout(FlowLayout.LEFT,6,0));
    selRow.add(cbMaterials); selRow.add(btnAddMat); selRow.add(btnDelMat);
    gl.gridx=1; left.add(selRow, gl);

    gl.gridy=1; gl.gridx=0; left.add(new JLabel("Nom:"), gl);
    JPanel nm = new JPanel(new FlowLayout(FlowLayout.LEFT,6,0));
    nm.add(tfName); nm.add(btnRename);
    gl.gridx=1; left.add(nm, gl);

    gl.gridy=2; gl.gridx=0; left.add(chkAlloy, gl);
    gl.gridx=1; left.add(tfConcVars, gl);

    gl.gridy=3; gl.gridx=0; left.add(chkTherm, gl);
    gl.gridy=4; gl.gridx=0; left.add(chkMag, gl);
    gl.gridy=5; gl.gridx=0; left.add(chkReact, gl);

    gl.gridy=6; gl.gridx=0; left.add(new JLabel("M (g/mol)"), gl);
    gl.gridx=1; left.add(tfMolMasses, gl);

    left.setPreferredSize(new Dimension(420, 0));
    left.setMinimumSize(new Dimension(380, 0));

    JPanel mid = new JPanel(new BorderLayout());
    JScrollPane spVars = new JScrollPane(varList);
    spVars.setBorder(BorderFactory.createTitledBorder("Variables"));
    mid.add(spVars, BorderLayout.CENTER);
    mid.setPreferredSize(new Dimension(220, 0));
    mid.setMinimumSize(new Dimension(200, 0));

    JPanel rightTop = new JPanel(new BorderLayout(8,8));
    rightTop.add(rightTopStack, BorderLayout.CENTER);
    rightTop.setBorder(BorderFactory.createTitledBorder("Définition de la variable sélectionnée"));

    JPanel rightBottom = new JPanel(new BorderLayout(6,6));
    rightBottom.add(new DomainCanvas(currentMaterial), BorderLayout.CENTER);
    rightBottom.add(new JScrollPane(taSelectedDomains), BorderLayout.SOUTH);
    rightBottom.setBorder(BorderFactory.createTitledBorder("Affectation des domaines au matériau"));

    JSplitPane rightSplitV = new JSplitPane(JSplitPane.VERTICAL_SPLIT, rightTop, rightBottom);
    rightSplitV.setResizeWeight(0.45);
    rightSplitV.setContinuousLayout(true);
    rightSplitV.setBorder(null);

    JSplitPane midRightSplit = new JSplitPane(JSplitPane.HORIZONTAL_SPLIT, mid, rightSplitV);
    midRightSplit.setResizeWeight(0.22);
    midRightSplit.setContinuousLayout(true);
    midRightSplit.setBorder(null);

    JSplitPane mainSplit = new JSplitPane(JSplitPane.HORIZONTAL_SPLIT, left, midRightSplit);
    mainSplit.setResizeWeight(0.0);
    mainSplit.setContinuousLayout(true);
    mainSplit.setBorder(null);

    JPanel p = new JPanel(new BorderLayout(12,12));
    p.add(mainSplit, BorderLayout.CENTER);
    return p;
}

JPanel uiCL() {

    // --- état / modèles ---
    String[] cols = {"Noms", "Lignes"};
    final javax.swing.table.DefaultTableModel dm = new javax.swing.table.DefaultTableModel(cols, 0) {
        @Override public boolean isCellEditable(int r, int c) { return c == 0; }
    };
    Globals.clModel = dm;

    final javax.swing.JTable table = new javax.swing.JTable(dm);
    table.setRowHeight(24);
    table.getColumnModel().getColumn(0).setPreferredWidth(150);
    table.getColumnModel().getColumn(1).setPreferredWidth(260);

    // --- helpers ---
    final java.util.function.Consumer<String> refreshLinesColumn = (name) -> {
        java.util.LinkedHashSet<String> set =
            Globals.boundaryLines.getOrDefault(name, new java.util.LinkedHashSet<>());
        String txt = String.join(",", set);
        for (int i = 0; i < dm.getRowCount(); i++) {
            if (name.equals(java.util.Objects.toString(dm.getValueAt(i, 0), ""))) {
                dm.setValueAt(txt, i, 1);
                break;
            }
        }
    };

    final java.util.function.Predicate<String> isBoundaryComplete = (name) -> {
        Globals.CLSpec s = Globals.boundarySpecs.get(name);
        java.util.LinkedHashSet<String> set = Globals.boundaryLines.get(name);
        if (s == null || set == null || set.isEmpty()) return false;
        return !s.ux.isEmpty() && !s.uy.isEmpty() && !s.p.isEmpty()
            && !s.T.isEmpty() && !s.B.isEmpty() && !s.mass.isEmpty();
    };

    final java.util.function.Predicate<Globals.CLSpec> usesImposedTypes = (s) -> {
        String all = (s.ux + ";" + s.uy + ";" + s.p + ";" + s.T + ";" + s.B)
                        .toLowerCase(java.util.Locale.ROOT);
        String[] imposed = {
            "fonctionvitesseimposee", "fonctionpression",
            "fonctiontempimposee", "fonctionfluximposee",
            "fonctionimposee"
        };
        for (String k : imposed) if (all.contains(k)) return true;
        return false;
    };

    // --- rendu rouge clair si incomplet ---
    javax.swing.table.DefaultTableCellRenderer rowRenderer =
        new javax.swing.table.DefaultTableCellRenderer() {
            @Override
            public java.awt.Component getTableCellRendererComponent(
                    javax.swing.JTable tbl, Object val, boolean isSel,
                    boolean hasFocus, int row, int col) {
                java.awt.Component c =
                    super.getTableCellRendererComponent(tbl, val, isSel, hasFocus, row, col);
                String name = java.util.Objects.toString(tbl.getValueAt(row, 0), "");
                boolean ok = isBoundaryComplete.test(name);
                if (!isSel) c.setBackground(ok ? java.awt.Color.white
                                               : new java.awt.Color(255, 230, 230));
                return c;
            }
        };
    table.getColumnModel().getColumn(0).setCellRenderer(rowRenderer);
    table.getColumnModel().getColumn(1).setCellRenderer(rowRenderer);

    javax.swing.JButton add = new javax.swing.JButton("+");
    javax.swing.JButton rem = new javax.swing.JButton("-");

    add.addActionListener(e -> {
        String name = "Frontière_" + (dm.getRowCount() + 1);
        dm.addRow(new Object[]{name, ""});
        Globals.boundarySpecs.putIfAbsent(name, new Globals.CLSpec());
        Globals.boundaryLines.putIfAbsent(name, new java.util.LinkedHashSet<>());
        int r = dm.getRowCount() - 1;
        table.changeSelection(r, 0, false, false);
        refreshLinesColumn.accept(name);
    });

    rem.addActionListener(e -> {
        int r = table.getSelectedRow();
        if (r >= 0) {
            String name = java.util.Objects.toString(dm.getValueAt(r, 0), "");
            dm.removeRow(r);
            Globals.boundarySpecs.remove(name);
            Globals.boundaryLines.remove(name);
        }
    });

    // renommage : déplacer proprement specs/lines si un ancien nom disparaît
    dm.addTableModelListener(ev -> {
        if (ev.getType() == javax.swing.event.TableModelEvent.UPDATE && ev.getColumn() == 0) {
            // noms courants
            java.util.Set<String> names = new java.util.LinkedHashSet<>();
            for (int i = 0; i < dm.getRowCount(); i++) {
                Object v = dm.getValueAt(i, 0);
                if (v != null && !v.toString().trim().isEmpty())
                    names.add(v.toString().trim());
            }
            // créer si absent
            for (String n : names) {
                Globals.boundarySpecs.putIfAbsent(n, new Globals.CLSpec());
                Globals.boundaryLines.putIfAbsent(n, new java.util.LinkedHashSet<>());
            }
            // trouver si une clé a "disparu" (ancien nom)
            java.util.List<String> disappeared = new java.util.ArrayList<>();
            for (String k : new java.util.ArrayList<>(Globals.boundarySpecs.keySet())) {
                if (!names.contains(k)) disappeared.add(k);
            }
            if (disappeared.size() == 1) {
                String oldName = disappeared.get(0);
                int r = ev.getFirstRow();
                if (r >= 0 && r < dm.getRowCount()) {
                    String newName = java.util.Objects.toString(dm.getValueAt(r, 0), "").trim();
                    if (!newName.isEmpty()) {
                        Globals.CLSpec spec = Globals.boundarySpecs.remove(oldName);
                        if (spec != null) Globals.boundarySpecs.put(newName, spec);
                        java.util.LinkedHashSet<String> lines = Globals.boundaryLines.remove(oldName);
                        if (lines != null) Globals.boundaryLines.put(newName, lines);
                        refreshLinesColumn.accept(newName);
                    }
                }
            }
        }
    });

    javax.swing.JPanel leftBtns = new javax.swing.JPanel(
        new java.awt.FlowLayout(java.awt.FlowLayout.LEFT, 8, 0));
    leftBtns.add(add);
    leftBtns.add(rem);

    javax.swing.JPanel left = new javax.swing.JPanel(new java.awt.BorderLayout(6, 6));
    left.add(leftBtns, java.awt.BorderLayout.NORTH);
    left.add(new javax.swing.JScrollPane(table), java.awt.BorderLayout.CENTER);
    left.setBorder(javax.swing.BorderFactory.createTitledBorder(
        "Conditions limites / Frontières"));
    left.setPreferredSize(new java.awt.Dimension(360, 0));

    // --------- Formulaire de la frontière sélectionnée ---------
    javax.swing.JTextField tfUx   = new javax.swing.JTextField(18);
    javax.swing.JTextField tfUy   = new javax.swing.JTextField(18);
    javax.swing.JTextField tfP    = new javax.swing.JTextField(18);
    javax.swing.JTextField tfT    = new javax.swing.JTextField(18);
    javax.swing.JTextField tfB    = new javax.swing.JTextField(18);
    javax.swing.JTextField tfMass = new javax.swing.JTextField(18);
    javax.swing.JCheckBox cbExternal = new javax.swing.JCheckBox("Frontière externe");

    javax.swing.JButton hUx = new javax.swing.JButton("?"),
                        hUy = new javax.swing.JButton("?"),
                        hP  = new javax.swing.JButton("?"),
                        hT  = new javax.swing.JButton("?"),
                        hB  = new javax.swing.JButton("?"),
                        hMass = new javax.swing.JButton("?");

    java.awt.Dimension dQ = new java.awt.Dimension(38, 24);
    for (javax.swing.JButton b : new javax.swing.JButton[]{hUx, hUy, hP, hT, hB, hMass})
        b.setPreferredSize(dQ);

    java.awt.event.ActionListener showHelp = new java.awt.event.ActionListener() {
        @Override public void actionPerformed(java.awt.event.ActionEvent e) {
            String msg;
            Object src = e.getSource();
            if (src == hUx || src == hUy) {
                msg = "Saisir: Type1;Valeur1;Type2;Valeur2;...\n"
                    + "Types autorisés:\n"
                    + " - noslip\n"
                    + " - FonctionVitesseImposee (fonction ou constante)\n"
                    + "Exemples:\n noslip;0\n FonctionVitesseImposee;u0";
            } else if (src == hP) {
                msg = "Saisir: Type1;Valeur1;Type2;Valeur2;...\n"
                    + "Types autorisés:\n"
                    + " - FonctionPression (fonction ou constante)\n"
                    + "Exemple:\n FonctionPression;P0";
            } else if (src == hT) {
                msg = "Saisir: Type1;Valeur1;Type2;Valeur2;...\n"
                    + "Types autorisés:\n"
                    + " - FonctionTempImposee (fonction ou constante)\n"
                    + " - FonctionFluxImposee (fonction ou constante)\n"
                    + " - Conv (coefficient de convection)\n"
                    + "Notes Conv:\n"
                    + " - Interface entre 2 domaines: flux = h*(T_domA - T_domB)\n"
                    + " - Frontière externe: flux = h*(Tamb - T_bord) (Tamb dans Paramètres)";
            } else if (src == hB) {
                msg = "Saisir: Type1;Valeur1;Type2;Valeur2;...\n"
                    + "Types autorisés:\n"
                    + " - FonctionImposee (fonction ou constante)\n"
                    + " - Isolantparfait\n"
                    + " - CondParfait";
            } else {
                msg = "Concentrations à la frontière si flux de vitesse imposé:\n"
                    + "Entrer directement: c1,c2,c3,...\n"
                    + "Dans le même ordre que les espèces de l’alliage du matériau\n"
                    + "(pas de mots-clés).";
            }
            javax.swing.JOptionPane.showMessageDialog(
                null, msg, "Aide", javax.swing.JOptionPane.INFORMATION_MESSAGE);
        }
    };
    for (javax.swing.JButton b : new javax.swing.JButton[]{hUx, hUy, hP, hT, hB, hMass})
        b.addActionListener(showHelp);

    javax.swing.JLabel uUx = new javax.swing.JLabel("m/s"),
                       uUy = new javax.swing.JLabel("m/s"),
                       uP  = new javax.swing.JLabel("Pa"),
                       uT  = new javax.swing.JLabel("K"),
                       uB  = new javax.swing.JLabel("T"),
                       uM  = new javax.swing.JLabel("%");

    javax.swing.JPanel form = new javax.swing.JPanel(new java.awt.GridBagLayout());
    java.awt.GridBagConstraints g = new java.awt.GridBagConstraints();
    g.insets = new java.awt.Insets(4, 8, 4, 4);
    g.anchor = java.awt.GridBagConstraints.WEST;
    g.fill   = java.awt.GridBagConstraints.HORIZONTAL;

    int r = 0;
    g.gridx = 0; g.gridy = r; form.add(new javax.swing.JLabel("ux:"), g);
    g.gridx = 1; form.add(tfUx, g);
    g.gridx = 2; form.add(uUx, g);
    g.gridx = 3; form.add(hUx, g); r++;

    g.gridx = 0; g.gridy = r; form.add(new javax.swing.JLabel("uy:"), g);
    g.gridx = 1; form.add(tfUy, g);
    g.gridx = 2; form.add(uUy, g);
    g.gridx = 3; form.add(hUy, g); r++;

    g.gridx = 0; g.gridy = r; form.add(new javax.swing.JLabel("P:"), g);
    g.gridx = 1; form.add(tfP, g);
    g.gridx = 2; form.add(uP, g);
    g.gridx = 3; form.add(hP, g); r++;

    g.gridx = 0; g.gridy = r; form.add(new javax.swing.JLabel("T:"), g);
    g.gridx = 1; form.add(tfT, g);
    g.gridx = 2; form.add(uT, g);
    g.gridx = 3; form.add(hT, g); r++;

    g.gridx = 0; g.gridy = r; form.add(new javax.swing.JLabel("B:"), g);
    g.gridx = 1; form.add(tfB, g);
    g.gridx = 2; form.add(uB, g);
    g.gridx = 3; form.add(hB, g); r++;

    g.gridx = 0; g.gridy = r; form.add(new javax.swing.JLabel("%masse:"), g);
    g.gridx = 1; form.add(tfMass, g);
    g.gridx = 2; form.add(uM, g);
    g.gridx = 3; form.add(hMass, g); r++;

    g.gridx = 1; g.gridy = r; g.gridwidth = 3; form.add(cbExternal, g); r++;

    // --- canvas d’affectation des lignes ---
    Canvas2D pickCanvas = new Canvas2D(false, () -> null, Canvas2D.EdgeView.ALL_EDGES);
    pickCanvas.setPreferredSize(new java.awt.Dimension(520, 360));

    // Tooltip id de la ligne sous la souris (aide au clic)
    pickCanvas.setToolTipText("");
    pickCanvas.addMouseMotionListener(new java.awt.event.MouseMotionAdapter() {
        @Override public void mouseMoved(java.awt.event.MouseEvent e) {
            java.awt.geom.Point2D p = e.getPoint();
            for (int i = Globals.shapes.size() - 1; i >= 0; --i) {
                ShapeItem it = Globals.shapes.get(i);
                if (it.kind != ShapeKind.LINE) continue;
                java.awt.Shape stroked = new java.awt.BasicStroke(6f).createStrokedShape(it.shape);
                if (stroked.contains(p)) { pickCanvas.setToolTipText(it.id); return; }
            }
            pickCanvas.setToolTipText(null);
        }
    });

    javax.swing.JButton btnAffect = new javax.swing.JButton("⟵ Affecter lignes sélectionnées");
    javax.swing.JButton btnClear  = new javax.swing.JButton("Vider la sélection");

    javax.swing.JPanel canvasPanel = new javax.swing.JPanel(new java.awt.BorderLayout(6, 6));
    canvasPanel.setBorder(javax.swing.BorderFactory.createTitledBorder(
        "Sélection des lignes de la frontière"));
    canvasPanel.add(new javax.swing.JScrollPane(pickCanvas), java.awt.BorderLayout.CENTER);

    javax.swing.JPanel canvasBtns = new javax.swing.JPanel(
        new java.awt.FlowLayout(java.awt.FlowLayout.LEFT, 6, 0));
    canvasBtns.add(btnAffect);
    canvasBtns.add(btnClear);
    canvasPanel.add(canvasBtns, java.awt.BorderLayout.SOUTH);

    // --- binding IHM <-> state ---
    java.util.function.Supplier<String> curBoundary = () -> {
        int rr = table.getSelectedRow();
        return (rr >= 0) ? java.util.Objects.toString(dm.getValueAt(rr, 0), "").trim() : null;
    };

    java.lang.Runnable loadFromState = () -> {
        String b = curBoundary.get(); if (b == null) return;
        Globals.CLSpec s = Globals.boundarySpecs.computeIfAbsent(b, k -> new Globals.CLSpec());
        tfUx.setText(s.ux); tfUy.setText(s.uy); tfP.setText(s.p);
        tfT.setText(s.T);   tfB.setText(s.B);   tfMass.setText(s.mass);
        cbExternal.setSelected(s.external);
        refreshLinesColumn.accept(b);
        table.repaint();
    };

    java.lang.Runnable saveToState = () -> {
        String b = curBoundary.get(); if (b == null) return;
        Globals.CLSpec s = Globals.boundarySpecs.computeIfAbsent(b, k -> new Globals.CLSpec());
        s.ux   = tfUx.getText().trim();
        s.uy   = tfUy.getText().trim();
        s.p    = tfP.getText().trim();
        s.T    = tfT.getText().trim();
        s.B    = tfB.getText().trim();
        s.mass = tfMass.getText().trim();
        s.external = cbExternal.isSelected();
        if (!s.external && usesImposedTypes.test(s)) {
            javax.swing.JOptionPane.showMessageDialog(
                form,
                "Conditions imposées (flux/vitesse/pression/température/Champ B)\n"
              + "autorisées uniquement sur les frontières externes.",
                "Avertissement",
                javax.swing.JOptionPane.WARNING_MESSAGE
            );
        }
        refreshLinesColumn.accept(b);
        table.repaint();
    };

    java.awt.event.KeyAdapter saveKA = new java.awt.event.KeyAdapter() {
        @Override public void keyReleased(java.awt.event.KeyEvent e) { saveToState.run(); }
    };
    for (javax.swing.JTextField t : new javax.swing.JTextField[]{tfUx, tfUy, tfP, tfT, tfB, tfMass})
        t.addKeyListener(saveKA);
    cbExternal.addActionListener(e -> saveToState.run());

    table.getSelectionModel().addListSelectionListener(e -> {
        if (!e.getValueIsAdjusting()) loadFromState.run();
    });

    btnAffect.addActionListener(e -> {
        String b = curBoundary.get();
        if (b == null || b.isBlank()) {
            javax.swing.JOptionPane.showMessageDialog(
                pickCanvas, "Sélectionnez une condition limite dans la table.",
                "Aucune CL sélectionnée", javax.swing.JOptionPane.WARNING_MESSAGE);
            return;
        }
        java.util.LinkedHashSet<String> set =
            Globals.boundaryLines.computeIfAbsent(b, k -> new java.util.LinkedHashSet<>());
        boolean any = false;
        for (ShapeItem it : Globals.shapes)
            if (it.selected && it.kind == ShapeKind.LINE) { set.add(it.id); any = true; }
        if (!any) {
            javax.swing.JOptionPane.showMessageDialog(
                pickCanvas, "Sélectionnez d’abord des lignes (clic dans le canvas).",
                "Aucune ligne", javax.swing.JOptionPane.INFORMATION_MESSAGE);
            return;
        }
        refreshLinesColumn.accept(b);
        for (ShapeItem it : Globals.shapes) it.selected = false; // visibilité
        pickCanvas.repaint();
        table.repaint();
    });

    btnClear.addActionListener(e -> {
        String b = curBoundary.get(); if (b == null) return;
        java.util.LinkedHashSet<String> set =
            Globals.boundaryLines.computeIfAbsent(b, k -> new java.util.LinkedHashSet<>());
        set.clear();
        refreshLinesColumn.accept(b);
        table.repaint();
    });

    // --- layout global ---
    javax.swing.JPanel rightMid = new javax.swing.JPanel(new java.awt.BorderLayout(12, 12));
    rightMid.add(form, java.awt.BorderLayout.WEST);
    rightMid.add(canvasPanel, java.awt.BorderLayout.CENTER);

    javax.swing.JPanel root = new javax.swing.JPanel(new java.awt.BorderLayout(12, 12));
    root.add(left, java.awt.BorderLayout.WEST);
    root.add(rightMid, java.awt.BorderLayout.CENTER);

    if (dm.getRowCount() == 0) add.doClick();
    loadFromState.run();

    return root;
}


JPanel uiCI() {

    // === ÉTAT GLOBAL ===
    if (Globals.initSpecs   == null) Globals.initSpecs   = new java.util.LinkedHashMap<>();
    if (Globals.initDomains == null) Globals.initDomains = new java.util.LinkedHashMap<>();
    if (Globals.domainAreas == null) Globals.domainAreas = new java.util.LinkedHashMap<>();

    // === TABLE "Conditions initiales" ===
    final String[] cols = {"Noms", "Domaines"};
    final javax.swing.table.DefaultTableModel dm = new javax.swing.table.DefaultTableModel(cols, 0) {
        @Override public boolean isCellEditable(int r, int c) { return c == 0; } // seul le nom est éditable
    };
    Globals.ciModel = dm;

    final javax.swing.JTable table = new javax.swing.JTable(dm);
    table.setRowHeight(24);
    table.getColumnModel().getColumn(0).setPreferredWidth(160);
    table.getColumnModel().getColumn(1).setPreferredWidth(320);

    // === AIDES ===
    final java.util.function.Supplier<String> curName = () -> {
        int r = table.getSelectedRow();
        return (r >= 0) ? java.util.Objects.toString(dm.getValueAt(r, 0), "").trim() : null;
    };

    final java.util.function.Consumer<String> refreshDomainsCol = (name) -> {
        java.util.LinkedHashSet<String> set = Globals.initDomains.getOrDefault(name, new java.util.LinkedHashSet<>());
        String txt = String.join(",", set);
        for (int i=0;i<dm.getRowCount();i++) {
            if (name.equals(java.util.Objects.toString(dm.getValueAt(i,0),""))) {
                dm.setValueAt(txt, i, 1);
                break;
            }
        }
    };

    final java.util.function.Predicate<String> isRowComplete = (name) -> {
        Globals.CLSpec s = Globals.initSpecs.get(name);
        java.util.LinkedHashSet<String> doms = Globals.initDomains.get(name);
        if (s == null || doms == null || doms.isEmpty()) return false;
        return !s.ux.isEmpty() && !s.uy.isEmpty() && !s.p.isEmpty()
            && !s.T.isEmpty()  && !s.B.isEmpty()  && !s.mass.isEmpty();
    };

    // === RENDERER UNIQUE (fond rose si incomplet) ===
    javax.swing.table.DefaultTableCellRenderer rowRenderer =
        new javax.swing.table.DefaultTableCellRenderer() {
            @Override
            public java.awt.Component getTableCellRendererComponent(
                    javax.swing.JTable tbl, Object val, boolean isSel, boolean hasFocus, int row, int col) {

                java.awt.Component c = super.getTableCellRendererComponent(tbl, val, isSel, hasFocus, row, col);
                String name = java.util.Objects.toString(tbl.getValueAt(row, 0), "");
                boolean ok = isRowComplete.test(name);
                if (!isSel) c.setBackground(ok ? java.awt.Color.white : new java.awt.Color(255,230,230));
                return c;
            }
        };
    table.getColumnModel().getColumn(0).setCellRenderer(rowRenderer);
    table.getColumnModel().getColumn(1).setCellRenderer(rowRenderer);

    // === BOUTONS TABLE ===
    javax.swing.JButton btnAdd = new javax.swing.JButton("+");
    javax.swing.JButton btnDel = new javax.swing.JButton("-");

    btnAdd.addActionListener(e -> {
        String name = "CI_" + (dm.getRowCount()+1);
        dm.addRow(new Object[]{ name, "" });
        Globals.initSpecs.putIfAbsent(name, new Globals.CLSpec());
        Globals.initDomains.putIfAbsent(name, new java.util.LinkedHashSet<>());
        int r = dm.getRowCount() - 1;
        table.changeSelection(r, 0, false, false);
        refreshDomainsCol.accept(name);
    });

    btnDel.addActionListener(e -> {
        int r = table.getSelectedRow();
        if (r >= 0) {
            String name = java.util.Objects.toString(dm.getValueAt(r,0), "");
            dm.removeRow(r);
            Globals.initSpecs.remove(name);
            Globals.initDomains.remove(name);
            table.repaint();
        }
    });

    // Renommage : déplacer proprement l'état (specs + domaines)
    dm.addTableModelListener(ev -> {
        if (ev.getType() == javax.swing.event.TableModelEvent.UPDATE && ev.getColumn() == 0) {
            // noms présents
            java.util.Set<String> names = new java.util.LinkedHashSet<>();
            for (int i=0;i<dm.getRowCount();i++) {
                Object v = dm.getValueAt(i,0);
                if (v != null && !v.toString().trim().isEmpty()) names.add(v.toString().trim());
            }
            // s’assurer que tout existe
            for (String n : names) {
                Globals.initSpecs.putIfAbsent(n, new Globals.CLSpec());
                Globals.initDomains.putIfAbsent(n, new java.util.LinkedHashSet<>());
            }
            // détecter un ancien nom (disparu)
            java.util.List<String> disappeared = new java.util.ArrayList<>();
            for (String k : new java.util.ArrayList<>(Globals.initSpecs.keySet())) {
                if (!names.contains(k)) disappeared.add(k);
            }
            if (disappeared.size() == 1) {
                String oldName = disappeared.get(0);
                int r = ev.getFirstRow();
                if (r >= 0 && r < dm.getRowCount()) {
                    String newName = java.util.Objects.toString(dm.getValueAt(r,0), "").trim();
                    if (!newName.isEmpty()) {
                        Globals.CLSpec spec = Globals.initSpecs.remove(oldName);
                        if (spec != null) Globals.initSpecs.put(newName, spec);
                        java.util.LinkedHashSet<String> doms = Globals.initDomains.remove(oldName);
                        if (doms != null) Globals.initDomains.put(newName, doms);
                        refreshDomainsCol.accept(newName);
                        table.repaint();
                    }
                }
            }
        }
    });

    // === FORMULAIRE DES VALEURS INITIALES ===
    javax.swing.JTextField tfUx   = new javax.swing.JTextField(18);
    javax.swing.JTextField tfUy   = new javax.swing.JTextField(18);
    javax.swing.JTextField tfP    = new javax.swing.JTextField(18);
    javax.swing.JTextField tfT    = new javax.swing.JTextField(18);
    javax.swing.JTextField tfB    = new javax.swing.JTextField(18);
    javax.swing.JTextField tfMass = new javax.swing.JTextField(18);

    javax.swing.JButton hMass = new javax.swing.JButton("?");
    hMass.setPreferredSize(new java.awt.Dimension(38,24));
    hMass.addActionListener(e -> javax.swing.JOptionPane.showMessageDialog(
        null,
        "Concentrations initiales : c1,c2,c3,... en % masse\n"
      + "dans l’ordre des espèces de l’alliage du matériau.",
        "Aide — % masse",
        javax.swing.JOptionPane.INFORMATION_MESSAGE
    ));

    javax.swing.JLabel uUx = new javax.swing.JLabel("m/s");
    javax.swing.JLabel uUy = new javax.swing.JLabel("m/s");
    javax.swing.JLabel uP  = new javax.swing.JLabel("Pa");
    javax.swing.JLabel uT  = new javax.swing.JLabel("K");
    javax.swing.JLabel uB  = new javax.swing.JLabel("T");
    javax.swing.JLabel uM  = new javax.swing.JLabel("%");

    javax.swing.JPanel form = new javax.swing.JPanel(new java.awt.GridBagLayout());
    java.awt.GridBagConstraints gf = new java.awt.GridBagConstraints();
    gf.insets = new java.awt.Insets(4,8,4,4);
    gf.anchor = java.awt.GridBagConstraints.WEST;
    gf.fill   = java.awt.GridBagConstraints.HORIZONTAL;

    int r0 = 0;
    gf.gridx=0; gf.gridy=r0; form.add(new javax.swing.JLabel("ux:"), gf);
    gf.gridx=1; form.add(tfUx, gf); gf.gridx=2; form.add(uUx, gf); r0++;

    gf.gridx=0; gf.gridy=r0; form.add(new javax.swing.JLabel("uy:"), gf);
    gf.gridx=1; form.add(tfUy, gf); gf.gridx=2; form.add(uUy, gf); r0++;

    gf.gridx=0; gf.gridy=r0; form.add(new javax.swing.JLabel("P:"), gf);
    gf.gridx=1; form.add(tfP,  gf); gf.gridx=2; form.add(uP,  gf); r0++;

    gf.gridx=0; gf.gridy=r0; form.add(new javax.swing.JLabel("T:"), gf);
    gf.gridx=1; form.add(tfT,  gf); gf.gridx=2; form.add(uT,  gf); r0++;

    gf.gridx=0; gf.gridy=r0; form.add(new javax.swing.JLabel("B:"), gf);
    gf.gridx=1; form.add(tfB,  gf); gf.gridx=2; form.add(uB,  gf); r0++;

    gf.gridx=0; gf.gridy=r0; form.add(new javax.swing.JLabel("%masse:"), gf);
    gf.gridx=1; form.add(tfMass, gf); gf.gridx=2; form.add(uM, gf); gf.gridx=3; form.add(hMass, gf); r0++;

    // === CANVAS DE SÉLECTION DES DOMAINES ===
    class CIDomainCanvas extends javax.swing.JPanel {
        java.awt.geom.AffineTransform worldToScreen = new java.awt.geom.AffineTransform();
        java.awt.geom.AffineTransform screenToWorld = new java.awt.geom.AffineTransform();

        CIDomainCanvas() {
            setPreferredSize(new java.awt.Dimension(520, 360));
            setBorder(javax.swing.BorderFactory.createTitledBorder("Sélection des domaines"));
            setBackground(new java.awt.Color(246,246,246));

            addComponentListener(new java.awt.event.ComponentAdapter() {
                @Override public void componentResized(java.awt.event.ComponentEvent e) { fitAll(); repaint(); }
            });

            addMouseListener(new java.awt.event.MouseAdapter() {
                @Override public void mouseClicked(java.awt.event.MouseEvent e) {
                    String n = curName.get(); if (n == null) return;

                    java.awt.geom.Point2D ptS = e.getPoint();
                    java.awt.geom.Point2D ptW = new java.awt.geom.Point2D.Double();
                    try { screenToWorld.transform(ptS, ptW); } catch (Exception ex) { return; }

                    java.util.LinkedHashSet<String> set =
                        Globals.initDomains.computeIfAbsent(n, k -> new java.util.LinkedHashSet<>());

                    for (java.util.Map.Entry<String, java.awt.geom.Area> en : Globals.domainAreas.entrySet()) {
                        String dom = en.getKey(); java.awt.geom.Area A = en.getValue();
                        if (A == null || A.isEmpty()) continue;
                        if (A.contains(ptW)) {
                            if (set.contains(dom)) set.remove(dom); else set.add(dom);
                            refreshDomainsCol.accept(n);
                            repaint(); table.repaint();
                            break;
                        }
                    }
                }
            });
        }

        void fitAll() {
            java.awt.geom.Rectangle2D bounds = null;
            for (java.awt.geom.Area A : Globals.domainAreas.values()) {
                if (A == null || A.isEmpty()) continue;
                java.awt.geom.Rectangle2D b = A.getBounds2D();
                bounds = (bounds == null) ? (java.awt.geom.Rectangle2D) b.clone() : bounds.createUnion(b);
            }
            if (bounds == null) bounds = new java.awt.geom.Rectangle2D.Double(0,0,400,300);

            java.awt.Insets in = getInsets();
            int W = Math.max(1, getWidth()-in.left-in.right-16);
            int H = Math.max(1, getHeight()-in.top -in.bottom-40);
            double sx = W / bounds.getWidth(), sy = H / bounds.getHeight(), s = 0.9 * Math.min(sx, sy);
            double tx = in.left + (W - s*bounds.getWidth())  * 0.5 - s*bounds.getX();
            double ty = in.top  + (H - s*bounds.getHeight()) * 0.5 - s*bounds.getY();

            worldToScreen = new java.awt.geom.AffineTransform();
            worldToScreen.translate(tx, ty);
            worldToScreen.scale(s, s);

            try { screenToWorld = worldToScreen.createInverse(); }
            catch (java.awt.geom.NoninvertibleTransformException ex) { screenToWorld = new java.awt.geom.AffineTransform(); }
        }

        @Override protected void paintComponent(java.awt.Graphics g) {
            super.paintComponent(g);

            java.awt.Graphics2D g2 = (java.awt.Graphics2D) g.create();
            g2.setRenderingHint(java.awt.RenderingHints.KEY_ANTIALIASING, java.awt.RenderingHints.VALUE_ANTIALIAS_ON);

            // axes + grille
            g2.setTransform(worldToScreen);
            g2.setStroke(new java.awt.BasicStroke(1f / (float) worldToScreen.getScaleX()));
            for (int x=-1000; x<=2000; x+=100){
                g2.setColor(new java.awt.Color(210,210,210));
                g2.draw(new java.awt.geom.Line2D.Double(x, -1000, x, 2000));
                for (int xx=x+20; xx<x+100; xx+=20){
                    g2.setColor(new java.awt.Color(235,235,235));
                    g2.draw(new java.awt.geom.Line2D.Double(xx, -1000, xx, 2000));
                }
            }
            for (int y=-1000; y<=2000; y+=100){
                g2.setColor(new java.awt.Color(210,210,210));
                g2.draw(new java.awt.geom.Line2D.Double(-1000, y, 2000, y));
                for (int yy=y+20; yy<y+100; yy+=20){
                    g2.setColor(new java.awt.Color(235,235,235));
                    g2.draw(new java.awt.geom.Line2D.Double(-1000, yy, 2000, yy));
                }
            }
            g2.setColor(new java.awt.Color(120,120,120));
            g2.setStroke(new java.awt.BasicStroke(1.4f / (float) worldToScreen.getScaleX()));
            g2.draw(new java.awt.geom.Line2D.Double(-1000, 0, 2000, 0));
            g2.draw(new java.awt.geom.Line2D.Double(0, -1000, 0, 2000));

            String n = curName.get();
            java.util.Set<String> mine = (n == null)
                ? java.util.Collections.emptySet()
                : Globals.initDomains.getOrDefault(n, new java.util.LinkedHashSet<>());

            for (java.util.Map.Entry<String, java.awt.geom.Area> en : Globals.domainAreas.entrySet()){
                String dom = en.getKey(); java.awt.geom.Area A = en.getValue();
                if (A == null || A.isEmpty()) continue;

                boolean sel = mine.contains(dom);
                g2.setComposite(java.awt.AlphaComposite.SrcOver.derive(sel ? 0.22f : 0.06f));
                g2.setColor(sel ? new java.awt.Color(40,170,80) : new java.awt.Color(50,50,50));
                g2.fill(A);
                g2.setComposite(java.awt.AlphaComposite.SrcOver);
                g2.setColor(sel ? new java.awt.Color(40,170,80) : new java.awt.Color(100,100,100));
                g2.setStroke(new java.awt.BasicStroke(1.8f / (float) worldToScreen.getScaleX()));
                g2.draw(A);

                // label
                java.awt.Rectangle b = A.getBounds();
                java.awt.geom.Point2D pt = worldToScreen.transform(
                        new java.awt.geom.Point2D.Double(b.getMaxX(), b.getMinY()), null);
                java.awt.geom.AffineTransform save = g2.getTransform();
                g2.setTransform(new java.awt.geom.AffineTransform());
                g2.setColor(new java.awt.Color(60,60,60));
                g2.drawString(dom, (float)pt.getX()+4, (float)pt.getY()+12);
                g2.setTransform(save);
            }

            g2.dispose();
        }
    }

    final CIDomainCanvas pickCanvas = new CIDomainCanvas();

    // Callback global: si les domaines changent ailleurs, on se met à jour
    Globals.canvasCI = pickCanvas;
    Globals.domainsChanged = () -> {
        pickCanvas.fitAll();
        pickCanvas.repaint();
        String n = curName.get();
        if (n != null) refreshDomainsCol.accept(n);
    };

    // Recalage quand le panneau devient visible
    pickCanvas.addHierarchyListener(e -> {
        if ((e.getChangeFlags() & java.awt.event.HierarchyEvent.SHOWING_CHANGED) != 0 && pickCanvas.isShowing()) {
            pickCanvas.fitAll();
            pickCanvas.repaint();
        }
    });

    javax.swing.JButton btnClear = new javax.swing.JButton("Vider la sélection");
    btnClear.addActionListener(e -> {
        String n = curName.get(); if (n == null) return;
        java.util.LinkedHashSet<String> set = Globals.initDomains.computeIfAbsent(n, k -> new java.util.LinkedHashSet<>());
        set.clear();
        refreshDomainsCol.accept(n);
        table.repaint();
        pickCanvas.repaint();
    });

    javax.swing.JPanel canvasPanel = new javax.swing.JPanel(new java.awt.BorderLayout(6,6));
    canvasPanel.setBorder(javax.swing.BorderFactory.createTitledBorder("Sélection des domaines (clic dans la zone)"));
    canvasPanel.add(new javax.swing.JScrollPane(pickCanvas), java.awt.BorderLayout.CENTER);
    javax.swing.JPanel canvasBtns = new javax.swing.JPanel(new java.awt.FlowLayout(java.awt.FlowLayout.LEFT,6,0));
    canvasBtns.add(btnClear);
    canvasPanel.add(canvasBtns, java.awt.BorderLayout.SOUTH);

    // === BINDINGS FORM <-> STATE ===
    final java.lang.Runnable loadFromState = () -> {
        String n = curName.get(); if (n == null) return;
        Globals.CLSpec s = Globals.initSpecs.computeIfAbsent(n, k -> new Globals.CLSpec());
        tfUx.setText(s.ux);
        tfUy.setText(s.uy);
        tfP .setText(s.p);
        tfT .setText(s.T);
        tfB .setText(s.B);
        tfMass.setText(s.mass);
        refreshDomainsCol.accept(n);
        table.repaint();
        pickCanvas.repaint();
    };

    final java.lang.Runnable saveToState = () -> {
        String n = curName.get(); if (n == null) return;
        Globals.CLSpec s = Globals.initSpecs.computeIfAbsent(n, k -> new Globals.CLSpec());
        s.ux   = tfUx.getText().trim();
        s.uy   = tfUy.getText().trim();
        s.p    = tfP.getText().trim();
        s.T    = tfT.getText().trim();
        s.B    = tfB.getText().trim();
        s.mass = tfMass.getText().trim();
        refreshDomainsCol.accept(n);
        table.repaint();
    };

    java.awt.event.KeyAdapter saveKA = new java.awt.event.KeyAdapter() {
        @Override public void keyReleased(java.awt.event.KeyEvent e) { saveToState.run(); }
    };
    for (javax.swing.JTextField t : new javax.swing.JTextField[]{tfUx, tfUy, tfP, tfT, tfB, tfMass}) t.addKeyListener(saveKA);

    table.getSelectionModel().addListSelectionListener(e -> {
        if (!e.getValueIsAdjusting()) loadFromState.run();
    });

    // === LAYOUT avec volets redimensionnables ===
    javax.swing.JPanel leftBtns = new javax.swing.JPanel(new java.awt.FlowLayout(java.awt.FlowLayout.LEFT,8,0));
    leftBtns.add(btnAdd); leftBtns.add(btnDel);

    javax.swing.JPanel left = new javax.swing.JPanel(new java.awt.BorderLayout(6,6));
    left.add(leftBtns, java.awt.BorderLayout.NORTH);
    left.add(new javax.swing.JScrollPane(table), java.awt.BorderLayout.CENTER);
    left.setBorder(javax.swing.BorderFactory.createTitledBorder("Conditions initiales"));
    left.setPreferredSize(new java.awt.Dimension(360, 0));

    javax.swing.JScrollPane formScroll = new javax.swing.JScrollPane(form);
    formScroll.setBorder(javax.swing.BorderFactory.createTitledBorder("Valeurs initiales"));

    javax.swing.JSplitPane rightInnerSplit = new javax.swing.JSplitPane(
            javax.swing.JSplitPane.HORIZONTAL_SPLIT, formScroll, canvasPanel);
    rightInnerSplit.setResizeWeight(0.35);
    rightInnerSplit.setContinuousLayout(true);
    rightInnerSplit.setBorder(null);

    javax.swing.JSplitPane mainSplit = new javax.swing.JSplitPane(
            javax.swing.JSplitPane.HORIZONTAL_SPLIT, left, rightInnerSplit);
    mainSplit.setResizeWeight(0.0);
    mainSplit.setContinuousLayout(true);
    mainSplit.setBorder(null);

    javax.swing.JPanel root = new javax.swing.JPanel(new java.awt.BorderLayout(12,12));
    root.add(mainSplit, java.awt.BorderLayout.CENTER);

    // === INIT ===
    if (dm.getRowCount() == 0) btnAdd.doClick();
    javax.swing.SwingUtilities.invokeLater(() -> {
        loadFromState.run();
        pickCanvas.fitAll();
        pickCanvas.repaint();
    });

    return root;
}










JPanel uiSolveur() {
    return formGrid(
        row("Schéma temporel", new JComboBox<>(new String[]{"Euler implicite", "BDF2", "RK4"})),
        row("Solveur linéaire", new JComboBox<>(new String[]{"CG", "BiCGStab", "GMRES"})),
        row("Préconditionneur", new JComboBox<>(new String[]{"ILU", "AMG", "Jacobi"})),
        row("Tolérance", new JTextField("1e-8")),
        row("Itérations max", new JTextField("500"))
    );
}
JPanel uiMaillage() {

    // ------ Modèle de liste des domaines (synchro live avec l'onglet Domaines) ------
    final DefaultListModel<String> domainsModel = new DefaultListModel<>();
    final JList<String> domainsList = new JList<>(domainsModel);
    domainsList.setFixedCellHeight(26);

    final boolean[] inLoad = {false};      // garde: ne pas sauver pendant un load
    final String[]  lastDom = {null};      // dernier domaine affiché (pour sauver avant switch)

    Runnable refillDomains = () -> {
        domainsModel.clear();
        if (Globals.domainsModel != null) {
            for (int r = 0; r < Globals.domainsModel.getRowCount(); r++) {
                Object v = Globals.domainsModel.getValueAt(r, 0);
                if (v != null && !v.toString().trim().isEmpty()) domainsModel.addElement(v.toString());
            }
        } else {
            for (String d : Globals.domainAreas.keySet()) domainsModel.addElement(d);
        }
        if (!domainsModel.isEmpty() && domainsList.getSelectedIndex() == -1) {
            domainsList.setSelectedIndex(0);
        }
    };
    // Suivre les modifs du tableau Domaines
    if (Globals.domainsModel != null) {
        Globals.domainsModel.addTableModelListener(ev -> SwingUtilities.invokeLater(refillDomains));
    }
    // Recharger lors de la première visibilité du panneau
    domainsList.addHierarchyListener(e -> {
        if ((e.getChangeFlags() & HierarchyEvent.SHOWING_CHANGED) != 0 && domainsList.isShowing()) {
            refillDomains.run();
        }
    });
    refillDomains.run();

    // Cell renderer avec fond rouge si incomplet
    class MeshListRenderer extends DefaultListCellRenderer {
        @Override
        public Component getListCellRendererComponent(JList<?> list, Object value, int index,
                                                      boolean isSelected, boolean cellHasFocus) {
            JLabel c = (JLabel) super.getListCellRendererComponent(list, value, index, isSelected, cellHasFocus);
            String dom = String.valueOf(value);
            boolean ok = Globals.isMeshSpecComplete(dom);
            if (!isSelected) c.setBackground(ok ? Color.white : new Color(255, 230, 230));
            c.setBorder(new javax.swing.border.EmptyBorder(2,8,2,8));
            return c;
        }
    }
    domainsList.setCellRenderer(new MeshListRenderer());

    JScrollPane domainsScroll = new JScrollPane(domainsList);
    domainsScroll.setBorder(BorderFactory.createTitledBorder("Domaines"));

    // ------ Canvas de prévisualisation des domaines ------
    class MeshPreviewCanvas extends JPanel {
        String selectedDom;
        MeshPreviewCanvas(){ setPreferredSize(new Dimension(560, 360)); setBackground(new Color(246,246,246)); }
        void setSelected(String d){ selectedDom = d; repaint(); }

        @Override protected void paintComponent(Graphics g){
            super.paintComponent(g);
            Graphics2D g2 = (Graphics2D) g.create();
            g2.setRenderingHint(RenderingHints.KEY_ANTIALIASING, RenderingHints.VALUE_ANTIALIAS_ON);

            // fit simple
            java.awt.geom.Rectangle2D all = null;
            for (java.awt.geom.Area A : Globals.domainAreas.values()){
                if (A==null || A.isEmpty()) continue;
                all = (all==null) ? (java.awt.geom.Rectangle2D)A.getBounds2D().clone() : all.createUnion(A.getBounds2D());
            }
            if (all == null) all = new java.awt.geom.Rectangle2D.Double(0,0,400,300);
            double pad = 20;
            double sx = (getWidth()-2*pad)/all.getWidth();
            double sy = (getHeight()-2*pad)/all.getHeight();
            double s  = 0.9*Math.min(sx, sy);
            g2.translate(getWidth()/2.0, getHeight()/2.0);
            g2.scale(s, s);
            g2.translate(-all.getCenterX(), -all.getCenterY());

            // dessin
            for (java.util.Map.Entry<String, java.awt.geom.Area> en : Globals.domainAreas.entrySet()){
                String name = en.getKey(); java.awt.geom.Area A = en.getValue();
                if (A==null || A.isEmpty()) continue;

                boolean sel = name.equals(selectedDom);
                g2.setComposite(AlphaComposite.SrcOver.derive(sel ? 0.20f : 0.08f));
                g2.setColor(sel ? new Color(40,120,230) : new Color(80,80,80));
                g2.fill(A);
                g2.setComposite(AlphaComposite.SrcOver);
                g2.setStroke(new BasicStroke(sel ? 2.0f : 1.2f));
                g2.setColor(sel ? new Color(40,120,230) : new Color(120,120,120));
                g2.draw(A);

                // label
                Rectangle b = A.getBounds();
                g2.setColor(new Color(60,60,60));
                g2.drawString(name, (float)b.getCenterX()+4, (float)b.getMinY()-4);
            }
            g2.dispose();
        }
    }
    MeshPreviewCanvas preview = new MeshPreviewCanvas();

    // ------ Éditeur de paramétrage ------
    JComboBox<String> cbType = new JComboBox<>();
    cbType.setPrototypeDisplayValue("Maillage Quadrilatere croissant");
    JPanel typeRow = formGrid(row("Type:", cbType));

    // Cartes des champs
    JTextField tfDx  = new JTextField(8);
    JTextField tfDy  = new JTextField(8);
    JPanel cardRegular = formGrid(row("dx", tfDx), row("dy", tfDy));

    JTextField tfDxMin = new JTextField(8);
    JTextField tfDxMax = new JTextField(8);
    JTextField tfDyMin = new JTextField(8);
    JTextField tfDyMax = new JTextField(8);
    JRadioButton rbGrow   = new JRadioButton("croissant depuis la frontière", true);
    JRadioButton rbDegrow = new JRadioButton("décroissant depuis la frontière");
    ButtonGroup growthGrp = new ButtonGroup(); growthGrp.add(rbGrow); growthGrp.add(rbDegrow);

    JPanel growthRow = new JPanel(new FlowLayout(FlowLayout.LEFT,8,0));
    growthRow.add(rbGrow); growthRow.add(rbDegrow);

    JPanel cardGraded = new JPanel(new GridBagLayout());
    GridBagConstraints gg = new GridBagConstraints();
    gg.insets = new Insets(6,6,6,6); gg.fill = GridBagConstraints.HORIZONTAL; gg.gridx=0; gg.gridy=0;
    cardGraded.add(new JLabel("dxmin"), gg); gg.gridx=1; cardGraded.add(tfDxMin, gg);
    gg.gridx=0; gg.gridy++;            cardGraded.add(new JLabel("dxmax"), gg); gg.gridx=1; cardGraded.add(tfDxMax, gg);
    gg.gridx=0; gg.gridy++;            cardGraded.add(new JLabel("dymin"), gg); gg.gridx=1; cardGraded.add(tfDyMin, gg);
    gg.gridx=0; gg.gridy++;            cardGraded.add(new JLabel("dymax"), gg); gg.gridx=1; cardGraded.add(tfDyMax, gg);
    gg.gridx=0; gg.gridy++; gg.gridwidth=2; cardGraded.add(growthRow, gg);

    JPanel cards = new JPanel(new CardLayout());
    cards.add(cardRegular, "REG");
    cards.add(cardGraded,  "GRAD");

    // mapping libellés <-> kind
    java.util.LinkedHashMap<String, Globals.MeshKind> labelToKind = new java.util.LinkedHashMap<>();
    labelToKind.put("Maillage Triangle régulier",      Globals.MeshKind.TRI_REG);
    labelToKind.put("Maillage Quadrilatere régulier",  Globals.MeshKind.RECT_REG);
    labelToKind.put("Maillage Triangle croissant",     Globals.MeshKind.TRI_GRAD);
    labelToKind.put("Maillage Quadrilatere croissant", Globals.MeshKind.RECT_GRAD);

    java.util.LinkedHashMap<Globals.MeshKind, String> kindToLabel = new java.util.LinkedHashMap<>();
    for (java.util.Map.Entry<String, Globals.MeshKind> e : labelToKind.entrySet())
        kindToLabel.put(e.getValue(), e.getKey());

    // parse util
    java.util.function.Function<JTextField, Double> readD = tf -> {
        String s = tf.getText().trim().replace(',', '.');
        if (s.isEmpty()) return null;
        try { return Double.parseDouble(s); } catch (NumberFormatException ex) { return null; }
    };

    // recharge les choix de type selon la géométrie du domaine
    Runnable refreshTypeChoices = () -> {
        String dom = domainsList.getSelectedValue();
        cbType.removeAllItems();
        if (dom == null) return;

        java.awt.geom.Area A = Globals.domainAreas.get(dom);
        boolean quadOK = Globals.isRightAnglePolygon(A, 15.0); // 90° ± 15°

        // Triangles toujours disponibles
        cbType.addItem("Maillage Triangle régulier");
        cbType.addItem("Maillage Triangle croissant");

        // Quadrilatere seulement si angles ~ droits
        if (quadOK) {
            cbType.addItem("Maillage Quadrilatere régulier");
            cbType.addItem("Maillage Quadrilatere croissant");
        }

        // Revenir à la sélection du state si possible
        Globals.MeshSpec s = Globals.meshSpecs.get(dom);
        if (s != null) {
            String lab = kindToLabel.get(s.kind);
            if (lab != null) cbType.setSelectedItem(lab);
        }
        if (cbType.getItemCount() > 0 && cbType.getSelectedIndex() == -1)
            cbType.setSelectedIndex(0);
    };

    // affiche la carte en fonction du type
    Runnable refreshCards = () -> {
        Object sel = cbType.getSelectedItem();
        Globals.MeshKind k = (sel==null) ? Globals.MeshKind.TRI_REG : labelToKind.get(sel.toString());
        CardLayout cl = (CardLayout) cards.getLayout();
        boolean isGrad = (k==Globals.MeshKind.TRI_GRAD || k==Globals.MeshKind.RECT_GRAD);
        cl.show(cards, isGrad ? "GRAD" : "REG");
    };

    // ---- Sauvegarde pour un domaine donné (sans effet si inLoad) ----
    java.util.function.Consumer<String> saveToDomain = (dom) -> {
        if (dom == null || inLoad[0]) return;
        Globals.MeshSpec s = Globals.meshSpecs.computeIfAbsent(dom, d -> new Globals.MeshSpec());

        Object sel = cbType.getSelectedItem();
        if (sel != null) s.kind = labelToKind.get(sel.toString());

        if (s.kind==Globals.MeshKind.TRI_REG || s.kind==Globals.MeshKind.RECT_REG) {
            s.dx = readD.apply(tfDx);
            s.dy = readD.apply(tfDy);
            s.dxmin = s.dxmax = s.dymin = s.dymax = null;
        } else {
            s.dxmin = readD.apply(tfDxMin);
            s.dxmax = readD.apply(tfDxMax);
            s.dymin = readD.apply(tfDyMin);
            s.dymax = readD.apply(tfDyMax);
            s.dx = s.dy = null;
            s.growFromBoundary = rbGrow.isSelected();
        }
    };

    // charge l'état d'un domaine
    Runnable loadDomainIntoForm = () -> {
        String dom = domainsList.getSelectedValue();
        preview.setSelected(dom);
        if (dom == null) return;

        inLoad[0] = true; // guard ON

        refreshTypeChoices.run();
        Globals.MeshSpec s = Globals.meshSpecs.computeIfAbsent(dom, d -> new Globals.MeshSpec());

        cbType.setSelectedItem(kindToLabel.getOrDefault(s.kind, "Maillage Triangle régulier"));
        refreshCards.run();

        tfDx.setText   (s.dx    == null ? "" : s.dx.toString());
        tfDy.setText   (s.dy    == null ? "" : s.dy.toString());
        tfDxMin.setText(s.dxmin == null ? "" : s.dxmin.toString());
        tfDxMax.setText(s.dxmax == null ? "" : s.dxmax.toString());
        tfDyMin.setText(s.dymin == null ? "" : s.dymin.toString());
        tfDyMax.setText(s.dymax == null ? "" : s.dymax.toString());
        rbGrow.setSelected(s.growFromBoundary);
        rbDegrow.setSelected(!s.growFromBoundary);

        inLoad[0] = false; // guard OFF
        lastDom[0] = dom;

        domainsList.repaint(); // met à jour le rouge/ok
    };

    // sauvegarde (pour le domaine actuellement sélectionné)
    Runnable saveFormToState = () -> {
        if (inLoad[0]) return;
        saveToDomain.accept(domainsList.getSelectedValue());
        domainsList.repaint(); // recoloration
    };

    // ------ Listeners ------
    // Sauver l'ancien domaine AVANT switch, puis charger le nouveau
    domainsList.addListSelectionListener(e -> {
        if (!e.getValueIsAdjusting()) {
            saveToDomain.accept(lastDom[0]); // sauvegarder domaine sortant
            loadDomainIntoForm.run();        // charger domaine entrant
        }
    });

    cbType.addActionListener(e -> {
        if (inLoad[0]) return;
        refreshCards.run();
        saveFormToState.run();
    });

    java.awt.event.KeyAdapter saverKA = new java.awt.event.KeyAdapter(){ 
        @Override public void keyReleased(java.awt.event.KeyEvent e){ saveFormToState.run(); } 
    };
    for (JTextField tf : new JTextField[]{tfDx, tfDy, tfDxMin, tfDxMax, tfDyMin, tfDyMax}) tf.addKeyListener(saverKA);

    rbGrow.addActionListener(e -> saveFormToState.run());
    rbDegrow.addActionListener(e -> saveFormToState.run());

    // ------ Layout ------
    JPanel left = new JPanel(new BorderLayout(6,6));
    left.add(domainsScroll, BorderLayout.CENTER);
    left.setPreferredSize(new Dimension(220, 0));

    JPanel editor = new JPanel(new BorderLayout(8,8));
    editor.add(typeRow, BorderLayout.NORTH);
    editor.add(cards,   BorderLayout.CENTER);

    JPanel right = new JPanel(new BorderLayout(8,8));
    right.add(editor, BorderLayout.WEST);
    right.add(preview, BorderLayout.CENTER);

    JPanel root = new JPanel(new BorderLayout(12,12));
    root.add(left, BorderLayout.WEST);
    root.add(right, BorderLayout.CENTER);

    // init: sélectionner le 1er domaine s’il existe
    if (!domainsModel.isEmpty()) domainsList.setSelectedIndex(0);

    return root;
}


String[] mustExistClasses = {
    "Step", "BoldSectionRenderer", "Globals", "Canvas2D",
    "ProjectState", "ProjectIO", "ShapeItem"
};
String[] mustExistMethods = {
    "uiParametres", "uiGeometrie", "uiDomaines", "uiMateriaux",
    "uiCL", "uiCI", "uiSolveur", "uiMaillage", "wrapTitled"
};

java.util.List<String> missing = new java.util.ArrayList<>();

// classes
for (String cn : mustExistClasses) {
    try { Class.forName(cn); } 
    catch (Throwable t) { missing.add("Classe manquante: " + cn); }
}

// méthodes globales (fonctions de niveau top dans JShell => regardées via méthodes publiques de la classe de snippet)
try {
    // Heuristique: JShell met les fonctions top-level dans une classe anonyme du loader;
    // on tente juste d'appeler via réflexion le nom exact dans 'this' (si iJava expose).
    // Si ça échoue, on fera un test d'appel simple.
    for (String m : mustExistMethods) {
        try {
            // On essaie de trouver une méthode statique sans paramètres
            java.lang.reflect.Method mm = Class.forName("REPL").getDeclaredMethod(m);
        } catch (Throwable ignore) {
            // fallback: on teste l'appel via un petit proxy
            // -> on ne peut pas vraiment appeler ici; on se contente du message
            // On marquera simplement "à vérifier manuellement".
            missing.add("Méthode à vérifier (définition avant showMHDGui): " + m + "()");
        }
    }
} catch (Throwable ignore) { /* En JShell, ce bloc peut ne pas marcher, on passe en 'à vérifier' */ }

if (missing.isEmpty()) {
    System.out.println("Diagnostic: OK (aucune dépendance manquante détectée).");
} else {
    System.out.println("Diagnostic: éléments manquants/troubles:");
    for (String m : missing) System.out.println(" - " + m);
}







Diagnostic: éléments manquants/troubles:
 - Classe manquante: Step
 - Classe manquante: BoldSectionRenderer
 - Classe manquante: Globals
 - Classe manquante: Canvas2D
 - Classe manquante: ProjectState
 - Classe manquante: ProjectIO
 - Classe manquante: ShapeItem
 - Méthode à vérifier (définition avant showMHDGui): uiParametres()
 - Méthode à vérifier (définition avant showMHDGui): uiGeometrie()
 - Méthode à vérifier (définition avant showMHDGui): uiDomaines()
 - Méthode à vérifier (définition avant showMHDGui): uiMateriaux()
 - Méthode à vérifier (définition avant showMHDGui): uiCL()
 - Méthode à vérifier (définition avant showMHDGui): uiCI()
 - Méthode à vérifier (définition avant showMHDGui): uiSolveur()
 - Méthode à vérifier (définition avant showMHDGui): uiMaillage()
 - Méthode à vérifier (définition avant showMHDGui): wrapTitled()


In [5]:
// ==========================================
//  Preflight : validation rapide côté Java
//  (version sans 'var' ni API "trop moderne" pour JShell)
// ==========================================
public final class Preflight {

    static final class Result {
        final java.util.LinkedHashMap<String, java.util.List<String>> bySection =
                new java.util.LinkedHashMap<>();

        boolean ok() {
            for (java.util.List<String> L : bySection.values()) if (!L.isEmpty()) return false;
            return true;
        }
        void add(String section, String msg){
            bySection.computeIfAbsent(section, k -> new java.util.ArrayList<>()).add(msg);
        }
        java.util.List<String> all(){
            java.util.List<String> out = new java.util.ArrayList<>();
            for (java.util.Map.Entry<String, java.util.List<String>> e : bySection.entrySet()){
                if (!e.getValue().isEmpty()){
                    out.add("[" + e.getKey() + "]");
                    out.addAll(e.getValue());
                }
            }
            return out;
        }
    }

    static Preflight.Result validate(MHDPersistence.ProjectState s){
        Preflight.Result r = new Preflight.Result();

        java.util.LinkedHashMap<String, MHDPersistence.ShapeDTO> shapes = new java.util.LinkedHashMap<>();
        for (MHDPersistence.ShapeDTO sh : s.shapes) shapes.put(sh.id, sh);

        java.util.Map<String, java.util.List<String>> edgeMap =
                (s.shapeEdgeLines != null) ? s.shapeEdgeLines : new java.util.LinkedHashMap<>();

        java.util.LinkedHashSet<String> domainNames = new java.util.LinkedHashSet<>(s.domainEdits.keySet());
        java.util.LinkedHashSet<String> materials   = new java.util.LinkedHashSet<>(s.materials);

        java.util.Map<String, MHDPersistence.CLSpecDTO> boundarySpecs =
                (s.boundarySpecs != null) ? s.boundarySpecs : new java.util.LinkedHashMap<>();
        java.util.Map<String, java.util.LinkedHashSet<String>> boundaryLines =
                (s.boundaryLines != null) ? s.boundaryLines : new java.util.LinkedHashMap<>();
        java.util.Map<String, MHDPersistence.CLSpecDTO> initSpecs =
                (s.initSpecs != null) ? s.initSpecs : new java.util.LinkedHashMap<>();
        java.util.Map<String, java.util.LinkedHashSet<String>> initDomains =
                (s.initDomains != null) ? s.initDomains : new java.util.LinkedHashMap<>();

        java.util.Map<String, MHDPersistence.MeshSpecDTO> meshSpecs =
                (s.meshSpecs != null) ? s.meshSpecs : new java.util.LinkedHashMap<>();

        // ---------- DOMAINES ----------
        {
            final String SEC = "Domaines";
            if (domainNames.isEmpty()) r.add(SEC, "Aucun domaine défini.");

            for (String dom : domainNames) {
                java.util.List<MHDPersistence.DomainEditDTO> edits = s.domainEdits.get(dom);
                if (edits == null || edits.isEmpty()) {
                    r.add(SEC, dom + " : aucun élément géométrique (edits vides).");
                    continue;
                }
                boolean hasShape = false;
                for (MHDPersistence.DomainEditDTO de : edits) {
                    if (de.shapeId != null && shapes.containsKey(de.shapeId)) { hasShape = true; break; }
                }
                if (!hasShape) r.add(SEC, dom + " : aucune forme connue référencée.");
            }

            for (String dom : domainNames) {
                String mat = s.domainOwner.get(dom);
                if (mat == null || mat.trim().isEmpty())
                    r.add(SEC, dom + " : matériau non défini.");
                else if (!materials.contains(mat))
                    r.add(SEC, dom + " : matériau '" + mat + "' inexistant.");
            }

            // Couche 1 : exiger qu’au moins une CL existe
            for (String dom : domainNames) {
                if (boundaryLines.isEmpty())
                    r.add(SEC, dom + " : aucune condition limite définie dans le projet.");
            }
        }

        // ---------- MATERIAUX ----------
        {
            final String SEC = "Matériaux";
            for (String m : materials) {
                java.util.Map<String, MHDPersistence.VarSpecDTO> vars = s.matVars.get(m);
                if (vars == null || vars.isEmpty()) {
                    r.add(SEC, "Matériau '" + m + "': aucune variable définie.");
                } else {
                    for (java.util.Map.Entry<String, MHDPersistence.VarSpecDTO> e : vars.entrySet()) {
                        String vname = e.getKey();
                        MHDPersistence.VarSpecDTO vs = e.getValue();
                        if (vs.useFunc) {
                            if (isEmpty(vs.funcExpr))
                                r.add(SEC, "Matériau '" + m + "', variable '" + vname + "': expression fonction vide.");
                        } else {
                            if (isEmpty(vs.csvPath))
                                r.add(SEC, "Matériau '" + m + "', variable '" + vname + "': table CSV non renseignée.");
                        }
                    }
                }
                java.util.List<MHDPersistence.ReactionSpecDTO> rx = s.reactions.get(m);
                if (rx != null) {
                    for (int i=0;i<rx.size();i++){
                        MHDPersistence.ReactionSpecDTO rr = rx.get(i);
                        if (isEmpty(rr.kExpr) && isEmpty(rr.energyMeV) && isEmpty(rr.products) && isEmpty(rr.reactants))
                            r.add(SEC, "Matériau '" + m + "': réaction #" + (i+1) + " totalement vide.");
                    }
                }
            }
        }

        // ---------- CONDITIONS LIMITES ----------
        {
            final String SEC = "Conditions limites";
            if (boundarySpecs.isEmpty()) r.add(SEC, "Aucun set de condition limite défini.");

            for (java.util.Map.Entry<String, MHDPersistence.CLSpecDTO> e : boundarySpecs.entrySet()){
                String name = e.getKey();
                MHDPersistence.CLSpecDTO S = e.getValue();
                if (isEmpty(S.ux) || isEmpty(S.uy) || isEmpty(S.p) || isEmpty(S.T) || isEmpty(S.B) || isEmpty(S.mass))
                    r.add(SEC, "CL '"+name+"': certains champs sont vides (ux, uy, p, T, B, mass).");
            }

            for (java.util.Map.Entry<String, java.util.LinkedHashSet<String>> e : boundaryLines.entrySet()){
                String name = e.getKey();
                java.util.LinkedHashSet<String> lines = e.getValue();
                if (lines==null || lines.isEmpty())
                    r.add(SEC, "CL '"+name+"': aucune arête assignée.");
                else {
                    for (String lid : lines){
                        MHDPersistence.ShapeDTO sh = shapes.get(lid);
                        if (sh == null || !"LINE".equals(sh.kind))
                            r.add(SEC, "CL '"+name+"': arête '"+lid+"' introuvable ou non-LINE.");
                    }
                }
            }

            java.util.HashSet<String> allCovered = new java.util.HashSet<>();
            for (java.util.Map.Entry<String, java.util.LinkedHashSet<String>> e : boundaryLines.entrySet()){
                MHDPersistence.CLSpecDTO spec = boundarySpecs.get(e.getKey());
                boolean ext = (spec==null) ? true : spec.external;
                if (!ext) continue;
                if (e.getValue() != null) allCovered.addAll(e.getValue());
            }
            java.util.LinkedHashSet<String> domUsedShapes = new java.util.LinkedHashSet<>();
            for (String dom : domainNames){
                java.util.List<MHDPersistence.DomainEditDTO> edits = s.domainEdits.get(dom);
                if (edits==null) continue;
                for (MHDPersistence.DomainEditDTO de : edits){
                    if (shapes.containsKey(de.shapeId)) domUsedShapes.add(de.shapeId);
                }
            }
            for (String sid : domUsedShapes){
                java.util.List<String> edges = edgeMap.containsKey(sid) ? edgeMap.get(sid) : java.util.Collections.<String>emptyList();
                for (String eid : edges){
                    MHDPersistence.ShapeDTO sh = shapes.get(eid);
                    if (sh!=null && "LINE".equals(sh.kind)) {
                        if (!allCovered.contains(eid)) {
                            r.add(SEC, "Arête extérieure non couverte par une CL externe : " + eid + " (forme " + sid + ")");
                        }
                    }
                }
            }
        }

        // ---------- CONDITIONS INITIALES ----------
        {
            final String SEC = "Conditions initiales";
            if (initSpecs.isEmpty()) r.add(SEC, "Aucune condition initiale définie.");

            for (java.util.Map.Entry<String, MHDPersistence.CLSpecDTO> e : initSpecs.entrySet()){
                String name = e.getKey();
                MHDPersistence.CLSpecDTO S = e.getValue();
                if (isEmpty(S.ux) || isEmpty(S.uy) || isEmpty(S.p) || isEmpty(S.T) || isEmpty(S.B) || isEmpty(S.mass))
                    r.add(SEC, "CI '"+name+"': certains champs sont vides (ux, uy, p, T, B, mass).");
                java.util.LinkedHashSet<String> doms = initDomains.get(name);
                if (doms==null || doms.isEmpty())
                    r.add(SEC, "CI '"+name+"': aucun domaine associé.");
                else {
                    for (String d : doms)
                        if (!domainNames.contains(d))
                            r.add(SEC, "CI '"+name+"': domaine inconnu '"+d+"'.");
                }
            }
        }

        // ---------- MAILLAGE ----------
        {
            final String SEC = "Maillage";
            if (meshSpecs.isEmpty()) r.add(SEC, "Aucun maillage défini.");

            for (java.util.Map.Entry<String, MHDPersistence.MeshSpecDTO> e : meshSpecs.entrySet()){
                String dom = e.getKey();
                MHDPersistence.MeshSpecDTO ms = e.getValue();
                if (!domainNames.contains(dom))
                    r.add(SEC, "Maillage: domaine inconnu '"+dom+"'.");
                try {
                    Globals.MeshKind kind = Globals.MeshKind.valueOf(ms.kind);
                    switch (kind) {
                        case TRI_REG:
                        case RECT_REG:
                            if (ms.dx == null || ms.dy == null)
                                r.add(SEC, "Maillage '"+dom+"': dx/dy requis pour "+ms.kind+".");
                            break;
                        case TRI_GRAD:
                        case RECT_GRAD:
                            if (ms.dxmin==null || ms.dxmax==null || ms.dymin==null || ms.dymax==null)
                                r.add(SEC, "Maillage '"+dom+"': dxmin/dxmax/dymin/dymax requis pour "+ms.kind+".");
                            break;
                    }
                } catch (Exception ex) {
                    r.add(SEC, "Maillage '"+dom+"': type invalide '"+ms.kind+"'.");
                }
            }
        }

        return r;
    }

    private static boolean isEmpty(String s){ return s==null || s.trim().isEmpty(); }
}


void showMHDGui(
    java.util.Map<Step, java.util.function.Supplier<JPanel>> factories,
    java.util.function.BiFunction<JComponent,String,JPanel> wrapTitledFn
){
    JFrame f = new JFrame("Projet MHD — Interface");
    f.setDefaultCloseOperation(JFrame.EXIT_ON_CLOSE);
    f.setMinimumSize(new Dimension(1200, 800));

    JPanel root = new JPanel(new BorderLayout(12, 12));
    root.setBorder(new EmptyBorder(12, 12, 12, 12));
    f.setContentPane(root);

    // ====== Menu Fichier ======
    final File[] currentFile = { null };
    JMenuBar mb = new JMenuBar();
    JMenu mf = new JMenu("Fichier");
    JMenuItem miNew  = new JMenuItem("Nouveau");
    JMenuItem miOpen = new JMenuItem("Ouvrir…");
    JMenuItem miSave = new JMenuItem("Enregistrer");
    JMenuItem miSaveAs = new JMenuItem("Enregistrer sous…");
    JMenuItem miExportJson = new JMenuItem("Exporter JSON…");
    JMenuItem miQuit = new JMenuItem("Quitter");

    miNew.addActionListener(e -> {
        MHDPersistence.ProjectState s = new MHDPersistence.ProjectState();
        s.applyToGlobals();
        currentFile[0] = null;
        f.setTitle("Projet MHD — Interface");
    });
    miOpen.addActionListener(e -> {
        File opened = MHDPersistence.ProjectIO.openWithChooser(f);
        if (opened != null) {
            currentFile[0] = opened;
            f.setTitle("Projet MHD — " + opened.getName());
        }
    });
    miSave.addActionListener(e -> {
        File saved = MHDPersistence.ProjectIO.saveOrSaveAs(f, currentFile[0]);
        if (saved != null) {
            currentFile[0] = saved;
            f.setTitle("Projet MHD — " + saved.getName());
        }
    });
    miSaveAs.addActionListener(e -> {
        File savedAs = MHDPersistence.ProjectIO.saveAsWithChooser(f);
        if (savedAs != null) {
            currentFile[0] = savedAs;
            f.setTitle("Projet MHD — " + savedAs.getName());
        }
    });
    miExportJson.addActionListener(e -> MHDPersistence.ProjectIOJson.saveJsonWithChooser(f));
    miQuit.setAccelerator(KeyStroke.getKeyStroke(
        KeyEvent.VK_Q, Toolkit.getDefaultToolkit().getMenuShortcutKeyMaskEx()));
    miQuit.addActionListener(e -> f.dispose());

    mf.add(miNew); mf.add(miOpen);
    mf.addSeparator();
    mf.add(miSave); mf.add(miSaveAs);
    mf.addSeparator();
    mf.add(miExportJson);
    mf.addSeparator();
    mf.add(miQuit);
    mb.add(mf);
    f.setJMenuBar(mb);

    // Barre haut
    JPanel top = new JPanel(new FlowLayout(FlowLayout.LEFT, 8, 0));
    top.add(new JLabel("Système de coordonnées :"));
    JComboBox<String> cbCoord = new JComboBox<>(new String[]{
        "2D cartésien","2D axisymétrique (r,z)","2D polaire (r,θ)","q2D stellerator"
    });
    cbCoord.setSelectedIndex(0);
    cbCoord.addActionListener(e -> {
        int i = cbCoord.getSelectedIndex();
        Globals.setCoordSys(
            switch (i){
                case 1 -> Globals.CoordSys.AXISYM_2D;
                case 2 -> Globals.CoordSys.POLAR_2D;
                case 3 -> Globals.CoordSys.Q2D_STELLARATOR;
                default -> Globals.CoordSys.CARTESIAN_2D;
            }
        );
    });
    top.add(cbCoord);
    root.add(top, BorderLayout.NORTH);

    // Liste des étapes
    DefaultListModel<Step> m = new DefaultListModel<>();
    for (Step s : Step.values()) m.addElement(s);
    JList<Step> steps = new JList<>(m);
    steps.setSelectionMode(ListSelectionModel.SINGLE_SELECTION);
    steps.setSelectedIndex(0);
    steps.setFixedCellHeight(28);
    try { steps.setCellRenderer(new BoldSectionRenderer()); } catch (Throwable ignore) {}

    JPanel left = new JPanel(new BorderLayout());
    left.add(new JLabel("  Étapes"), BorderLayout.NORTH);
    left.add(new JScrollPane(steps), BorderLayout.CENTER);
    left.setPreferredSize(new Dimension(220, 0));
    root.add(left, BorderLayout.WEST);

    // Cartes au centre : création paresseuse + cache
    JPanel cards = new JPanel(new CardLayout());
    java.util.Map<Step, JComponent> cache = new java.util.LinkedHashMap<>();
    for (Step s : Step.values()) cards.add(new JPanel(), s.name());
    root.add(cards, BorderLayout.CENTER);

    Runnable showStep = () -> {
        Step sel = steps.getSelectedValue();
        if (sel == null) return;
        if (!cache.containsKey(sel)) {
            java.util.function.Supplier<JPanel> sup = factories.get(sel);   // <— QUALIFIÉ
            JPanel panel = (sup != null) ? sup.get() : new JPanel();
            String title = sel.toString();
            JComponent wrapped = (wrapTitledFn != null) ? wrapTitledFn.apply(panel, title) : panel;
            cache.put(sel, wrapped);
            cards.add(wrapped, sel.name()); // remplace placeholder
        }
        ((CardLayout) cards.getLayout()).show(cards, sel.name());
        cards.revalidate();
        cards.repaint();
        SwingUtilities.invokeLater(() -> cache.get(sel).requestFocusInWindow());
    };

    // Bas : bouton RUN (pré-vérification Java)
    JButton run = new JButton("LANCER CALCUL");
    run.addActionListener(ae -> {
        MHDPersistence.ProjectState snap = MHDPersistence.ProjectState.fromGlobals();
        Preflight.Result res = Preflight.validate(snap);
        if (!res.ok()) {
            StringBuilder msg = new StringBuilder(1024);
            for (var e : res.bySection.entrySet()){
                if (e.getValue().isEmpty()) continue;
                msg.append("■ ").append(e.getKey()).append("\n");
                for (String line : e.getValue()) msg.append("  – ").append(line).append('\n');
                msg.append('\n');
            }
            javax.swing.JOptionPane.showMessageDialog(
                f, msg.toString(), "Vérification avant calcul — erreurs détectées",
                javax.swing.JOptionPane.ERROR_MESSAGE
            );
            return;
        }
        javax.swing.JOptionPane.showMessageDialog(
            f, "Pré-vérification Java OK.\n(Étape suivante : appel du script Python de validation stricte + solveur.)",
            "Info", javax.swing.JOptionPane.INFORMATION_MESSAGE
        );
    });

    JPanel bottom = new JPanel(new BorderLayout());
    bottom.add(run, BorderLayout.EAST);
    root.add(bottom, BorderLayout.SOUTH);

    f.pack();
    f.setLocationRelativeTo(null);
    f.setVisible(true);

    showStep.run();
    steps.addListSelectionListener(e -> { if (!e.getValueIsAdjusting()) showStep.run(); });
}


In [6]:
// Imports utiles (si pas déjà faits)
import javax.swing.*;
import java.awt.*;
import java.util.*;
import java.util.function.*;

// 1) Anti-headless, utile dans certains REPL/IDE
System.setProperty("java.awt.headless", "false");

// 2) Handler global pour voir toute exception au lancement
Thread.setDefaultUncaughtExceptionHandler((t, ex) -> {
    ex.printStackTrace();
    try {
        JOptionPane.showMessageDialog(null, ex.toString(), "Erreur non interceptée", JOptionPane.ERROR_MESSAGE);
    } catch (Throwable ignore) {}
});

// 3) (Optionnel) Look&Feel natif
try { UIManager.setLookAndFeel(UIManager.getSystemLookAndFeelClassName()); } catch (Exception ignore) {}

// 4) Lancement sur l'EDT avec factories + wrap
SwingUtilities.invokeLater(() -> {
    try {
        // Map des écrans (adapte les noms aux tiennes si différent)
        Map<Step, Supplier<JPanel>> factories = new LinkedHashMap<>();
        factories.put(Step.PARAMETRES, () -> uiParametres());
        factories.put(Step.GEOMETRIE,  () -> uiGeometrie());
        factories.put(Step.DOMAINES,   () -> uiDomaines());
        factories.put(Step.MATERIAUX,  () -> uiMateriaux());
        factories.put(Step.CL,         () -> uiCL());
        factories.put(Step.CI,         () -> uiCI());
        factories.put(Step.SOLVEUR,    () -> uiSolveur());
        factories.put(Step.MAILLAGE,   () -> uiMaillage());

        // Emballage avec titre (utilise ta fonction existante)
        BiFunction<JComponent,String,JPanel> wrap = (comp, title) -> wrapTitled(comp, title);

        // Appel effectif : c'est ICI que la fenêtre est créée
        showMHDGui(factories, wrap);

        // Petit log pour vérifier qu'on est bien passé ici
        System.out.println("UI lancée (EDT=" + SwingUtilities.isEventDispatchThread() + 
                           ", headless=" + java.awt.GraphicsEnvironment.isHeadless() + ")");
    } catch (Throwable ex) {
        ex.printStackTrace();
        JOptionPane.showMessageDialog(null, ex.toString(), "Erreur au lancement", JOptionPane.ERROR_MESSAGE);
    }
});
